This program is used to compute $\mathscr{D}(u)$ of exceptional types. 

In [1]:
# Import the python version of CHEVIE-GAP by Meinolf Geck
from chv1r6180 import *

# Import other packages
import numpy as np
import pandas as pd
import ast
import itertools

                                                                     
  ###################################################################
  ##  A PYTHON VERSION OF CHEVIE-GAP FOR (FINITE) COXETER GROUPS   ##
  ##      (by Meinolf Geck,  version 1r6p180, 27 Jan 2014)         ##
  ##                                                               ##
  ##  To get started type "help(coxeter)" or "help(allfunctions)"; ##
  ##  see also http://dx.doi.org/10.1112/S1461157012001064.        ##
  ##  For notes about this version type  "versioninfo(1.6)".       ##
  ##  Check www.mathematik.uni-stuttgart.de/~geckmf for updates.   ##
  ##                                                               ##
  ##  Import into "sage" (4.7 or higher, www.sagemath.org) works.  ##
  ##                                                               ##
  ##       The proposed name for this module is "PyCox".           ##
  ##                  All comments welcome!                        ##
  ##################

In [2]:
# First we define some functions

# Function "minimal_W_cy" is used to give the minimal sets among some sets containing roots
def minimal_W_cy(Phi_cy_Set):
    Set=[]
    L=len(Phi_cy_Set)
    for i in range(L):
        k=0
        for j in range(L):
            if set(Phi_cy_Set[i])>set(Phi_cy_Set[j]):
                k=1
                break
            if set(Phi_cy_Set[i])==set(Phi_cy_Set[j]) and i>j:
                k=1
                break
        if k==0:
            Set.append(Phi_cy_Set[i])
    return Set

# Function "max_orbit" is used to choose the maximal orbits among some orbits
def max_orbit(orbit_set,Data):
    max_orbit_set=[]
    for O in orbit_set:
        j=0
        for O_1 in orbit_set:
            if O_1 in list((Data.loc[Data['orbit']==O]).loc[:,'Georbit'])[0]:
                j+=1
        if j==1:
            max_orbit_set.append(O)
    return(max_orbit_set)


def convert_fuc(W1_roots,W2_roots,Index,dict):
    length=len(W2_roots[0])
    if Index in range(length):
        return(dict.get(Index))
    Root_W2=W2_roots[Index]
    Root_W1=np.zeros(length)
    for i in range(length):
        Root_W1+=np.array(W1_roots[dict.get(i)])*Root_W2[i]
    return(W1_roots.index(tuple([int(k) for k in list(Root_W1)])))
    
# Function "switch_A3_D3" is used to switch the charnames between A3 and D3.
def switch_A3_D3(rep,target_form):
    rep=list(rep)
    dict_index_A3=['[4]','[3, 1]','[2, 2]','[2, 1, 1]','[1, 1, 1, 1]']
    dict_A3_D3={'[4]':'[[], [3]]','[3, 1]':'[[1], [2]]','[2, 2]':'[[], [2, 1]]','[2, 1, 1]':'[[1], [1, 1]]','[1, 1, 1, 1]':'[[], [1, 1, 1]]'}
    dict_index_D3=['[[], [3]]','[[1], [2]]','[[], [2, 1]]','[[1], [1, 1]]','[[], [1, 1, 1]]']
    dict_D3_A3={'[[], [3]]':'[4]','[[1], [2]]':'[3, 1]','[[], [2, 1]]':'[2, 2]','[[1], [1, 1]]':'[2, 1, 1]','[[], [1, 1, 1]]':'[1, 1, 1, 1]'}
    for position in range(len(rep)):
        if rep[position] in dict_index_A3:
            rep1=rep[position]
            rep2=dict_A3_D3.get(rep1)
            rep[position]=rep2
    if tuple(rep) in target_form:
        return tuple(rep)
    for position in range(len(rep)):
        if rep[position] in dict_index_D3:
            rep1=rep[position]
            rep2=dict_D3_A3.get(rep1)
            rep[position]=rep2
    if tuple(rep) in target_form:
        return tuple(rep)
    for position in range(1,len(rep)):
        if rep[position] in dict_index_A3:
            rep1=rep[position]
            rep2=dict_A3_D3.get(rep1)
            rep[position]=rep2
    if tuple(rep) in target_form:
        return tuple(rep)
    for position in range(len(rep)):
        if rep[position] in dict_index_D3:
            rep1=rep[position]
            rep2=dict_D3_A3.get(rep1)
            rep[position]=rep2
    for position in range(len(rep)-1):
        if rep[position] in dict_index_A3:
            rep1=rep[position]
            rep2=dict_A3_D3.get(rep1)
            rep[position]=rep2
    return tuple(rep)

def dual_rep_F4(rep):
    dual_dict={('1_1',):('1_1',),('1_2',):('1_3',),('1_3',):('1_2',),('1_4',):('1_4',),('2_1',):('2_3',),('2_2',):('2_4',),('2_3',):('2_1',),('2_4',):('2_2',),('4_1',):('4_1',),('9_1',):('9_1',),('9_2',):('9_3',),('9_3',):('9_2',),('9_4',):('9_4',),('6_1',):('6_1',),('6_2',):('6_2',),('12',):('12',),('4_2',):('4_2',),('4_3',):('4_4',),('4_4',):('4_3',),('4_5',):('4_5',),('8_1',):('8_3',),('8_2',):('8_4',),('8_3',):('8_1',),('8_4',):('8_2',),('16',):('16',)}
    return dual_dict.get(rep)

def dual_rep_G2(rep):
    if rep==("phi_{1,3}'",):
        return ("phi_{1,3}''",)
    elif rep==("phi_{1,3}''",):
        return ("phi_{1,3}'",)
    else:
        return rep

$E_6$

In [3]:
# Prepare data of E6 which includes name of each representation, corresponding orbit via Springer correspondence and the orbit greater than or equal to it
E6_Data=pd.DataFrame({'charnames':["('1_p',)","('6_p',)","('20_p',)","('30_p',)","('64_p',)","('15_q',)","('60_p',)","('81_p',)","('24_p',)","('80_s',)","('60_s',)","('10_s',)",'("81_p\'",)','("60_p\'",)','("24_p\'",)','("64_p\'",)','("30_p\'",)','("15_q\'",)','("20_p\'",)','("6_p\'",)','("1_p\'",)'],
'orbit':['E6','E6(a1)','D5','E6(a3)','D5(a1)','A5','A4+A1','A4','D4','D4(a1)','A3+A1','2A2+A1','A3','A2+2A1','2A2','A2+A1','A2','3A1','2A1','A1','0'],
'Georbit':[['E6'], ['E6', 'E6(a1)'], ['E6', 'E6(a1)', 'D5'], ['E6', 'E6(a1)', 'D5', 'E6(a3)'], ['E6', 'E6(a1)', 'D5', 'E6(a3)', 'D5(a1)'], ['E6', 'E6(a1)', 'D5', 'E6(a3)', 'A5'], ['E6', 'E6(a1)', 'D5', 'E6(a3)', 'A5', 'D5(a1)', 'A4+A1'], ['E6', 'E6(a1)', 'D5', 'E6(a3)', 'A5', 'D5(a1)', 'A4+A1', 'A4'], ['E6', 'E6(a1)', 'D5', 'E6(a3)', 'D5(a1)', 'D4'], ['E6', 'E6(a1)', 'D5', 'E6(a3)', 'A5', 'D5(a1)', 'A4+A1', 'A4', 'D4', 'D4(a1)'], ['E6', 'E6(a1)', 'D5', 'E6(a3)', 'A5', 'D5(a1)', 'A4+A1', 'A4', 'D4', 'D4(a1)', 'A3+A1'], ['E6', 'E6(a1)', 'D5', 'E6(a3)', 'A5', 'D5(a1)', 'A4+A1', 'A4', 'D4', 'D4(a1)', 'A3+A1', '2A2+A1'], ['E6', 'E6(a1)', 'D5', 'E6(a3)', 'A5', 'D5(a1)', 'A4+A1', 'A4', 'D4', 'D4(a1)', 'A3+A1', 'A3'], ['E6', 'E6(a1)', 'D5', 'E6(a3)', 'A5', 'D5(a1)', 'A4+A1', 'A4', 'D4', 'D4(a1)', 'A3+A1', 'A3', '2A2+A1', 'A2+2A1'], ['E6', 'E6(a1)', 'D5', 'E6(a3)', 'A5', 'D5(a1)', 'A4+A1', 'A4', 'D4', 'D4(a1)', 'A3+A1', '2A2+A1', '2A2'], ['E6', 'E6(a1)', 'D5', 'E6(a3)', 'A5', 'D5(a1)', 'A4+A1', 'A4', 'D4', 'D4(a1)', 'A3+A1', '2A2+A1', '2A2', 'A3', 'A2+2A1', 'A2+A1'], ['E6', 'E6(a1)', 'D5', 'E6(a3)', 'A5', 'D5(a1)', 'A4+A1', 'A4', 'D4', 'D4(a1)', 'A3+A1', '2A2+A1', '2A2', 'A3', 'A2+2A1', 'A2+A1', 'A2'], ['E6', 'E6(a1)', 'D5', 'E6(a3)', 'A5', 'D5(a1)', 'A4+A1', 'A4', 'D4', 'D4(a1)', 'A3+A1', '2A2+A1', '2A2', 'A3', 'A2+2A1', 'A2+A1', 'A2', '3A1'], ['E6', 'E6(a1)', 'D5', 'E6(a3)', 'A5', 'D5(a1)', 'A4+A1', 'A4', 'D4', 'D4(a1)', 'A3+A1', '2A2+A1', '2A2', 'A3', 'A2+2A1', 'A2+A1', 'A2', '3A1', '2A1'], ['E6', 'E6(a1)', 'D5', 'E6(a3)', 'A5', 'D5(a1)', 'A4+A1', 'A4', 'D4', 'D4(a1)', 'A3+A1', '2A2+A1', '2A2', 'A3', 'A2+2A1', 'A2+A1', 'A2', '3A1', '2A1', 'A1'], ['E6', 'E6(a1)', 'D5', 'E6(a3)', 'A5', 'D5(a1)', 'A4+A1', 'A4', 'D4', 'D4(a1)', 'A3+A1', '2A2+A1', '2A2', 'A3', 'A2+2A1', 'A2+A1', 'A2', '3A1', '2A1', 'A1', '0']]})

# example: first five lines of E6_Data
E6_Data.head()

,charnames,orbit,Georbit
0,"('1_p',)",E6,[E6]
1,"('6_p',)",E6(a1),"[E6, E6(a1)]"
2,"('20_p',)",D5,"[E6, E6(a1), D5]"
3,"('30_p',)",E6(a3),"[E6, E6(a1), D5, E6(a3)]"
4,"('64_p',)",D5(a1),"[E6, E6(a1), D5, E6(a3), D5(a1)]"


When $W_c$ is of type $E_6$, i.e. $c$ hyperspecial :

In [4]:
# Construct W of type E6
W=coxeter("E",6)

# Give the range of cover fold
for cover_fold in range(1,13):
    
    # Take the minimal Phi_cy sets 
    Phi_cy_Set=[]
    for y1,y2,y3,y4,y5,y6 in itertools.product(range(2),range(2),range(2),range(2),range(2),range(2)):
        y=np.array([y1,y2,y3,y4,y5,y6])
        if np.sum(y*np.array(W.roots[35]))<=cover_fold:
            Phi_cy=[]
            if y1==0:
                Phi_cy.append(0)
            if y2==0:
                Phi_cy.append(1)
            if y3==0:
                Phi_cy.append(2)
            if y4==0:
                Phi_cy.append(3)
            if y5==0:
                Phi_cy.append(4)
            if y6==0:
                Phi_cy.append(5)
            if np.sum(y*np.array(W.roots[35]))==cover_fold:
                Phi_cy.append(35)
            Phi_cy_Set.append(Phi_cy)
    min_Phi_cy_Set=minimal_W_cy(Phi_cy_Set)
    
    # In the following, we compute by going through all representations
    Orbit_Set=[]
    Induc_Wcy_W_set=[]
    W_cy_special_rep_set=[]
    for Phi_cy in min_Phi_cy_Set:
        W_cy=reflectionsubgroup(W,Phi_cy)
        W_cy_chartable=pd.DataFrame(chartable(W_cy))
        W_cy_special_rep=W_cy_chartable.loc[W_cy_chartable['b']==W_cy_chartable['a']]['charnames']
        W_cy_special_rep_set.append(W_cy_special_rep)
        induc_Wcy_W=inductiontable(W_cy,W,invchar=(lambda G:chartable(G, chars=False)['b']))
        Induc_Wcy_W=pd.DataFrame(induc_Wcy_W['scalar'],columns=induc_Wcy_W['charH'],index=induc_Wcy_W['charW'])
        Induc_Wcy_W_set.append(Induc_Wcy_W)
    for char in E6_Data.loc[:,'charnames']:
        orbit_set=[]
        for i in range(len(min_Phi_cy_Set)):
            Phi_cy=min_Phi_cy_Set[i]
            W_cy=reflectionsubgroup(W,Phi_cy)
            W_cy_chartable=pd.DataFrame(chartable(W_cy))
            W_cy_special_rep=W_cy_special_rep_set[i]
            Induc_Wcy_W=Induc_Wcy_W_set[i]
            for rep_now in W_cy_special_rep:
                j_induction_now=(Induc_Wcy_W[rep_now][Induc_Wcy_W[rep_now]==1].index)[0]
                if list((E6_Data.loc[E6_Data['charnames']==str(j_induction_now)]).loc[:,'orbit'])[0] in list((E6_Data.loc[E6_Data['charnames']==char]).loc[:,'Georbit'])[0]:
                    rep_sign_position=(W_cy_chartable.loc[W_cy_chartable['charnames']==rep_now]).loc[:,'permsgn'].iloc[0]
                    rep_sign=W_cy_chartable.iloc[rep_sign_position,0]
                    j_induction_sign=(Induc_Wcy_W[rep_sign][Induc_Wcy_W[rep_sign]==1].index)[0]
                    orbit_set.append(list((E6_Data.loc[E6_Data['charnames']==str(j_induction_sign)]).loc[:,'orbit'])[0])
        Orbit_Set.append(list(set(orbit_set)))
    Max_Orbit_Set=[]
    for x in Orbit_Set:
            Max_Orbit_Set.append(max_orbit(x,E6_Data))
    
    # Restore the results in E6_Data
    E6_Data[str("E6_E6_")+str(cover_fold)]=Max_Orbit_Set

When $W_c$ is of type $A_2A_2A_2$:

In [5]:
# Construct W of type E6 and Wc of type A2A2A2 with a cartan matrix
W1=coxeter("E",6)
W2=coxeter([[2,-1,0,0,0,0],[-1,2,0,0,0,0],[0,0,2,-1,0,0],[0,0,-1,2,0,0],[0,0,0,0,2,-1],[0,0,0,0,-1,2]])

# Record the saturation dictionary
Sat_dict={('[1, 1, 1]', '[1, 1, 1]', '[1, 1, 1]'):'0',('[1, 1, 1]', '[1, 1, 1]', '[2, 1]'):'A1',('[1, 1, 1]', '[1, 1, 1]', '[3]'):'A2',
          ('[1, 1, 1]', '[2, 1]', '[1, 1, 1]'):'A1', ('[1, 1, 1]', '[2, 1]', '[2, 1]'):'2A1', ('[1, 1, 1]', '[2, 1]', '[3]'):'A2+A1',
          ('[1, 1, 1]', '[3]', '[1, 1, 1]'):'A2',('[1, 1, 1]', '[3]', '[2, 1]'):'A2+A1',('[1, 1, 1]', '[3]', '[3]'):'2A2',
         ('[2, 1]', '[1, 1, 1]', '[1, 1, 1]'):'A1',('[2, 1]', '[1, 1, 1]', '[2, 1]'):'2A1',('[2, 1]', '[1, 1, 1]', '[3]'):'A2+A1',
          ('[2, 1]', '[2, 1]', '[1, 1, 1]'):'2A1',('[2, 1]', '[2, 1]', '[2, 1]'):'3A1',('[2, 1]', '[2, 1]', '[3]'):'A2+2A1',
         ('[2, 1]', '[3]', '[1, 1, 1]'):'A2+A1', ('[2, 1]', '[3]', '[2, 1]'):'A2+2A1',('[2, 1]', '[3]', '[3]'):'2A2+A1',
         ('[3]', '[1, 1, 1]', '[1, 1, 1]'):'A2',('[3]', '[1, 1, 1]', '[2, 1]'):'A2+A1', ('[3]', '[1, 1, 1]', '[3]'):'2A2',
          ('[3]', '[2, 1]', '[1, 1, 1]'):'A2+A1', ('[3]', '[2, 1]', '[2, 1]'):'A2+2A1', ('[3]', '[2, 1]', '[3]'):'2A2+A1',
         ('[3]', '[3]', '[1, 1, 1]'):'2A2',('[3]', '[3]', '[2, 1]'):'2A2+A1',('[3]', '[3]', '[3]'):'D4(a1)'}

weight=[1,2,2,1,2]
drop_weight=3
node_set_1=[list(p) for p in itertools.product(range(drop_weight+1), repeat=len(weight))]
node_set_2=[]
for node in node_set_1:
    c=(-np.sum(np.array(node)*weight)-1)%drop_weight
    node_set_2.append(np.append(node,c))
    if c==0:
        node_set_2.append(np.append(node,drop_weight))

for cover_fold in range(1,5):
    node_set_3=[]
    for node in node_set_2:
        if node[0]+node[1]<=cover_fold and node[2]+node[3]<=cover_fold and node[4]+node[5]<=cover_fold:
            add_point=[]
            for i in range(6):
                if node[i]==0:
                    add_point.append(i)
            if node[0]+node[1]==cover_fold:
                add_point.append(W2.roots.index(tuple(np.array(W2.roots[0])+np.array(W2.roots[1]))))
            if node[2]+node[3]==cover_fold:
                add_point.append(W2.roots.index(tuple(np.array(W2.roots[2])+np.array(W2.roots[3]))))
            if node[4]+node[5]==cover_fold:
                add_point.append(W2.roots.index(tuple(np.array(W2.roots[4])+np.array(W2.roots[5]))))
            node_set_3.append(add_point)
    
    node_set_3=[list(tpl) for tpl in {tuple(item) for item in node_set_3}]
    minimal_node_set_W2=minimal_W_cy(node_set_3)
    
    dict={0:0,1:2,2:4,3:5,4:1,5:71}
    minimal_node_set_W1=[]
    for node in minimal_node_set_W2:
        minimal_node_set_W1.append([convert_fuc(W1.roots,W2.roots,Index,dict) for Index in node])

    Target=[]
    Induc_W1cy_W1_set=[]
    Induc_W2cy_W2_set=[]
    W1_cy_special_rep_set=[]
    for i in range(len(minimal_node_set_W1)):
        Phi1_cy=minimal_node_set_W1[i]
        Phi2_cy=minimal_node_set_W2[i]
        W1_cy=reflectionsubgroup(W1,Phi1_cy)
        W2_cy=reflectionsubgroup(W2,Phi2_cy)
        W1_cy_chartable=pd.DataFrame(chartable(W1_cy))
        W1_cy_special_rep=W1_cy_chartable.loc[W1_cy_chartable['b']==W1_cy_chartable['a']]['charnames']
        W1_cy_special_rep_set.append(W1_cy_special_rep)
        induc_W1cy_W1=inductiontable(W1_cy,W1,invchar=(lambda G:chartable(G, chars=False)['b']))
        Induc_W1cy_W1=pd.DataFrame(induc_W1cy_W1['scalar'],columns=induc_W1cy_W1['charH'],index=induc_W1cy_W1['charW'])
        Induc_W1cy_W1_set.append(Induc_W1cy_W1)
        induc_W2cy_W2=inductiontable(W2_cy,W2,invchar=(lambda G:chartable(G, chars=False)['b']))
        Induc_W2cy_W2=pd.DataFrame(induc_W2cy_W2['scalar'],columns=induc_W2cy_W2['charH'],index=induc_W2cy_W2['charW'])
        Induc_W2cy_W2_set.append(Induc_W2cy_W2)
    for char in E6_Data.loc[:,'charnames']:
        j_induction_sign_set=[]
        for i in range(len(minimal_node_set_W1)):
            Phi1_cy=minimal_node_set_W1[i]
            Phi2_cy=minimal_node_set_W2[i]
            W1_cy=reflectionsubgroup(W1,Phi1_cy)
            W2_cy=reflectionsubgroup(W2,Phi2_cy)
            W1_cy_chartable=pd.DataFrame(chartable(W1_cy))
            W1_cy_special_rep=W1_cy_special_rep_set[i]
            Induc_W1cy_W1=Induc_W1cy_W1_set[i]
            Induc_W2cy_W2=Induc_W2cy_W2_set[i]
            for rep_now in W1_cy_special_rep:
                j_induction_now=(Induc_W1cy_W1[rep_now][Induc_W1cy_W1[rep_now]==1].index)[0]
                if list((E6_Data.loc[E6_Data['charnames']==str(j_induction_now)]).loc[:,'orbit'])[0] in list((E6_Data.loc[E6_Data['charnames']==char]).loc[:,'Georbit'])[0]:
                    rep_sign_position=(W1_cy_chartable.loc[W1_cy_chartable['charnames']==rep_now]).loc[:,'permsgn'].iloc[0]
                    rep_sign=W1_cy_chartable.iloc[rep_sign_position,0]
                    j_induction_sign=(Induc_W2cy_W2[rep_sign][Induc_W2cy_W2[rep_sign]==1].index)[0]
                    j_induction_sign_set.append(j_induction_sign)
        Target.append(max_orbit(list(set([Sat_dict.get(x) for x in set(j_induction_sign_set)])),E6_Data))
    E6_Data["E6_A2*A2*A2_"+str(cover_fold)]=Target

When $W_c$ is of the type $A_1A_5$:

In [6]:
# Construct W of type E6 and Wc of type A1A5 with a cartan matrix
W1=coxeter("E",6)
W2=coxeter([[2, -1, 0, 0, 0,0],[-1, 2, -1, 0, 0,0],[0,-1, 2, -1, 0,0],[0, 0, -1, 2, -1,0],[0, 0, 0, -1, 2,0],[0,0,0,0,0,2]])

# Record the saturation dictionary
Sat_dict={('[1, 1, 1, 1, 1, 1]', '[1, 1]'):'0',('[1, 1, 1, 1, 1, 1]', '[2]'):'A1', ('[2, 1, 1, 1, 1]', '[1, 1]'):'A1', ('[2, 1, 1, 1, 1]', '[2]'):'2A1',
         ('[2, 2, 1, 1]', '[1, 1]'):'2A1', ('[2, 2, 1, 1]', '[2]'):'3A1', ('[2, 2, 2]', '[1, 1]'):'3A1',('[2, 2, 2]', '[2]'):'A2',
         ('[3, 1, 1, 1]', '[1, 1]'):'A2',('[3, 1, 1, 1]', '[2]'):'A2+A1',('[3, 2, 1]', '[1, 1]'):'A2+A1',('[3, 2, 1]', '[2]'):'A2+2A1',
         ('[3, 3]', '[1, 1]'):'2A2',('[3, 3]', '[2]'):'2A2+A1',('[4, 1, 1]', '[1, 1]'):'A3',('[4, 1, 1]', '[2]'):'A3+A1',
         ('[4, 2]', '[1, 1]'):'A3+A1',('[4, 2]', '[2]'):'D4(a1)', ('[5, 1]', '[1, 1]'):'A4', ('[5, 1]', '[2]'):'A4+A1',
         ('[6]', '[1, 1]'):'A5',('[6]', '[2]'):'E6(a3)'}

weight=[1,2,3,2,1]
drop_weight=2
node_set_1=[list(p) for p in itertools.product(range(drop_weight+1), repeat=len(weight))]
node_set_2=[]
for node in node_set_1:
    c=(-np.sum(np.array(node)*weight)-1)%drop_weight
    node_set_2.append(np.append(node,c))
    if c==0:
        node_set_2.append(np.append(node,drop_weight))

for cover_fold in range(1,7):
    node_set_3=[]
    for node in node_set_2:
        if node[0]+node[1]+node[2]+node[3]+node[4]<=cover_fold and node[5]<=cover_fold:
            add_point=[]
            for i in range(6):
                if node[i]==0:
                    add_point.append(i)
            if node[0]+node[1]+node[2]+node[3]+node[4]==cover_fold:
                add_point.append(W2.roots.index(tuple(np.array(W2.roots[0])+np.array(W2.roots[1])+np.array(W2.roots[2])+np.array(W2.roots[3])+np.array(W2.roots[4]))))
            if node[5]==cover_fold:
                add_point.append(W2.roots.index(tuple(np.array(W2.roots[5]))))
            node_set_3.append(add_point)
    
    node_set_3=[list(tpl) for tpl in {tuple(item) for item in node_set_3}]
    minimal_node_set_W2=minimal_W_cy(node_set_3)
    
    dict={0:0,1:2,2:3,3:4,4:5,5:71}
    minimal_node_set_W1=[]
    for node in minimal_node_set_W2:
        minimal_node_set_W1.append([convert_fuc(W1.roots,W2.roots,Index,dict) for Index in node])

    Target=[]
    Induc_W1cy_W1_set=[]
    Induc_W2cy_W2_set=[]
    W1_cy_special_rep_set=[]
    for i in range(len(minimal_node_set_W1)):
        Phi1_cy=minimal_node_set_W1[i]
        Phi2_cy=minimal_node_set_W2[i]
        W1_cy=reflectionsubgroup(W1,Phi1_cy)
        W2_cy=reflectionsubgroup(W2,Phi2_cy)
        W1_cy_chartable=pd.DataFrame(chartable(W1_cy))
        W1_cy_special_rep=W1_cy_chartable.loc[W1_cy_chartable['b']==W1_cy_chartable['a']]['charnames']
        W1_cy_special_rep_set.append(W1_cy_special_rep)
        induc_W1cy_W1=inductiontable(W1_cy,W1,invchar=(lambda G:chartable(G, chars=False)['b']))
        Induc_W1cy_W1=pd.DataFrame(induc_W1cy_W1['scalar'],columns=induc_W1cy_W1['charH'],index=induc_W1cy_W1['charW'])
        Induc_W1cy_W1_set.append(Induc_W1cy_W1)
        induc_W2cy_W2=inductiontable(W2_cy,W2,invchar=(lambda G:chartable(G, chars=False)['b']))
        Induc_W2cy_W2=pd.DataFrame(induc_W2cy_W2['scalar'],columns=induc_W2cy_W2['charH'],index=induc_W2cy_W2['charW'])
        Induc_W2cy_W2_set.append(Induc_W2cy_W2)
    for char in E6_Data.loc[:,'charnames']:
        j_induction_sign_set=[]
        for i in range(len(minimal_node_set_W1)):
            Phi1_cy=minimal_node_set_W1[i]
            Phi2_cy=minimal_node_set_W2[i]
            W1_cy=reflectionsubgroup(W1,Phi1_cy)
            W2_cy=reflectionsubgroup(W2,Phi2_cy)
            W1_cy_chartable=pd.DataFrame(chartable(W1_cy))
            W1_cy_special_rep=W1_cy_special_rep_set[i]
            Induc_W1cy_W1=Induc_W1cy_W1_set[i]
            Induc_W2cy_W2=Induc_W2cy_W2_set[i]
            for rep_now in W1_cy_special_rep:
                j_induction_now=(Induc_W1cy_W1[rep_now][Induc_W1cy_W1[rep_now]==1].index)[0]
                if list((E6_Data.loc[E6_Data['charnames']==str(j_induction_now)]).loc[:,'orbit'])[0] in list((E6_Data.loc[E6_Data['charnames']==char]).loc[:,'Georbit'])[0]:
                    rep_sign_position=(W1_cy_chartable.loc[W1_cy_chartable['charnames']==rep_now]).loc[:,'permsgn'].iloc[0]
                    rep_sign=W1_cy_chartable.iloc[rep_sign_position,0]
                    j_induction_sign=(Induc_W2cy_W2[rep_sign][Induc_W2cy_W2[rep_sign]==1].index)[0]
                    j_induction_sign_set.append(j_induction_sign)
        Target.append(max_orbit(list(set([Sat_dict.get(x) for x in set(j_induction_sign_set)])),E6_Data))
    E6_Data["E6_A1*A5_"+str(cover_fold)]=Target

In [7]:
# One can check the results in this Data
E6_Data

,charnames,orbit,Georbit,E6_E6_1,E6_E6_2,E6_E6_3,E6_E6_4,E6_E6_5,E6_E6_6,E6_E6_7,...,E6_A2*A2*A2_1,E6_A2*A2*A2_2,E6_A2*A2*A2_3,E6_A2*A2*A2_4,E6_A1*A5_1,E6_A1*A5_2,E6_A1*A5_3,E6_A1*A5_4,E6_A1*A5_5,E6_A1*A5_6
0,"('1_p',)",E6,[E6],[0],[3A1],[2A2+A1],[D4(a1)],[A4+A1],[E6(a3)],[E6(a3)],...,[0],[3A1],[2A2+A1],[D4(a1)],[0],[3A1],[2A2+A1],[D4(a1)],[A4+A1],[E6(a3)]
1,"('6_p',)",E6(a1),"[E6, E6(a1)]",[A1],[A2+2A1],[D4(a1)],[D5(a1)],[E6(a3)],[D5],[E6(a1)],...,[A1],[A2+2A1],[D4(a1)],[D4(a1)],[A1],[A2+2A1],[D4(a1)],[A4+A1],[E6(a3)],[E6(a3)]
2,"('20_p',)",D5,"[E6, E6(a1), D5]",[2A1],[A3+A1],[A4+A1],[E6(a3)],[D5],[E6(a1)],[E6(a1)],...,[2A1],[2A2+A1],[D4(a1)],[D4(a1)],[2A1],[A3+A1],[A4+A1],[E6(a3)],[E6(a3)],[E6(a3)]
3,"('30_p',)",E6(a3),"[E6, E6(a1), D5, E6(a3)]",[A2],[D4(a1)],[E6(a3)],[D5],[E6(a1)],[E6],[E6],...,[A2],[D4(a1)],[D4(a1)],[D4(a1)],[A2],[D4(a1)],[E6(a3)],[E6(a3)],[E6(a3)],[E6(a3)]
4,"('64_p',)",D5(a1),"[E6, E6(a1), D5, E6(a3), D5(a1)]",[A2+A1],[A4+A1],[E6(a3)],[E6(a1)],[E6(a1)],[E6],[E6],...,[A2+A1],[D4(a1)],[D4(a1)],[D4(a1)],[A2+A1],[A4+A1],[E6(a3)],[E6(a3)],[E6(a3)],[E6(a3)]
5,"('15_q',)",A5,"[E6, E6(a1), D5, E6(a3), A5]",[A2],[D4],[E6(a3)],[D5],[E6(a1)],[E6],[E6],...,[A2],[D4(a1)],[D4(a1)],[D4(a1)],[A2],[D4(a1)],[E6(a3)],[E6(a3)],[E6(a3)],[E6(a3)]
6,"('60_p',)",A4+A1,"[E6, E6(a1), D5, E6(a3), A5, D5(a1), A4+A1]",[A2+2A1],[D5(a1)],[D5],[E6(a1)],[E6],[E6],[E6],...,[A2+2A1],[D4(a1)],[D4(a1)],[D4(a1)],[A2+2A1],[A4+A1],[E6(a3)],[E6(a3)],[E6(a3)],[E6(a3)]
7,"('81_p',)",A4,"[E6, E6(a1), D5, E6(a3), A5, D5(a1), A4+A1, A4]",[A3],[D5(a1)],[D5],[E6(a1)],[E6],[E6],[E6],...,[A2+2A1],[D4(a1)],[D4(a1)],[D4(a1)],[A3],[A4+A1],[E6(a3)],[E6(a3)],[E6(a3)],[E6(a3)]
8,"('24_p',)",D4,"[E6, E6(a1), D5, E6(a3), D5(a1), D4]",[2A2],[A5],[E6(a3)],[E6(a1)],[E6(a1)],[E6],[E6],...,[2A2],[D4(a1)],[D4(a1)],[D4(a1)],[2A2],[A5],[E6(a3)],[E6(a3)],[E6(a3)],[E6(a3)]
9,"('80_s',)",D4(a1),"[E6, E6(a1), D5, E6(a3), A5, D5(a1), A4+A1, A4...",[D4(a1)],[E6(a3)],[E6(a1)],[E6],[E6],[E6],[E6],...,[2A2+A1],[D4(a1)],[D4(a1)],[D4(a1)],[A3+A1],[E6(a3)],[E6(a3)],[E6(a3)],[E6(a3)],[E6(a3)]


$E_7$

In [8]:
E7_Data=pd.DataFrame({'charnames':["('1_a',)", '("7_a\'",)', "('27_a',)", '("56_a\'",)', '("21_b\'",)', "('35_b',)", "('120_a',)", '("189_b\'",)', "('210_a',)", "('105_b',)", "('168_a',)", '("315_a\'",)', '("189_c\'",)', "('280_b',)", "('405_a',)", '("70_a\'",)', '("216_a\'",)', '("378_a\'",)', "('210_b',)", "('420_a',)", '("512_a\'",)', "('105_c',)", "('84_a',)", '("420_a\'",)', '("210_b\'",)', "('378_a',)", '("405_a\'",)', '("105_c\'",)', "('315_a',)", "('216_a',)", '("280_b\'",)', "('70_a',)", "('189_c',)", '("210_a\'",)', '("168_a\'",)', '("105_b\'",)', "('189_b',)", '("120_a\'",)', "('15_a',)", "('56_a',)", '("35_b\'",)', "('21_b',)", '("27_a\'",)', "('7_a',)", '("1_a\'",)'],
                     'orbit':['E7', 'E7(a1)', 'E7(a2)', 'E7(a3)', 'E6', 'D6', 'E6(a1)', 'E7(a4)', 'D6(a1)', 'A6', 'D5+A1', 'E7(a5)', 'D5', 'D6(a2)', 'E6(a3)', 'A5+A1', "A5'", 'D5(a1)+A1', 'A4+A2', 'D5(a1)', 'A4+A1', "A5''", 'D4+A1', 'A4', 'A3+A2+A1', 'A3+A2', 'D4(a1)+A1', 'D4', 'D4(a1)', 'A3+2A1', "(A3+A1)'", '2A2+A1', "(A3+A1)''", 'A3', '2A2', 'A2+3A1', 'A2+2A1', 'A2+A1', '4A1', 'A2', "3A1'", "3A1''", '2A1', 'A1', '1'],
                     'Georbit':[['E7'], ['E7(a1)', 'E7'], ['E7(a2)', 'E7', 'E7(a1)'], ['E7(a3)', 'E7(a2)', 'E7', 'E7(a1)'], ['E6', 'E7(a2)', 'E7', 'E7(a1)'], ['D6', 'E7(a2)', 'E7', 'E7(a3)', 'E7(a1)'], ['E6(a1)', 'E7(a2)', 'E7', 'E7(a3)', 'E7(a1)', 'E6'], ['E7(a4)', 'D6', 'E7(a2)', 'E6(a1)', 'E7', 'E7(a3)', 'E7(a1)', 'E6'], ['D6(a1)', 'D6', 'E7(a2)', 'E6(a1)', 'E7', 'E7(a3)', 'E7(a1)', 'E6', 'E7(a4)'], ['A6', 'D6', 'E7(a2)', 'E6(a1)', 'E7', 'E7(a3)', 'E7(a1)', 'E6', 'E7(a4)'], ['D5+A1', 'D6', 'E7(a2)', 'E6(a1)', 'E7', 'E7(a3)', 'E7(a1)', 'E6', 'E7(a4)'], ['E7(a5)', 'D6', 'D5+A1', 'E7(a2)', 'E6(a1)', 'E7', 'E7(a3)', 'E7(a1)', 'E6', 'E7(a4)', 'D6(a1)', 'A6'], ['D5', 'D6', 'D5+A1', 'E7(a2)', 'E6(a1)', 'E7', 'E7(a3)', 'E7(a1)', 'E6', 'E7(a4)', 'D6(a1)'], ['D6(a2)', 'D6', 'D5+A1', 'E7(a5)', 'E7(a2)', 'E6(a1)', 'E7', 'E7(a3)', 'E7(a1)', 'E6', 'E7(a4)', 'D6(a1)', 'A6'], ['E6(a3)', 'D6', 'D5+A1', 'E7(a5)', 'E7(a2)', 'E6(a1)', 'E7', 'E7(a3)', 'D5', 'E7(a1)', 'E6', 'E7(a4)', 'D6(a1)', 'A6'], ['A5+A1', 'D6', 'D6(a2)', 'D5+A1', 'E7(a5)', 'E7(a2)', 'E6(a1)', 'E7', 'E7(a3)', 'E7(a1)', 'E6', 'E7(a4)', 'D6(a1)', 'A6'], ["A5'", 'D6', 'D6(a2)', 'D5+A1', 'E6(a3)', 'E7(a5)', 'E7(a2)', 'E6(a1)', 'E7', 'E7(a3)', 'D5', 'E7(a1)', 'E6', 'E7(a4)', 'D6(a1)', 'A6'], ['D5(a1)+A1', 'D6', 'D6(a2)', 'D5+A1', 'E6(a3)', 'E7(a5)', 'E7(a2)', 'E6(a1)', 'E7', 'E7(a3)', 'D5', 'E7(a1)', 'E6', 'E7(a4)', 'D6(a1)', 'A6'], ['A4+A2', 'D6', 'D6(a2)', "A5'", 'D5(a1)+A1', 'D5+A1', 'E6(a3)', 'E7(a5)', 'E7(a2)', 'A5+A1', 'E6(a1)', 'E7', 'E7(a3)', 'D5', 'E7(a1)', 'E6', 'E7(a4)', 'D6(a1)', 'A6'], ['D5(a1)', 'D6', 'D6(a2)', 'D5(a1)+A1', 'D5+A1', 'E6(a3)', 'E7(a5)', 'E7(a2)', 'E6(a1)', 'E7', 'E7(a3)', 'D5', 'E7(a1)', 'E6', 'E7(a4)', 'D6(a1)', 'A6'], ['A4+A1', 'D6', 'D6(a2)', "A5'", 'D5(a1)', 'D5(a1)+A1', 'D5+A1', 'E6(a3)', 'A4+A2', 'E7(a5)', 'E7(a2)', 'A5+A1', 'E6(a1)', 'E7', 'E7(a3)', 'D5', 'E7(a1)', 'E6', 'E7(a4)', 'D6(a1)', 'A6'], ["A5''", 'D6', 'D6(a2)', 'D5+A1', 'E7(a5)', 'E7(a2)', 'A5+A1', 'E6(a1)', 'E7', 'E7(a3)', 'E7(a1)', 'E6', 'E7(a4)', 'D6(a1)', 'A6'], ['D4+A1', 'D6', 'D6(a2)', 'D5(a1)', 'D5(a1)+A1', 'D5+A1', 'E6(a3)', 'E7(a5)', 'E7(a2)', 'E6(a1)', 'E7', 'E7(a3)', 'D5', 'E7(a1)', 'E6', 'E7(a4)', 'D6(a1)', 'A6'], ['A4', 'D6', 'D6(a2)', "A5'", "A5''", 'D5(a1)', 'D5(a1)+A1', 'D5+A1', 'E6(a3)', 'A4+A2', 'A4+A1', 'E7(a5)', 'E7(a2)', 'A5+A1', 'E6(a1)', 'E7', 'E7(a3)', 'D5', 'E7(a1)', 'E6', 'E7(a4)', 'D6(a1)', 'A6'], ['A3+A2+A1', 'D6', 'D6(a2)', "A5'", 'D5(a1)', 'D5(a1)+A1', 'D5+A1', 'E6(a3)', 'A4+A2', 'D4+A1', 'A4+A1', 'E7(a5)', 'E7(a2)', 'A5+A1', 'E6(a1)', 'E7', 'E7(a3)', 'D5', 'E7(a1)', 'E6', 'E7(a4)', 'D6(a1)', 'A6'], ['A3+A2', 'D6', 'D6(a2)', "A5'", 'A3+A2+A1', "A5''", 'D5(a1)', 'D5(a1)+A1', 'D5+A1', 'E6(a3)', 'A4', 'A4+A2', 'D4+A1', 'A4+A1', 'E7(a5)', 'E7(a2)', 'A5+A1', 'E6(a1)', 'E7', 'E7(a3)', 'D5', 'E7(a1)', 'E6', 'E7(a4)', 'D6(a1)', 'A6'], ['D4(a1)+A1', 'D6', 'D6(a2)', "A5'", 'A3+A2+A1', "A5''", 'D5(a1)', 'D5(a1)+A1', 'D5+A1', 'E6(a3)', 'A4', 'A4+A2', 'D4+A1', 'A4+A1', 'E7(a5)', 'E7(a2)', 'A5+A1', 'E6(a1)', 'A3+A2', 'E7', 'E7(a3)', 'D5', 'E7(a1)', 'E6', 'E7(a4)', 'D6(a1)', 'A6'], ['D4', 'D6', 'D6(a2)', 'D5(a1)', 'D5(a1)+A1', 'D5+A1', 'E6(a3)', 'D4+A1', 'E7(a5)', 'E7(a2)', 'E6(a1)', 'E7', 'E7(a3)', 'D5', 'E7(a1)', 'E6', 'E7(a4)', 'D6(a1)', 'A6'], ['D4(a1)', 'D6', 'D6(a2)', "A5'", 'A3+A2+A1', "A5''", 'D5(a1)', 'D4(a1)+A1', 'D5(a1)+A1', 'D5+A1', 'E6(a3)', 'A4', 'A4+A2', 'D4+A1', 'A4+A1', 'E7(a5)', 'E7(a2)', 'A5+A1', 'E6(a1)', 'A3+A2', 'E7', 'E7(a3)', 'D5', 'E7(a1)', 'E6', 'E7(a4)', 'D6(a1)', 'D4', 'A6'], ['A3+2A1', 'D6', 'D6(a2)', "A5'", 'A3+A2+A1', "A5''", 'D5(a1)', 'D4(a1)+A1', 'D5(a1)+A1', 'D5+A1', 'E6(a3)', 'A4', 'A4+A2', 'D4+A1', 'A4+A1', 'E7(a5)', 'E7(a2)', 'A5+A1', 'E6(a1)', 'A3+A2', 'E7', 'E7(a3)', 'D5', 'E7(a1)', 'E6', 'E7(a4)', 'D6(a1)', 'A6'], ["(A3+A1)'", 'D6', 'D6(a2)', "A5'", 'A3+A2+A1', "A5''", 'D5(a1)', 'D4(a1)+A1', 'D5(a1)+A1', 'D5+A1', 'E6(a3)', 'A4', 'A4+A2', 'A3+2A1', 'D4+A1', 'A4+A1', 'E7(a5)', 'E7(a2)', 'A5+A1', 'E6(a1)', 'A3+A2', 'E7', 'E7(a3)', 'D5', 'E7(a1)', 'E6', 'D4(a1)', 'E7(a4)', 'D6(a1)', 'D4', 'A6'], ['2A2+A1', 'D6', 'D6(a2)', "A5'", 'A3+A2+A1', "A5''", 'D5(a1)', 'D4(a1)+A1', 'D5(a1)+A1', 'D5+A1', "(A3+A1)'", 'E6(a3)', 'A4', 'A4+A2', 'A3+2A1', 'D4+A1', 'A4+A1', 'E7(a5)', 'E7(a2)', 'A5+A1', 'E6(a1)', 'A3+A2', 'E7', 'E7(a3)', 'D5', 'E7(a1)', 'E6', 'D4(a1)', 'E7(a4)', 'D6(a1)', 'D4', 'A6'], ["(A3+A1)''", 'D6', 'D6(a2)', "A5'", 'A3+A2+A1', "A5''", 'D5(a1)', 'D4(a1)+A1', 'D5(a1)+A1', 'D5+A1', 'E6(a3)', 'A4', 'A4+A2', 'A3+2A1', 'D4+A1', 'A4+A1', 'E7(a5)', 'E7(a2)', 'A5+A1', 'E6(a1)', 'A3+A2', 'E7', 'E7(a3)', 'D5', 'E7(a1)', 'E6', 'E7(a4)', 'D6(a1)', 'A6'], ['A3', 'D6', 'D6(a2)', "A5'", 'A3+A2+A1', "A5''", 'D5(a1)', 'D4(a1)+A1', 'D5(a1)+A1', 'D5+A1', "(A3+A1)'", 'E6(a3)', 'A4', 'A4+A2', 'A3+2A1', 'D4+A1', 'A4+A1', 'E7(a5)', 'E7(a2)', 'A5+A1', 'E6(a1)', 'A3+A2', 'E7', 'E7(a3)', 'D5', 'E7(a1)', 'E6', 'D4(a1)', "(A3+A1)''", 'E7(a4)', 'D6(a1)', 'D4', 'A6'], ['2A2', 'D6', 'D6(a2)', '2A2+A1', "A5'", 'A3+A2+A1', "A5''", 'D5(a1)', 'D4(a1)+A1', 'D5(a1)+A1', 'D5+A1', "(A3+A1)'", 'E6(a3)', 'A4', 'A4+A2', 'A3+2A1', 'D4+A1', 'A4+A1', 'E7(a5)', 'E7(a2)', 'A5+A1', 'E6(a1)', 'A3+A2', 'E7', 'E7(a3)', 'D5', 'E7(a1)', 'E6', 'D4(a1)', "(A3+A1)''", 'E7(a4)', 'D6(a1)', 'D4', 'A6'], ['A2+3A1', 'D6', 'D6(a2)', '2A2+A1', "A5'", 'A3+A2+A1', "A5''", 'D5(a1)', 'D4(a1)+A1', 'D5(a1)+A1', 'D5+A1', "(A3+A1)'", 'E6(a3)', 'A4', 'A4+A2', 'A3+2A1', 'D4+A1', 'A4+A1', 'E7(a5)', 'E7(a2)', 'A5+A1', 'E6(a1)', 'A3+A2', 'E7', 'E7(a3)', 'D5', 'E7(a1)', 'E6', 'D4(a1)', 'E7(a4)', 'D6(a1)', 'D4', 'A6'], ['A2+2A1', 'D6', 'A2+3A1', 'D6(a2)', '2A2+A1', "A5'", 'A3+A2+A1', "A5''", 'D5(a1)', 'D4(a1)+A1', 'D5(a1)+A1', 'D5+A1', "(A3+A1)'", 'A3', 'E6(a3)', 'A4', 'A4+A2', 'A3+2A1', 'D4+A1', 'A4+A1', 'E7(a5)', 'E7(a2)', 'A5+A1', 'E6(a1)', 'A3+A2', 'E7', 'E7(a3)', 'D5', 'E7(a1)', 'E6', 'D4(a1)', "(A3+A1)''", 'E7(a4)', '2A2', 'D6(a1)', 'D4', 'A6'], ['A2+A1', 'D6', 'A2+3A1', 'D6(a2)', '2A2+A1', "A5'", 'A3+A2+A1', "A5''", 'A2+2A1', 'D5(a1)', 'D4(a1)+A1', 'D5(a1)+A1', 'D5+A1', "(A3+A1)'", 'A3', 'E6(a3)', 'A4', 'A4+A2', 'A3+2A1', 'D4+A1', 'A4+A1', 'E7(a5)', 'E7(a2)', 'A5+A1', 'E6(a1)', 'A3+A2', 'E7', 'E7(a3)', 'D5', 'E7(a1)', 'E6', 'D4(a1)', "(A3+A1)''", 'E7(a4)', '2A2', 'D6(a1)', 'D4', 'A6'], ['4A1', 'D6', 'A2+3A1', 'D6(a2)', '2A2+A1', "A5'", 'A3+A2+A1', "A5''", 'A2+2A1', 'D5(a1)', 'D4(a1)+A1', 'D5(a1)+A1', 'D5+A1', "(A3+A1)'", 'A3', 'A2+A1', 'E6(a3)', 'A4', 'A4+A2', 'A3+2A1', 'D4+A1', 'A4+A1', 'E7(a5)', 'E7(a2)', 'A5+A1', 'E6(a1)', 'A3+A2', 'E7', 'E7(a3)', 'D5', 'E7(a1)', 'E6', 'D4(a1)', "(A3+A1)''", 'E7(a4)', '2A2', 'D6(a1)', 'D4', 'A6'], ['A2', 'D6', 'A2+3A1', 'D6(a2)', '2A2+A1', "A5'", 'A3+A2+A1', "A5''", 'A2+2A1', 'D5(a1)', 'D4(a1)+A1', 'D5(a1)+A1', 'D5+A1', "(A3+A1)'", 'A3', 'A2+A1', 'E6(a3)', 'A4', 'A4+A2', 'A3+2A1', 'D4+A1', 'A4+A1', 'E7(a5)', 'E7(a2)', 'A5+A1', 'E6(a1)', 'A3+A2', 'E7', 'E7(a3)', 'D5', 'E7(a1)', 'E6', 'D4(a1)', "(A3+A1)''", 'E7(a4)', '2A2', 'D6(a1)', 'D4', 'A6'], ["3A1'", 'D6', 'A2+3A1', 'D6(a2)', '2A2+A1', "A5'", 'A3+A2+A1', "A5''", 'A2+2A1', 'D5(a1)', 'D4(a1)+A1', 'D5(a1)+A1', 'D5+A1', "(A3+A1)'", 'A2', 'A3', '4A1', 'A2+A1', 'E6(a3)', 'A4', 'A4+A2', 'A3+2A1', 'D4+A1', 'A4+A1', 'E7(a5)', 'E7(a2)', 'A5+A1', 'E6(a1)', 'A3+A2', 'E7', 'E7(a3)', 'D5', 'E7(a1)', 'E6', 'D4(a1)', "(A3+A1)''", 'E7(a4)', '2A2', 'D6(a1)', 'D4', 'A6'], ["3A1''", 'D6', 'A2+3A1', 'D6(a2)', '2A2+A1', "A5'", 'A3+A2+A1', "A5''", 'A2+2A1', 'D5(a1)', 'D4(a1)+A1', 'D5(a1)+A1', 'D5+A1', "(A3+A1)'", 'A3', '4A1', 'A2+A1', 'E6(a3)', 'A4', 'A4+A2', 'A3+2A1', 'D4+A1', 'A4+A1', 'E7(a5)', 'E7(a2)', 'A5+A1', 'E6(a1)', 'A3+A2', 'E7', 'E7(a3)', 'D5', 'E7(a1)', 'E6', 'D4(a1)', "(A3+A1)''", 'E7(a4)', '2A2', 'D6(a1)', 'D4', 'A6'], ['2A1', 'D6', 'A2+3A1', 'D6(a2)', "3A1''", '2A2+A1', "A5'", 'A3+A2+A1', "A5''", 'A2+2A1', 'D5(a1)', 'D4(a1)+A1', 'D5(a1)+A1', 'D5+A1', "(A3+A1)'", 'A2', 'A3', '4A1', 'A2+A1', 'E6(a3)', 'A4', 'A4+A2', 'A3+2A1', 'D4+A1', 'A4+A1', 'E7(a5)', "3A1'", 'E7(a2)', 'A5+A1', 'E6(a1)', 'A3+A2', 'E7', 'E7(a3)', 'D5', 'E7(a1)', 'E6', 'D4(a1)', "(A3+A1)''", 'E7(a4)', '2A2', 'D6(a1)', 'D4', 'A6'], ['A1', 'D6', 'A2+3A1', 'D6(a2)', "3A1''", '2A2+A1', "A5'", 'A3+A2+A1', "A5''", 'A2+2A1', 'D5(a1)', 'D4(a1)+A1', 'D5(a1)+A1', 'D5+A1', "(A3+A1)'", '2A1', 'A2', 'A3', '4A1', 'A2+A1', 'E6(a3)', 'A4', 'A4+A2', 'A3+2A1', 'D4+A1', 'A4+A1', 'E7(a5)', "3A1'", 'E7(a2)', 'A5+A1', 'E6(a1)', 'A3+A2', 'E7', 'E7(a3)', 'D5', 'E7(a1)', 'E6', 'D4(a1)', "(A3+A1)''", 'E7(a4)', '2A2', 'D6(a1)', 'D4', 'A6'], ['1', 'D6', 'A2+3A1', 'D6(a2)', "3A1''", '2A2+A1', "A5'", 'A3+A2+A1', "A5''", 'A2+2A1', 'D5(a1)', 'D4(a1)+A1', 'D5(a1)+A1', 'D5+A1', "(A3+A1)'", '2A1', 'A2', 'A3', '4A1', 'A2+A1', 'E6(a3)', 'A4', 'A4+A2', 'A3+2A1', 'D4+A1', 'A4+A1', 'A1', 'E7(a5)', "3A1'", 'E7(a2)', 'A5+A1', 'E6(a1)', 'A3+A2', 'E7', 'E7(a3)', 'D5', 'E7(a1)', 'E6', 'D4(a1)', "(A3+A1)''", 'E7(a4)', '2A2', 'D6(a1)', 'D4', 'A6']]})

When $W_c$ is of type $E_7$:

In [9]:
# Construct W of type E7
W=coxeter("E",7)

# Give the range of cover fold
for cover_fold in range(1,19):
    
    # Take the minimal Phi_cy sets 
    Phi_cy_Set=[]
    for y1,y2,y3,y4,y5,y6,y7 in itertools.product(range(2),range(2),range(2),range(2),range(2),range(2),range(2)):
        y=np.array([y1,y2,y3,y4,y5,y6,y7])
        if np.sum(y*np.array(W.roots[62]))<=cover_fold:
            Phi_cy=[]
            if y1==0:
                Phi_cy.append(0)
            if y2==0:
                Phi_cy.append(1)
            if y3==0:
                Phi_cy.append(2)
            if y4==0:
                Phi_cy.append(3)
            if y5==0:
                Phi_cy.append(4)
            if y6==0:
                Phi_cy.append(5)
            if y7==0:
                Phi_cy.append(6)
            if np.sum(y*np.array(W.roots[62]))==cover_fold:
                Phi_cy.append(62)
            Phi_cy_Set.append(Phi_cy)
    min_Phi_cy_Set=minimal_W_cy(Phi_cy_Set)
    
    # In the following, we compute by going through all representations
    Orbit_Set=[]
    Induc_Wcy_W_set=[]
    W_cy_special_rep_set=[]
    for Phi_cy in min_Phi_cy_Set:
        W_cy=reflectionsubgroup(W,Phi_cy)
        W_cy_chartable=pd.DataFrame(chartable(W_cy))
        W_cy_special_rep=W_cy_chartable.loc[W_cy_chartable['b']==W_cy_chartable['a']]['charnames']
        W_cy_special_rep_set.append(W_cy_special_rep)
        induc_Wcy_W=inductiontable(W_cy,W,invchar=(lambda G:chartable(G, chars=False)['b']))
        Induc_Wcy_W=pd.DataFrame(induc_Wcy_W['scalar'],columns=induc_Wcy_W['charH'],index=induc_Wcy_W['charW'])
        Induc_Wcy_W_set.append(Induc_Wcy_W)
    for char in E7_Data.loc[:,'charnames']:
        orbit_set=[]
        for i in range(len(min_Phi_cy_Set)):
            Phi_cy=min_Phi_cy_Set[i]
            W_cy=reflectionsubgroup(W,Phi_cy)
            W_cy_chartable=pd.DataFrame(chartable(W_cy))
            W_cy_special_rep=W_cy_special_rep_set[i]
            Induc_Wcy_W=Induc_Wcy_W_set[i]
            for rep_now in W_cy_special_rep:
                j_induction_now=(Induc_Wcy_W[rep_now][Induc_Wcy_W[rep_now]==1].index)[0]
                if list((E7_Data.loc[E7_Data['charnames']==str(j_induction_now)]).loc[:,'orbit'])[0] in list((E7_Data.loc[E7_Data['charnames']==char]).loc[:,'Georbit'])[0]:
                    if rep_now==("512_a'",):
                        rep_sign=rep_now
                    else:
                        rep_sign_position=(W_cy_chartable.loc[W_cy_chartable['charnames']==rep_now]).loc[:,'permsgn'].iloc[0]
                        rep_sign=W_cy_chartable.iloc[rep_sign_position,0]
                    j_induction_sign=(Induc_Wcy_W[rep_sign][Induc_Wcy_W[rep_sign]==1].index)[0]
                    orbit_set.append(list((E7_Data.loc[E7_Data['charnames']==str(j_induction_sign)]).loc[:,'orbit'])[0])
        Orbit_Set.append(list(set(orbit_set)))
    Max_Orbit_Set=[]
    for x in Orbit_Set:
            Max_Orbit_Set.append(max_orbit(x,E7_Data))
    
    # Restore the results in E7_Data
    E7_Data[str("E7_E7_")+str(cover_fold)]=Max_Orbit_Set

When $W_c$ is of the type $A_1D_6$

In [10]:
# Construct W of type E7 and Wc of type A1D6 with a cartan matrix
W1=coxeter("E",7)
W2=coxeter([[2, 0, -1, 0, 0, 0,0], [0, 2, -1, 0, 0, 0,0],[-1, -1, 2, -1, 0, 0,0],[0, 0, -1, 2, -1, 0,0],[0, 0, 0, -1, 2, -1,0], [0, 0, 0, 0, -1, 2,0],[0,0,0,0,0,0,2]])

# Record the saturation dictionary
Sat_dict={ ('[[], [1, 1, 1, 1, 1, 1]]', '[1, 1]'):'1',('[[], [1, 1, 1, 1, 1, 1]]', '[2]'):'A1',('[[1], [1, 1, 1, 1, 1]]', '[1, 1]'):'A1',('[[1], [1, 1, 1, 1, 1]]', '[2]'):'2A1',
        ('[[], [2, 1, 1, 1, 1]]', '[1, 1]'):'2A1',('[[], [2, 1, 1, 1, 1]]', '[2]'):"3A1''",('[[1, 1], [1, 1, 1, 1]]', '[1, 1]'):'2A1', ('[[1, 1, 1], -]', '[1, 1]'):"3A1''",
         ('[[1, 1, 1], +]', '[1, 1]'):"3A1'",('[[1, 1], [1, 1, 1, 1]]', '[2]'):"3A1'",('[[1, 1, 1], -]', '[2]'):'4A1',('[[1, 1, 1], +]', '[2]'):'A2',
        ('[[], [2, 2, 1, 1]]', '[1, 1]'):"3A1'",('[[], [2, 2, 1, 1]]', '[2]'):'4A1', ('[[1], [2, 1, 1, 1]]', '[1, 1]'):'A2',('[[], [2, 2, 2]]', '[1, 1]'):'4A1',
         ('[[1], [2, 1, 1, 1]]', '[2]'):'A2+A1',('[[], [3, 1, 1, 1]]', '[1, 1]'):'A3',('[[], [2, 2, 2]]', '[2]'):'A2+A1',('[[], [3, 1, 1, 1]]', '[2]'):"(A3+A1)''",
        ('[[1, 1], [2, 1, 1]]', '[1, 1]'):'A2+A1',('[[1, 1], [2, 1, 1]]', '[2]'):'A2+2A1', ('[[1], [2, 2, 1]]', '[1, 1]'):'A2+2A1', ('[[1, 1], [2, 2]]', '[1, 1]'):'2A2',
         ('[[], [3, 2, 1]]', '[1, 1]'):"(A3+A1)'",('[[1], [2, 2, 1]]', '[2]'):'A2+3A1',('[[2], [2, 1, 1]]', '[1, 1]'):'A3',('[[2, 1], -]', '[1, 1]'):"(A3+A1)''",
         ('[[2, 1], +]', '[1, 1]'):"(A3+A1)'",('[[1, 1], [2, 2]]', '[2]'):"2A2+A1",('[[], [3, 2, 1]]', '[2]'):"A3+2A1",('[[2], [2, 1, 1]]', '[2]'):"(A3+A1)'",
         ('[[1], [3, 1, 1]]', '[1, 1]'):'D4(a1)',('[[2, 1], -]', '[2]'):'A3+2A1',('[[2, 1], +]', '[2]'):'D4(a1)',('[[1, 1], [3, 1]]', '[1, 1]'):'D4(a1)+A1',
         ('[[2], [2, 2]]', '[1, 1]'):'A3+2A1',('[[1], [3, 1, 1]]', '[2]'):'D4(a1)+A1',('[[], [4, 1, 1]]', '[1, 1]'):'D4',('[[1, 1], [3, 1]]', '[2]'):'A3+A2',
         ('[[2], [2, 2]]', '[2]'):'D4(a1)+A1',('[[1], [3, 2]]', '[1, 1]'):'A3+A2',('[[], [4, 1, 1]]', '[2]'):'D4+A1',('[[], [4, 2]]', '[1, 1]'):'D4+A1',
         ('[[1], [3, 2]]', '[2]'):'A3+A2+A1',('[[2], [3, 1]]', '[1, 1]'):'A4',('[[3], -]', '[1, 1]'):"A5''",('[[3], +]', '[1, 1]'):"A5'",
          ('[[2], [4]]', '[1, 1]'):'D6(a2)',('[[1], [4, 1]]', '[2]'):'D5(a1)+A1', ('[[], [5, 1]]', '[1, 1]'):'D5',('[[2], [4]]', '[2]'):'E7(a5)',
         ('[[], [4, 2]]', '[2]'):'D4+A1',('[[2], [3, 1]]', '[2]'):'A3+A2+A1',('[[1], [4, 1]]', '[1, 1]'):'A4',('[[3], -]', '[2]'):"A5+A1",
         ('[[3], +]', '[2]'):"E6(a3)",('[[1], [5]]', '[1, 1]'):'D6(a1)',('[[], [5, 1]]', '[2]'):'D5+A1',('[[1], [5]]', '[2]'):'E7(a4)',
         ('[[], [6]]', '[1, 1]'):'D6',('[[], [6]]', '[2]'):'E7(a3)'}

weight=[2,3,4,3,2,1]
drop_weight=2
node_set_1=[list(p) for p in itertools.product(range(drop_weight+1), repeat=len(weight))]
node_set_2=[]
for node in node_set_1:
    c=(-np.sum(np.array(node)*weight)-1)%drop_weight
    node_set_2.append(np.append(node,c))
    if c==0:
        node_set_2.append(np.append(node,drop_weight))

for cover_fold in range(1,11):
    node_set_3=[]
    for node in node_set_2:
        if node[0]+node[1]+2*node[2]+2*node[3]+2*node[4]+node[5]<=cover_fold and node[6]<=cover_fold:
            add_point=[]
            for i in range(7):
                if node[i]==0:
                    add_point.append(i)
            if node[0]+node[1]+2*node[2]+2*node[3]+2*node[4]+node[5]==cover_fold:
                add_point.append(30)
            if node[6]==cover_fold:
                add_point.append(6)
            node_set_3.append(add_point)
    
    node_set_3=[list(tpl) for tpl in {tuple(item) for item in node_set_3}]
    minimal_node_set_W2=minimal_W_cy(node_set_3)
    
    dict={0:1,1:2,2:3,3:4,4:5,5:6,6:125}
    minimal_node_set_W1=[]
    for node in minimal_node_set_W2:
        minimal_node_set_W1.append([convert_fuc(W1.roots,W2.roots,Index,dict) for Index in node])

    Target=[]
    Induc_W1cy_W1_set=[]
    Induc_W2cy_W2_set=[]
    W1_cy_special_rep_set=[]
    for i in range(len(minimal_node_set_W1)):
        Phi1_cy=minimal_node_set_W1[i]
        Phi2_cy=minimal_node_set_W2[i]
        W1_cy=reflectionsubgroup(W1,Phi1_cy)
        W2_cy=reflectionsubgroup(W2,Phi2_cy)
        W1_cy_chartable=pd.DataFrame(chartable(W1_cy))
        W1_cy_special_rep=W1_cy_chartable.loc[W1_cy_chartable['b']==W1_cy_chartable['a']]['charnames']
        W1_cy_special_rep_set.append(W1_cy_special_rep)
        induc_W1cy_W1=inductiontable(W1_cy,W1,invchar=(lambda G:chartable(G, chars=False)['b']))
        Induc_W1cy_W1=pd.DataFrame(induc_W1cy_W1['scalar'],columns=induc_W1cy_W1['charH'],index=induc_W1cy_W1['charW'])
        Induc_W1cy_W1_set.append(Induc_W1cy_W1)
        induc_W2cy_W2=inductiontable(W2_cy,W2,invchar=(lambda G:chartable(G, chars=False)['b']))
        Induc_W2cy_W2=pd.DataFrame(induc_W2cy_W2['scalar'],columns=induc_W2cy_W2['charH'],index=induc_W2cy_W2['charW'])
        Induc_W2cy_W2_set.append(Induc_W2cy_W2)
    for char in E7_Data.loc[:,'charnames']:
        j_induction_sign_set=[]
        for i in range(len(minimal_node_set_W1)):
            Phi1_cy=minimal_node_set_W1[i]
            Phi2_cy=minimal_node_set_W2[i]
            W1_cy=reflectionsubgroup(W1,Phi1_cy)
            W2_cy=reflectionsubgroup(W2,Phi2_cy)
            W1_cy_chartable=pd.DataFrame(chartable(W1_cy))
            W1_cy_special_rep=W1_cy_special_rep_set[i]
            Induc_W1cy_W1=Induc_W1cy_W1_set[i]
            Induc_W2cy_W2=Induc_W2cy_W2_set[i]
            for rep_now in W1_cy_special_rep:
                j_induction_now=(Induc_W1cy_W1[rep_now][Induc_W1cy_W1[rep_now]==1].index)[0]
                if list((E7_Data.loc[E7_Data['charnames']==str(j_induction_now)]).loc[:,'orbit'])[0] in list((E7_Data.loc[E7_Data['charnames']==char]).loc[:,'Georbit'])[0]:
                    rep_sign_position=(W1_cy_chartable.loc[W1_cy_chartable['charnames']==rep_now]).loc[:,'permsgn'].iloc[0]
                    rep_sign=W1_cy_chartable.iloc[rep_sign_position,0]
                    j_induction_sign=(Induc_W2cy_W2[rep_sign][Induc_W2cy_W2[rep_sign]==1].index)[0]
                    j_induction_sign_set.append(j_induction_sign)
        Target.append(max_orbit(list(set([Sat_dict.get(x) for x in set(j_induction_sign_set)])),E7_Data))
    E7_Data["E7_A1D6_"+str(cover_fold)]=Target

When $W_c$ is of the type $A_2A_5$

In [11]:
# Construct W of type E7 and Wc of type A2A5 with a cartan matrix
W1=coxeter("E",7)
W2=coxeter([[2, -1, 0, 0, 0,0,0],[-1, 2, -1, 0, 0,0,0],[0, -1, 2, -1, 0,0,0],[0, 0, -1, 2, -1,0,0],[0, 0, 0, -1, 2,0,0],[0,0,0,0,0,2,-1],[0,0,0,0,0,-1,2]])

# Record the saturation dictionary
Sat_dict={('[1, 1, 1, 1, 1, 1]', '[1, 1, 1]'):'1',('[1, 1, 1, 1, 1, 1]', '[2, 1]'):'A1',('[1, 1, 1, 1, 1, 1]', '[3]'):'A2',('[2, 1, 1, 1, 1]', '[1, 1, 1]'):'A1', ('[2, 1, 1, 1, 1]', '[2, 1]'):'2A1',('[2, 1, 1, 1, 1]', '[3]'):'A2+A1',
         ('[2, 2, 1, 1]', '[1, 1, 1]'):'2A1',('[2, 2, 1, 1]', '[2, 1]'):"3A1'",('[2, 2, 1, 1]', '[3]'):'A2+2A1',('[2, 2, 2]', '[1, 1, 1]'):"3A1''",('[2, 2, 2]', '[2, 1]'):'4A1', ('[2, 2, 2]', '[3]'):'A2+3A1',
         ('[3, 1, 1, 1]', '[1, 1, 1]'):'A2',('[3, 1, 1, 1]', '[2, 1]'):'A2+A1',('[3, 1, 1, 1]', '[3]'):'2A2',('[3, 2, 1]', '[1, 1, 1]'):'A2+A1',('[3, 2, 1]', '[2, 1]'):'A2+2A1',('[3, 2, 1]', '[3]'):'2A2+A1',
         ('[3, 3]', '[1, 1, 1]'):'2A2',('[3, 3]', '[2, 1]'):'2A2+A1',('[3, 3]', '[3]'):'D4(a1)',('[4, 1, 1]', '[1, 1, 1]'):'A3',('[4, 1, 1]', '[2, 1]'):"(A3+A1)'",('[4, 1, 1]', '[3]'):'A3+A2',
         ('[4, 2]', '[1, 1, 1]'):"(A3+A1)''",('[4, 2]', '[2, 1]'):'A3+2A1',('[4, 2]', '[3]'):'A3+A2+A1',('[5, 1]', '[1, 1, 1]'):'A4',('[5, 1]', '[2, 1]'):'A4+A1',('[5, 1]', '[3]'):'A4+A2',
         ('[6]', '[1, 1, 1]'):"A5''",('[6]', '[2, 1]'):'A5+A1',('[6]', '[3]'):'E7(a5)'}

weight=[2,4,3,2,1,2]
drop_weight=3
node_set_1=[list(p) for p in itertools.product(range(drop_weight+1), repeat=len(weight))]
node_set_2=[]
for node in node_set_1:
    c=(-np.sum(np.array(node)*weight)-1)%drop_weight
    node_set_2.append(np.append(node,c))
    if c==0:
        node_set_2.append(np.append(node,drop_weight))

for cover_fold in range(1,7):
    node_set_3=[]
    for node in node_set_2:
        if node[0]+node[1]+node[2]+node[3]+node[4]<=cover_fold and node[5]+node[6]<=cover_fold:
            add_point=[]
            for i in range(7):
                if node[i]==0:
                    add_point.append(i)
            if node[0]+node[1]+node[2]+node[3]+node[4]==cover_fold:
                add_point.append(17)
            if node[5]+node[6]==cover_fold:
                add_point.append(11)
            node_set_3.append(add_point)
    
    node_set_3=[list(tpl) for tpl in {tuple(item) for item in node_set_3}]
    minimal_node_set_W2=minimal_W_cy(node_set_3)
    
    dict={0:1,1:3,2:4,3:5,4:6,5:0,6:125}
    minimal_node_set_W1=[]
    for node in minimal_node_set_W2:
        minimal_node_set_W1.append([convert_fuc(W1.roots,W2.roots,Index,dict) for Index in node])

    Target=[]
    Induc_W1cy_W1_set=[]
    Induc_W2cy_W2_set=[]
    W1_cy_special_rep_set=[]
    for i in range(len(minimal_node_set_W1)):
        Phi1_cy=minimal_node_set_W1[i]
        Phi2_cy=minimal_node_set_W2[i]
        W1_cy=reflectionsubgroup(W1,Phi1_cy)
        W2_cy=reflectionsubgroup(W2,Phi2_cy)
        W1_cy_chartable=pd.DataFrame(chartable(W1_cy))
        W1_cy_special_rep=W1_cy_chartable.loc[W1_cy_chartable['b']==W1_cy_chartable['a']]['charnames']
        W1_cy_special_rep_set.append(W1_cy_special_rep)
        induc_W1cy_W1=inductiontable(W1_cy,W1,invchar=(lambda G:chartable(G, chars=False)['b']))
        Induc_W1cy_W1=pd.DataFrame(induc_W1cy_W1['scalar'],columns=induc_W1cy_W1['charH'],index=induc_W1cy_W1['charW'])
        Induc_W1cy_W1_set.append(Induc_W1cy_W1)
        induc_W2cy_W2=inductiontable(W2_cy,W2,invchar=(lambda G:chartable(G, chars=False)['b']))
        Induc_W2cy_W2=pd.DataFrame(induc_W2cy_W2['scalar'],columns=induc_W2cy_W2['charH'],index=induc_W2cy_W2['charW'])
        Induc_W2cy_W2_set.append(Induc_W2cy_W2)
    for char in E7_Data.loc[:,'charnames']:
        j_induction_sign_set=[]
        for i in range(len(minimal_node_set_W1)):
            Phi1_cy=minimal_node_set_W1[i]
            Phi2_cy=minimal_node_set_W2[i]
            W1_cy=reflectionsubgroup(W1,Phi1_cy)
            W2_cy=reflectionsubgroup(W2,Phi2_cy)
            W1_cy_chartable=pd.DataFrame(chartable(W1_cy))
            W1_cy_special_rep=W1_cy_special_rep_set[i]
            Induc_W1cy_W1=Induc_W1cy_W1_set[i]
            Induc_W2cy_W2=Induc_W2cy_W2_set[i]
            for rep_now in W1_cy_special_rep:
                j_induction_now=(Induc_W1cy_W1[rep_now][Induc_W1cy_W1[rep_now]==1].index)[0]
                if list((E7_Data.loc[E7_Data['charnames']==str(j_induction_now)]).loc[:,'orbit'])[0] in list((E7_Data.loc[E7_Data['charnames']==char]).loc[:,'Georbit'])[0]:
                    rep_sign_position=(W1_cy_chartable.loc[W1_cy_chartable['charnames']==rep_now]).loc[:,'permsgn'].iloc[0]
                    rep_sign=W1_cy_chartable.iloc[rep_sign_position,0]
                    j_induction_sign=(Induc_W2cy_W2[rep_sign][Induc_W2cy_W2[rep_sign]==1].index)[0]
                    j_induction_sign_set.append(j_induction_sign)
        Target.append(max_orbit(list(set([Sat_dict.get(x) for x in set(j_induction_sign_set)])),E7_Data))
    E7_Data["E7_A2A5_"+str(cover_fold)]=Target

When $W_c$ is of the type $2A_3+A_1$

In [12]:
# Construct W of type E7 and Wc of type 2A3+A1 with a cartan matrix
W1=coxeter("E",7)
W2=coxeter([[2, 0, 0, 0, 0,0,0],[0, 2, -1, 0, 0,0,0],[0, -1, 2, -1, 0,0,0],[0, 0, -1, 2, 0,0,0],[0, 0, 0, 0, 2,-1,0],[0,0,0,0,-1,2,-1],[0,0,0,0,0,-1,2]])

# Record the saturation dictionary
Sat_dict={('[1, 1, 1, 1]', '[1, 1, 1, 1]', '[1, 1]'):'1',('[1, 1, 1, 1]', '[1, 1, 1, 1]', '[2]'):'A1',('[1, 1, 1, 1]', '[2, 1, 1]', '[1, 1]'):'A1',('[1, 1, 1, 1]', '[2, 1, 1]', '[2]'):'2A1',
         ('[1, 1, 1, 1]', '[2, 2]', '[1, 1]'):'2A1',('[1, 1, 1, 1]', '[2, 2]', '[2]'):"3A1''",('[1, 1, 1, 1]', '[3, 1]', '[1, 1]'):'A2',('[1, 1, 1, 1]', '[3, 1]', '[2]'):'A2+A1',
         ('[1, 1, 1, 1]', '[4]', '[1, 1]'):'A3',('[1, 1, 1, 1]', '[4]', '[2]'):"(A3+A1)''",('[2, 1, 1]', '[1, 1, 1, 1]', '[1, 1]'):'A1',('[2, 1, 1]', '[1, 1, 1, 1]', '[2]'):'2A1',
         ('[2, 1, 1]', '[2, 1, 1]', '[1, 1]'):'2A1', ('[2, 1, 1]', '[2, 1, 1]', '[2]'):"3A1'",('[2, 1, 1]', '[2, 2]', '[1, 1]'):"3A1'",('[2, 1, 1]', '[2, 2]', '[2]'):'4A1',
         ('[2, 1, 1]', '[3, 1]', '[1, 1]'):'A2+A1',('[2, 1, 1]', '[3, 1]', '[2]'):'A2+2A1',('[2, 1, 1]', '[4]', '[1, 1]'):"(A3+A1)'",('[2, 1, 1]', '[4]', '[2]'):'A3+2A1',
         ('[2, 2]', '[1, 1, 1, 1]', '[1, 1]'):'2A1',('[2, 2]', '[1, 1, 1, 1]', '[2]'):"3A1''",('[2, 2]', '[2, 1, 1]', '[1, 1]'):"3A1'",('[2, 2]', '[2, 1, 1]', '[2]'):'4A1',
         ('[2, 2]', '[2, 2]', '[1, 1]'):'A2',('[2, 2]', '[2, 2]', '[2]'):'A2+A1',('[2, 2]', '[3, 1]', '[1, 1]'):'A2+2A1',('[2, 2]', '[3, 1]', '[2]'):'A2+3A1',
         ('[2, 2]', '[4]', '[1, 1]'):'D4(a1)',('[2, 2]', '[4]', '[2]'):'D4(a1)+A1',('[3, 1]', '[1, 1, 1, 1]', '[1, 1]'):'A2',('[3, 1]', '[1, 1, 1, 1]', '[2]'):'A2+A1',
         ('[3, 1]', '[2, 1, 1]', '[1, 1]'):'A2+A1',('[3, 1]', '[2, 1, 1]', '[2]'):'A2+2A1',('[3, 1]', '[2, 2]', '[1, 1]'):'A2+2A1',('[3, 1]', '[2, 2]', '[2]'):'A2+3A1',
         ('[3, 1]', '[3, 1]', '[1, 1]'):'2A2',('[3, 1]', '[3, 1]', '[2]'):'2A2+A1',('[3, 1]', '[4]', '[1, 1]'):'A3+A2',('[3, 1]', '[4]', '[2]'):'A3+A2+A1',
         ('[4]', '[1, 1, 1, 1]', '[1, 1]'):'A3',('[4]', '[1, 1, 1, 1]', '[2]'):"(A3+A1)''",('[4]', '[2, 1, 1]', '[1, 1]'):"(A3+A1)'",('[4]', '[2, 1, 1]', '[2]'):'A3+2A1',
         ('[4]', '[2, 2]', '[1, 1]'):'D4(a1)',('[4]', '[2, 2]', '[2]'):'D4(a1)+A1',('[4]', '[3, 1]', '[1, 1]'):'A3+A2',('[4]', '[3, 1]', '[2]'):'A3+A2+A1',
         ('[4]', '[4]', '[1, 1]'):'A4',('[4]', '[4]', '[2]'):'A4+A1'}

weight=[2,3,2,1,3,2]
drop_weight=4
node_set_1=[list(p) for p in itertools.product(range(drop_weight+1), repeat=len(weight))]
node_set_2=[]
for node in node_set_1:
    c=(-np.sum(np.array(node)*weight)-1)%drop_weight
    node_set_2.append(np.append(node,c))
    if c==0:
        node_set_2.append(np.append(node,drop_weight))

for cover_fold in range(1,6):
    node_set_3=[]
    for node in node_set_2:
        if node[0]<=cover_fold and node[1]+node[2]+node[3]<=cover_fold and node[4]+node[5]+node[6]<=cover_fold:
            add_point=[]
            for i in range(7):
                if node[i]==0:
                    add_point.append(i)
            if node[0]==cover_fold:
                add_point.append(0)
            if node[1]+node[2]+node[3]==cover_fold:
                add_point.append(11)
            if node[4]+node[5]+node[6]==cover_fold:
                add_point.append(12)
            node_set_3.append(add_point)
    
    node_set_3=[list(tpl) for tpl in {tuple(item) for item in node_set_3}]
    minimal_node_set_W2=minimal_W_cy(node_set_3)
    
    dict={0:1,1:4,2:5,3:6,4:2,5:0,6:125}
    minimal_node_set_W1=[]
    for node in minimal_node_set_W2:
        minimal_node_set_W1.append([convert_fuc(W1.roots,W2.roots,Index,dict) for Index in node])

    Target=[]
    Induc_W1cy_W1_set=[]
    Induc_W2cy_W2_set=[]
    W1_cy_special_rep_set=[]
    for i in range(len(minimal_node_set_W1)):
        Phi1_cy=minimal_node_set_W1[i]
        Phi2_cy=minimal_node_set_W2[i]
        W1_cy=reflectionsubgroup(W1,Phi1_cy)
        W2_cy=reflectionsubgroup(W2,Phi2_cy)
        W1_cy_chartable=pd.DataFrame(chartable(W1_cy))
        W1_cy_special_rep=W1_cy_chartable.loc[W1_cy_chartable['b']==W1_cy_chartable['a']]['charnames']
        W1_cy_special_rep_set.append(W1_cy_special_rep)
        induc_W1cy_W1=inductiontable(W1_cy,W1,invchar=(lambda G:chartable(G, chars=False)['b']))
        Induc_W1cy_W1=pd.DataFrame(induc_W1cy_W1['scalar'],columns=induc_W1cy_W1['charH'],index=induc_W1cy_W1['charW'])
        Induc_W1cy_W1_set.append(Induc_W1cy_W1)
        induc_W2cy_W2=inductiontable(W2_cy,W2,invchar=(lambda G:chartable(G, chars=False)['b']))
        Induc_W2cy_W2=pd.DataFrame(induc_W2cy_W2['scalar'],columns=induc_W2cy_W2['charH'],index=induc_W2cy_W2['charW'])
        Induc_W2cy_W2_set.append(Induc_W2cy_W2)
    for char in E7_Data.loc[:,'charnames']:
        j_induction_sign_set=[]
        for i in range(len(minimal_node_set_W1)):
            Phi1_cy=minimal_node_set_W1[i]
            Phi2_cy=minimal_node_set_W2[i]
            W1_cy=reflectionsubgroup(W1,Phi1_cy)
            W2_cy=reflectionsubgroup(W2,Phi2_cy)
            W1_cy_chartable=pd.DataFrame(chartable(W1_cy))
            W1_cy_special_rep=W1_cy_special_rep_set[i]
            Induc_W1cy_W1=Induc_W1cy_W1_set[i]
            Induc_W2cy_W2=Induc_W2cy_W2_set[i]
            for rep_now in W1_cy_special_rep:
                j_induction_now=(Induc_W1cy_W1[rep_now][Induc_W1cy_W1[rep_now]==1].index)[0]
                if list((E7_Data.loc[E7_Data['charnames']==str(j_induction_now)]).loc[:,'orbit'])[0] in list((E7_Data.loc[E7_Data['charnames']==char]).loc[:,'Georbit'])[0]:
                    rep_sign_position=(W1_cy_chartable.loc[W1_cy_chartable['charnames']==rep_now]).loc[:,'permsgn'].iloc[0]
                    rep_sign=W1_cy_chartable.iloc[rep_sign_position,0]
                    j_induction_sign=(Induc_W2cy_W2[rep_sign][Induc_W2cy_W2[rep_sign]==1].index)[0]
                    j_induction_sign_set.append(j_induction_sign)
        Target.append(max_orbit(list(set([Sat_dict.get(x) for x in set(j_induction_sign_set)])),E7_Data))
    E7_Data["E7_2A3+A1_"+str(cover_fold)]=Target

When $W_c$ is of the type $A_7$

In [13]:
# Construct W of type E7 and Wc of type A7 with a cartan matrix
W1=coxeter("E",7)
W2=coxeter("A",7)

# Record the saturation dictionary
Sat_dict={('[1, 1, 1, 1, 1, 1, 1, 1]',):'1',('[2, 1, 1, 1, 1, 1, 1]',):'A1',('[2, 2, 1, 1, 1, 1]',):'2A1',('[2, 2, 2, 1, 1]',):"3A1'",('[2, 2, 2, 2]',):'A2',
         ('[3, 1, 1, 1, 1, 1]',):'A2',('[3, 2, 1, 1, 1]',):'A2+A1',('[3, 2, 2, 1]',):'A2+2A1',('[3, 3, 1, 1]',):'2A2',('[3, 3, 2]',):'2A2+A1',
         ('[4, 1, 1, 1, 1]',):'A3',('[4, 2, 1, 1]',):"(A3+A1)'",('[4, 2, 2]',):'D4(a1)',('[4, 3, 1]',):'A3+A2',('[4, 4]',):'A4',
         ('[5, 1, 1, 1]',):'A4',('[5, 2, 1]',):'A4+A1',('[5, 3]',):'A4+A2',('[6, 1, 1]',):"A5'",('[6, 2]',):'E6(a3)',('[7, 1]',):'A6',('[8]',):'E6(a1)'}

weight=[1,2,3,4,3,2]
drop_weight=2
node_set_1=[list(p) for p in itertools.product(range(drop_weight+1), repeat=len(weight))]
node_set_2=[]
for node in node_set_1:
    c=(-np.sum(np.array(node)*weight)-1)%drop_weight
    node_set_2.append(np.append(node,c))
    if c==0:
        node_set_2.append(np.append(node,drop_weight))

for cover_fold in range(1,10):
    node_set_3=[]
    for node in node_set_2:
        if node[0]+node[1]+node[2]+node[3]+node[4]+node[5]+node[6]<=cover_fold:
            add_point=[]
            for i in range(7):
                if node[i]==0:
                    add_point.append(i)
            if node[0]+node[1]+node[2]+node[3]+node[4]+node[5]+node[6]==cover_fold:
                add_point.append(27)
            node_set_3.append(add_point)

    
    node_set_3=[list(tpl) for tpl in {tuple(item) for item in node_set_3}]
    minimal_node_set_W2=minimal_W_cy(node_set_3)
    
    dict={0:6,1:5,2:4,3:3,4:2,5:0,6:125}
    minimal_node_set_W1=[]
    for node in minimal_node_set_W2:
        minimal_node_set_W1.append([convert_fuc(W1.roots,W2.roots,Index,dict) for Index in node])

    Target=[]
    Induc_W1cy_W1_set=[]
    Induc_W2cy_W2_set=[]
    W1_cy_special_rep_set=[]
    for i in range(len(minimal_node_set_W1)):
        Phi1_cy=minimal_node_set_W1[i]
        Phi2_cy=minimal_node_set_W2[i]
        W1_cy=reflectionsubgroup(W1,Phi1_cy)
        W2_cy=reflectionsubgroup(W2,Phi2_cy)
        W1_cy_chartable=pd.DataFrame(chartable(W1_cy))
        W1_cy_special_rep=W1_cy_chartable.loc[W1_cy_chartable['b']==W1_cy_chartable['a']]['charnames']
        W1_cy_special_rep_set.append(W1_cy_special_rep)
        induc_W1cy_W1=inductiontable(W1_cy,W1,invchar=(lambda G:chartable(G, chars=False)['b']))
        Induc_W1cy_W1=pd.DataFrame(induc_W1cy_W1['scalar'],columns=induc_W1cy_W1['charH'],index=induc_W1cy_W1['charW'])
        Induc_W1cy_W1_set.append(Induc_W1cy_W1)
        induc_W2cy_W2=inductiontable(W2_cy,W2,invchar=(lambda G:chartable(G, chars=False)['b']))
        Induc_W2cy_W2=pd.DataFrame(induc_W2cy_W2['scalar'],columns=induc_W2cy_W2['charH'],index=induc_W2cy_W2['charW'])
        Induc_W2cy_W2_set.append(Induc_W2cy_W2)
    for char in E7_Data.loc[:,'charnames']:
        j_induction_sign_set=[]
        for i in range(len(minimal_node_set_W1)):
            Phi1_cy=minimal_node_set_W1[i]
            Phi2_cy=minimal_node_set_W2[i]
            W1_cy=reflectionsubgroup(W1,Phi1_cy)
            W2_cy=reflectionsubgroup(W2,Phi2_cy)
            W1_cy_chartable=pd.DataFrame(chartable(W1_cy))
            W1_cy_special_rep=W1_cy_special_rep_set[i]
            Induc_W1cy_W1=Induc_W1cy_W1_set[i]
            Induc_W2cy_W2=Induc_W2cy_W2_set[i]
            for rep_now in W1_cy_special_rep:
                j_induction_now=(Induc_W1cy_W1[rep_now][Induc_W1cy_W1[rep_now]==1].index)[0]
                if list((E7_Data.loc[E7_Data['charnames']==str(j_induction_now)]).loc[:,'orbit'])[0] in list((E7_Data.loc[E7_Data['charnames']==char]).loc[:,'Georbit'])[0]:
                    rep_sign_position=(W1_cy_chartable.loc[W1_cy_chartable['charnames']==rep_now]).loc[:,'permsgn'].iloc[0]
                    rep_sign=W1_cy_chartable.iloc[rep_sign_position,0]
                    if rep_sign not in Induc_W2cy_W2.columns:
                        rep_sign=switch_A3_D3(rep_sign,Induc_W2cy_W2.columns)
                    j_induction_sign=(Induc_W2cy_W2[rep_sign][Induc_W2cy_W2[rep_sign]==1].index)[0]
                    j_induction_sign_set.append(j_induction_sign)
        Target.append(max_orbit(list(set([Sat_dict.get(x) for x in set(j_induction_sign_set)])),E7_Data))
    E7_Data["E7_A7_"+str(cover_fold)]=Target

$E_8$

In [14]:
E8_Data=pd.DataFrame({'charnames':["('1_x',)", "('8_z',)", "('35_x',)", "('112_z',)", "('210_x',)", "('84_x',)", "('560_z',)", "('700_x',)", "('567_x',)", "('400_z',)", "('1400_z',)", "('1400_x',)", "('1344_x',)", "('448_z',)", "('3240_z',)", "('2240_x',)", "('2268_x',)", "('4096_z',)", "('1400_zz',)", "('972_x',)", "('525_x',)", "('4200_x',)", "('4536_z',)", "('2800_z',)", "('2835_x',)", "('6075_x',)", "('5600_z',)", "('4200_z',)", "('3200_x',)", "('4480_y',)", "('7168_w',)", "('4200_y',)", "('3150_y',)", "('1344_w',)", "('2016_w',)", "('420_y',)", "('2100_y',)", '("5600_z\'",)', '("4200_z\'",)', '("3200_x\'",)', '("6075_x\'",)', '("2835_x\'",)', '("4536_z\'",)', '("4200_x\'",)', '("2800_z\'",)', '("4096_x\'",)', '("840_x\'",)', '("700_xx\'",)', '("2240_x\'",)', '("1400_zz\'",)', '("2268_x\'",)', '("3240_z\'",)', '("1400_x\'",)', '("1050_x\'",)', '("175_x\'",)', '("525_x\'",)', '("1400_z\'",)', '("1344_x\'",)', '("448_z\'",)', '("700_x\'",)', '("400_z\'",)', '("567_x\'",)', '("560_z\'",)', '("210_x\'",)', '("50_x\'",)', '("112_z\'",)', '("84_x\'",)', '("35_x\'",)', '("8_z\'",)', '("1_x\'",)'],
                     'orbit':['E8', 'E8(a1)', 'E8(a2)', 'E8(a3)', 'E8(a4)', 'E7', 'E8(b4)', 'E8(a5)', 'E7(a1)', 'D7', 'E8(b5)', 'E8(a6)', 'E7(a2)', 'E6+A1', 'D7(a1)', 'E8(b6)', 'E7(a3)', 'E6(a1)+A1', 'A7', 'D6', 'E6', 'D7(a2)', 'D5+A2', 'E6(a1)', 'A6+A1', 'E7(a4)', 'D6(a1)', 'A6', 'D5+A1', 'E8(a7)', 'E7(a5)', 'D6(a2)', 'E6(a3)+A1', 'D5(a1)+A2', 'A5+A1', 'A4+A3', 'D5', 'E6(a3)', 'D4+A2', 'A5', 'D5(a1)+A1', 'A4+A2+A1', 'A4+A2', 'A4+2A1', 'D5(a1)', 'A4+A1', '2A3', 'D4+A1', 'D4(a1)+A2', 'A3+A2+A1', 'A4', 'A3+A2', 'D4(a1)+A1', 'A3+2A1', '2A2+2A1', 'D4', 'D4(a1)', 'A3+A1', '2A2+A1', '2A2', 'A2+3A1', 'A3', 'A2+2A1', 'A2+A1', '4A1', 'A2', '3A1', '2A1', 'A1', '1'],
                     'Georbit':[['E8'], ['E8(a1)', 'E8'], ['E8(a2)', 'E8', 'E8(a1)'], ['E8(a3)', 'E8', 'E8(a1)', 'E8(a2)'], ['E8(a4)', 'E8', 'E8(a1)', 'E8(a2)', 'E8(a3)'], ['E7', 'E8', 'E8(a1)', 'E8(a2)', 'E8(a3)'], ['E8(b4)', 'E8', 'E8(a1)', 'E8(a2)', 'E8(a3)', 'E8(a4)', 'E7'], ['E8(a5)', 'E8', 'E8(a1)', 'E8(a2)', 'E8(a3)', 'E8(a4)', 'E8(b4)', 'E7'], ['E7(a1)', 'E8', 'E8(a1)', 'E8(a2)', 'E8(a3)', 'E8(a4)', 'E8(b4)', 'E7'], ['D7', 'E8(a5)', 'E8', 'E8(a1)', 'E8(a2)', 'E8(a3)', 'E8(a4)', 'E8(b4)', 'E7'], ['E8(b5)', 'E7(a1)', 'E8(a5)', 'E8', 'E8(a1)', 'E8(a2)', 'E8(a3)', 'E8(a4)', 'E8(b4)', 'E7'], ['E8(a6)', 'E7(a1)', 'E8(a5)', 'E8', 'E8(a1)', 'D7', 'E8(a2)', 'E8(a3)', 'E8(a4)', 'E8(b4)', 'E8(b5)', 'E7'], ['E7(a2)', 'E7(a1)', 'E8(a5)', 'E8', 'E8(a1)', 'E8(a2)', 'E8(a3)', 'E8(a4)', 'E8(b4)', 'E8(b5)', 'E7'], ['E6+A1', 'E7(a1)', 'E8(a5)', 'E8', 'E8(a1)', 'E7(a2)', 'E8(a2)', 'E8(a3)', 'E8(a4)', 'E8(b4)', 'E8(b5)', 'E7'], ['D7(a1)', 'E7(a1)', 'E8(a5)', 'E8', 'E8(a1)', 'D7', 'E7(a2)', 'E8(a6)', 'E8(a2)', 'E8(a3)', 'E8(a4)', 'E8(b4)', 'E8(b5)', 'E7'], ['E8(b6)', 'E7(a1)', 'E6+A1', 'E8(a5)', 'E8', 'E8(a1)', 'D7', 'E7(a2)', 'E8(a6)', 'E8(a2)', 'E8(a3)', 'E8(a4)', 'D7(a1)', 'E8(b4)', 'E8(b5)', 'E7'], ['E7(a3)', 'E7(a1)', 'E8(a5)', 'E8', 'E8(a1)', 'D7', 'E7(a2)', 'E8(a6)', 'E8(a2)', 'E8(a3)', 'E8(a4)', 'D7(a1)', 'E8(b4)', 'E8(b5)', 'E7'], ['E6(a1)+A1', 'E7(a1)', 'E8(b6)', 'E6+A1', 'E8(a5)', 'E8', 'E8(a1)', 'D7', 'E7(a3)', 'E7(a2)', 'E8(a6)', 'E8(a2)', 'E8(a3)', 'E8(a4)', 'D7(a1)', 'E8(b4)', 'E8(b5)', 'E7'], ['A7', 'E7(a1)', 'E8(b6)', 'E6+A1', 'E8(a5)', 'E8', 'E8(a1)', 'D7', 'E7(a2)', 'E8(a6)', 'E8(a2)', 'E8(a3)', 'E8(a4)', 'D7(a1)', 'E8(b4)', 'E8(b5)', 'E7'], ['D6', 'E7(a1)', 'E8(a5)', 'E8', 'E8(a1)', 'D7', 'E7(a3)', 'E7(a2)', 'E8(a6)', 'E8(a2)', 'E8(a3)', 'E8(a4)', 'D7(a1)', 'E8(b4)', 'E8(b5)', 'E7'], ['E6', 'E7(a1)', 'E6+A1', 'E8(a5)', 'E8', 'E8(a1)', 'E7(a2)', 'E8(a2)', 'E8(a3)', 'E8(a4)', 'E8(b4)', 'E8(b5)', 'E7'], ['D7(a2)', 'E7(a1)', 'A7', 'E8(b6)', 'E6+A1', 'E8(a5)', 'E8', 'E8(a1)', 'D7', 'E7(a3)', 'E7(a2)', 'E8(a6)', 'E8(a2)', 'E8(a3)', 'E8(a4)', 'D7(a1)', 'E8(b4)', 'E6(a1)+A1', 'E8(b5)', 'E7'], ['D5+A2', 'D6', 'E7(a1)', 'A7', 'E8(b6)', 'E6+A1', 'E8(a5)', 'E8', 'E8(a1)', 'D7', 'E7(a3)', 'E7(a2)', 'E8(a6)', 'E8(a2)', 'E8(a3)', 'E8(a4)', 'D7(a1)', 'E8(b4)', 'E6(a1)+A1', 'E8(b5)', 'E7', 'D7(a2)'], ['E6(a1)', 'E7(a1)', 'E8(b6)', 'E6+A1', 'E8(a5)', 'E8', 'E8(a1)', 'D7', 'E7(a3)', 'E7(a2)', 'E8(a6)', 'E8(a2)', 'E8(a3)', 'E8(a4)', 'D7(a1)', 'E8(b4)', 'E6', 'E6(a1)+A1', 'E8(b5)', 'E7'], ['A6+A1', 'D6', 'E7(a1)', 'A7', 'E8(b6)', 'E6+A1', 'E8(a5)', 'E8', 'E8(a1)', 'D5+A2', 'D7', 'E7(a3)', 'E7(a2)', 'E8(a6)', 'E8(a2)', 'E8(a3)', 'E8(a4)', 'D7(a1)', 'E8(b4)', 'E6(a1)+A1', 'E8(b5)', 'E7', 'D7(a2)'], ['E7(a4)', 'D6', 'E7(a1)', 'A7', 'E8(b6)', 'E6+A1', 'E8(a5)', 'E8', 'E8(a1)', 'E6(a1)', 'D5+A2', 'D7', 'E7(a3)', 'E7(a2)', 'E8(a6)', 'E8(a2)', 'E8(a3)', 'E8(a4)', 'D7(a1)', 'E8(b4)', 'E6', 'E6(a1)+A1', 'E8(b5)', 'E7', 'D7(a2)'], ['D6(a1)', 'D6', 'E7(a1)', 'A7', 'E8(b6)', 'E6+A1', 'E7(a4)', 'E8(a5)', 'E8', 'E8(a1)', 'E6(a1)', 'D5+A2', 'D7', 'E7(a3)', 'E7(a2)', 'E8(a6)', 'E8(a2)', 'E8(a3)', 'E8(a4)', 'D7(a1)', 'E8(b4)', 'E6', 'E6(a1)+A1', 'E8(b5)', 'E7', 'D7(a2)'], ['A6', 'D6', 'E7(a1)', 'A6+A1', 'A7', 'E8(b6)', 'E6+A1', 'E7(a4)', 'E8(a5)', 'E8', 'E8(a1)', 'E6(a1)', 'D5+A2', 'D7', 'E7(a3)', 'E7(a2)', 'E8(a6)', 'E8(a2)', 'E8(a3)', 'E8(a4)', 'D7(a1)', 'E8(b4)', 'E6', 'E6(a1)+A1', 'E8(b5)', 'E7', 'D7(a2)'], ['D5+A1', 'D6', 'E7(a1)', 'D6(a1)', 'A7', 'E8(b6)', 'E6+A1', 'E7(a4)', 'E8(a5)', 'E8', 'E8(a1)', 'E6(a1)', 'D5+A2', 'D7', 'E7(a3)', 'E7(a2)', 'E8(a6)', 'E8(a2)', 'E8(a3)', 'E8(a4)', 'D7(a1)', 'E8(b4)', 'E6', 'E6(a1)+A1', 'E8(b5)', 'E7', 'D7(a2)'], ['E8(a7)', 'D6', 'E7(a1)', 'A6+A1', 'D6(a1)', 'A7', 'E8(b6)', 'E6+A1', 'E7(a4)', 'E8(a5)', 'E8', 'E8(a1)', 'E6(a1)', 'A6', 'D5+A2', 'D7', 'E7(a3)', 'E7(a2)', 'E8(a6)', 'E8(a2)', 'E8(a3)', 'E8(a4)', 'D7(a1)', 'E8(b4)', 'E6', 'E6(a1)+A1', 'E8(b5)', 'E7', 'D7(a2)'], ['E7(a5)', 'D6', 'E7(a1)', 'A6+A1', 'D6(a1)', 'A7', 'E8(b6)', 'E6+A1', 'E7(a4)', 'E8(a5)', 'E8', 'E8(a1)', 'E6(a1)', 'A6', 'D5+A2', 'D7', 'E7(a3)', 'E7(a2)', 'D5+A1', 'E8(a7)', 'E8(a6)', 'E8(a2)', 'E8(a3)', 'E8(a4)', 'D7(a1)', 'E8(b4)', 'E6', 'E6(a1)+A1', 'E8(b5)', 'E7', 'D7(a2)'], ['D6(a2)', 'E7(a5)', 'D6', 'E7(a1)', 'A6+A1', 'D6(a1)', 'A7', 'E8(b6)', 'E6+A1', 'E7(a4)', 'E8(a5)', 'E8', 'E8(a1)', 'E6(a1)', 'A6', 'D5+A2', 'D7', 'E7(a3)', 'E7(a2)', 'D5+A1', 'E8(a7)', 'E8(a6)', 'E8(a2)', 'E8(a3)', 'E8(a4)', 'D7(a1)', 'E8(b4)', 'E6', 'E6(a1)+A1', 'E8(b5)', 'E7', 'D7(a2)'], ['E6(a3)+A1', 'E7(a5)', 'D6', 'E7(a1)', 'A6+A1', 'D6(a1)', 'A7', 'E8(b6)', 'E6+A1', 'E7(a4)', 'E8(a5)', 'E8', 'E8(a1)', 'E6(a1)', 'A6', 'D5+A2', 'D7', 'E7(a3)', 'E7(a2)', 'D5+A1', 'E8(a7)', 'E8(a6)', 'E8(a2)', 'E8(a3)', 'E8(a4)', 'D7(a1)', 'E8(b4)', 'E6', 'E6(a1)+A1', 'E8(b5)', 'E7', 'D7(a2)'], ['D5(a1)+A2', 'E7(a5)', 'D6', 'E7(a1)', 'A6+A1', 'D6(a1)', 'D6(a2)', 'A7', 'E8(b6)', 'E6+A1', 'E7(a4)', 'E8(a5)', 'E8', 'E8(a1)', 'E6(a1)', 'A6', 'D5+A2', 'D7', 'E7(a3)', 'E7(a2)', 'D5+A1', 'E6(a3)+A1', 'E8(a7)', 'E8(a6)', 'E8(a2)', 'E8(a3)', 'E8(a4)', 'D7(a1)', 'E8(b4)', 'E6', 'E6(a1)+A1', 'E8(b5)', 'E7', 'D7(a2)'], ['A5+A1', 'E7(a5)', 'D6', 'E7(a1)', 'A6+A1', 'D6(a1)', 'D6(a2)', 'A7', 'E8(b6)', 'E6+A1', 'E7(a4)', 'E8(a5)', 'E8', 'E8(a1)', 'E6(a1)', 'A6', 'D5+A2', 'D7', 'E7(a3)', 'E7(a2)', 'D5+A1', 'E6(a3)+A1', 'E8(a7)', 'E8(a6)', 'E8(a2)', 'E8(a3)', 'E8(a4)', 'D7(a1)', 'E8(b4)', 'E6', 'E6(a1)+A1', 'E8(b5)', 'E7', 'D7(a2)'], ['A4+A3', 'E7(a5)', 'A5+A1', 'D6', 'E7(a1)', 'A6+A1', 'D6(a1)', 'D6(a2)', 'A7', 'E8(b6)', 'E6+A1', 'E7(a4)', 'E8(a5)', 'E8', 'E8(a1)', 'E6(a1)', 'A6', 'D5+A2', 'D7', 'E7(a3)', 'E7(a2)', 'D5+A1', 'E6(a3)+A1', 'E8(a7)', 'E8(a6)', 'E8(a2)', 'E8(a3)', 'E8(a4)', 'D7(a1)', 'E8(b4)', 'D5(a1)+A2', 'E6', 'E6(a1)+A1', 'E8(b5)', 'E7', 'D7(a2)'], ['D5', 'D6', 'E7(a1)', 'D6(a1)', 'A7', 'E8(b6)', 'E6+A1', 'E7(a4)', 'E8(a5)', 'E8', 'E8(a1)', 'E6(a1)', 'D5+A2', 'D7', 'E7(a3)', 'E7(a2)', 'D5+A1', 'E8(a6)', 'E8(a2)', 'E8(a3)', 'E8(a4)', 'D7(a1)', 'E8(b4)', 'E6', 'E6(a1)+A1', 'E8(b5)', 'E7', 'D7(a2)'], ['E6(a3)', 'E7(a5)', 'D6', 'E7(a1)', 'A6+A1', 'D6(a1)', 'A7', 'E8(b6)', 'E6+A1', 'E7(a4)', 'E8(a5)', 'E8', 'E8(a1)', 'E6(a1)', 'A6', 'D5+A2', 'D7', 'E7(a3)', 'E7(a2)', 'D5+A1', 'E6(a3)+A1', 'E8(a7)', 'E8(a6)', 'E8(a2)', 'E8(a3)', 'D5', 'E8(a4)', 'D7(a1)', 'E8(b4)', 'E6', 'E6(a1)+A1', 'E8(b5)', 'E7', 'D7(a2)'], ['D4+A2', 'E7(a5)', 'D6', 'E7(a1)', 'A6+A1', 'D6(a1)', 'D6(a2)', 'A7', 'E8(b6)', 'E6+A1', 'E7(a4)', 'E8(a5)', 'E8', 'E8(a1)', 'E6(a1)', 'A6', 'D5+A2', 'D7', 'E7(a3)', 'E7(a2)', 'D5+A1', 'E6(a3)+A1', 'E8(a7)', 'E8(a6)', 'E8(a2)', 'E8(a3)', 'E8(a4)', 'D7(a1)', 'E8(b4)', 'D5(a1)+A2', 'E6', 'E6(a1)+A1', 'E8(b5)', 'E7', 'D7(a2)'], ['A5', 'E7(a5)', 'A5+A1', 'D6', 'E7(a1)', 'A6+A1', 'D6(a1)', 'D6(a2)', 'A7', 'E8(b6)', 'E6+A1', 'E7(a4)', 'E8(a5)', 'E8', 'E8(a1)', 'E6(a1)', 'A6', 'D5+A2', 'D7', 'E7(a3)', 'E7(a2)', 'D5+A1', 'E6(a3)+A1', 'E8(a7)', 'E8(a6)', 'E8(a2)', 'E8(a3)', 'D5', 'E8(a4)', 'E6(a3)', 'D7(a1)', 'E8(b4)', 'E6', 'E6(a1)+A1', 'E8(b5)', 'E7', 'D7(a2)'], ['D5(a1)+A1', 'E7(a5)', 'D6', 'E7(a1)', 'A6+A1', 'D6(a1)', 'D6(a2)', 'A7', 'E8(b6)', 'E6+A1', 'E7(a4)', 'E8(a5)', 'E8', 'E8(a1)', 'E6(a1)', 'A6', 'D5+A2', 'D7', 'E7(a3)', 'E7(a2)', 'D5+A1', 'E6(a3)+A1', 'E8(a7)', 'E8(a6)', 'E8(a2)', 'E8(a3)', 'D5', 'E8(a4)', 'D4+A2', 'E6(a3)', 'D7(a1)', 'E8(b4)', 'D5(a1)+A2', 'E6', 'E6(a1)+A1', 'E8(b5)', 'E7', 'D7(a2)'], ['A4+A2+A1', 'A4+A3', 'E7(a5)', 'A5+A1', 'D6', 'E7(a1)', 'A6+A1', 'D6(a1)', 'D6(a2)', 'A7', 'E8(b6)', 'E6+A1', 'E7(a4)', 'E8(a5)', 'E8', 'E8(a1)', 'E6(a1)', 'A6', 'D5+A2', 'D7', 'E7(a3)', 'E7(a2)', 'D5+A1', 'E6(a3)+A1', 'E8(a7)', 'E8(a6)', 'E8(a2)', 'E8(a3)', 'E8(a4)', 'D4+A2', 'D7(a1)', 'E8(b4)', 'D5(a1)+A2', 'E6', 'E6(a1)+A1', 'E8(b5)', 'E7', 'D7(a2)'], ['A4+A2', 'A4+A3', 'E7(a5)', 'A5+A1', 'D6', 'E7(a1)', 'A6+A1', 'D6(a1)', 'D6(a2)', 'A7', 'E8(b6)', 'E6+A1', 'E7(a4)', 'E8(a5)', 'E8', 'A4+A2+A1', 'E8(a1)', 'E6(a1)', 'A6', 'D5+A2', 'D7', 'E7(a3)', 'E7(a2)', 'D5+A1', 'D5(a1)+A1', 'E6(a3)+A1', 'E8(a7)', 'E8(a6)', 'E8(a2)', 'A5', 'E8(a3)', 'D5', 'E8(a4)', 'D4+A2', 'E6(a3)', 'D7(a1)', 'E8(b4)', 'D5(a1)+A2', 'E6', 'E6(a1)+A1', 'E8(b5)', 'E7', 'D7(a2)'], ['A4+2A1', 'A4+A3', 'E7(a5)', 'A5+A1', 'D6', 'E7(a1)', 'A6+A1', 'D6(a1)', 'D6(a2)', 'A7', 'E8(b6)', 'E6+A1', 'E7(a4)', 'E8(a5)', 'E8', 'A4+A2+A1', 'E8(a1)', 'E6(a1)', 'A6', 'D5+A2', 'D7', 'E7(a3)', 'E7(a2)', 'D5+A1', 'D5(a1)+A1', 'E6(a3)+A1', 'E8(a7)', 'E8(a6)', 'E8(a2)', 'A5', 'A4+A2', 'E8(a3)', 'D5', 'E8(a4)', 'D4+A2', 'E6(a3)', 'D7(a1)', 'E8(b4)', 'D5(a1)+A2', 'E6', 'E6(a1)+A1', 'E8(b5)', 'E7', 'D7(a2)'], ['D5(a1)', 'E7(a5)', 'D6', 'E7(a1)', 'A6+A1', 'D6(a1)', 'D6(a2)', 'A7', 'E8(b6)', 'E6+A1', 'E7(a4)', 'E8(a5)', 'E8', 'E8(a1)', 'E6(a1)', 'A6', 'D5+A2', 'D7', 'E7(a3)', 'E7(a2)', 'D5+A1', 'D5(a1)+A1', 'E6(a3)+A1', 'E8(a7)', 'E8(a6)', 'E8(a2)', 'E8(a3)', 'D5', 'E8(a4)', 'D4+A2', 'E6(a3)', 'D7(a1)', 'E8(b4)', 'D5(a1)+A2', 'E6', 'E6(a1)+A1', 'E8(b5)', 'E7', 'D7(a2)'], ['A4+A1', 'A4+A3', 'E7(a5)', 'A5+A1', 'D6', 'E7(a1)', 'A6+A1', 'D6(a1)', 'D6(a2)', 'A7', 'E8(b6)', 'E6+A1', 'E7(a4)', 'E8(a5)', 'E8', 'A4+A2+A1', 'E8(a1)', 'E6(a1)', 'A6', 'D5+A2', 'D7', 'E7(a3)', 'E7(a2)', 'D5+A1', 'D5(a1)+A1', 'E6(a3)+A1', 'E8(a7)', 'D5(a1)', 'E8(a6)', 'E8(a2)', 'A5', 'A4+A2', 'E8(a3)', 'A4+2A1', 'D5', 'E8(a4)', 'D4+A2', 'E6(a3)', 'D7(a1)', 'E8(b4)', 'D5(a1)+A2', 'E6', 'E6(a1)+A1', 'E8(b5)', 'E7', 'D7(a2)'], ['2A3', 'A4+A3', 'E7(a5)', 'A5+A1', 'D6', 'E7(a1)', 'A6+A1', 'D6(a1)', 'D6(a2)', 'A7', 'E8(b6)', 'E6+A1', 'E7(a4)', 'E8(a5)', 'E8', 'A4+A2+A1', 'E8(a1)', 'E6(a1)', 'A6', 'D5+A2', 'D7', 'E7(a3)', 'E7(a2)', 'D5+A1', 'D5(a1)+A1', 'E6(a3)+A1', 'E8(a7)', 'E8(a6)', 'E8(a2)', 'A5', 'A4+A2', 'E8(a3)', 'A4+2A1', 'D5', 'E8(a4)', 'D4+A2', 'E6(a3)', 'D7(a1)', 'E8(b4)', 'D5(a1)+A2', 'E6', 'E6(a1)+A1', 'E8(b5)', 'E7', 'D7(a2)'], ['D4+A1', 'E7(a5)', 'D6', 'E7(a1)', 'A6+A1', 'D6(a1)', 'D6(a2)', 'A7', 'E8(b6)', 'E6+A1', 'E7(a4)', 'E8(a5)', 'E8', 'E8(a1)', 'E6(a1)', 'A6', 'D5+A2', 'D7', 'E7(a3)', 'E7(a2)', 'D5+A1', 'D5(a1)+A1', 'E6(a3)+A1', 'E8(a7)', 'D5(a1)', 'E8(a6)', 'E8(a2)', 'E8(a3)', 'D5', 'E8(a4)', 'D4+A2', 'E6(a3)', 'D7(a1)', 'E8(b4)', 'D5(a1)+A2', 'E6', 'E6(a1)+A1', 'E8(b5)', 'E7', 'D7(a2)'], ['D4(a1)+A2', 'A4+A3', 'E7(a5)', 'A5+A1', 'D6', 'E7(a1)', 'A6+A1', 'D6(a1)', 'D6(a2)', 'A7', 'E8(b6)', 'E6+A1', 'E7(a4)', 'E8(a5)', 'E8', 'A4+A2+A1', 'E8(a1)', 'E6(a1)', 'A6', 'D5+A2', 'D7', 'E7(a3)', 'E7(a2)', 'D5+A1', '2A3', 'D5(a1)+A1', 'E6(a3)+A1', 'E8(a7)', 'D5(a1)', 'E8(a6)', 'E8(a2)', 'A5', 'A4+A2', 'E8(a3)', 'A4+2A1', 'D5', 'E8(a4)', 'D4+A2', 'E6(a3)', 'D7(a1)', 'E8(b4)', 'D5(a1)+A2', 'A4+A1', 'E6', 'E6(a1)+A1', 'E8(b5)', 'E7', 'D7(a2)'], ['A3+A2+A1', 'A4+A3', 'E7(a5)', 'A5+A1', 'D6', 'E7(a1)', 'A6+A1', 'D6(a1)', 'D6(a2)', 'A7', 'E8(b6)', 'E6+A1', 'E7(a4)', 'E8(a5)', 'E8', 'A4+A2+A1', 'D4(a1)+A2', 'E8(a1)', 'E6(a1)', 'A6', 'D5+A2', 'D7', 'E7(a3)', 'E7(a2)', 'D5+A1', '2A3', 'D5(a1)+A1', 'E6(a3)+A1', 'E8(a7)', 'D4+A1', 'D5(a1)', 'E8(a6)', 'E8(a2)', 'A5', 'A4+A2', 'E8(a3)', 'A4+2A1', 'D5', 'E8(a4)', 'D4+A2', 'E6(a3)', 'D7(a1)', 'E8(b4)', 'D5(a1)+A2', 'A4+A1', 'E6', 'E6(a1)+A1', 'E8(b5)', 'E7', 'D7(a2)'], ['A4', 'A4+A3', 'E7(a5)', 'A5+A1', 'D6', 'E7(a1)', 'A6+A1', 'D6(a1)', 'D6(a2)', 'A7', 'E8(b6)', 'E6+A1', 'E7(a4)', 'E8(a5)', 'E8', 'A4+A2+A1', 'E8(a1)', 'E6(a1)', 'A6', 'D5+A2', 'D7', 'E7(a3)', 'E7(a2)', 'D5+A1', 'D5(a1)+A1', 'E6(a3)+A1', 'E8(a7)', 'D5(a1)', 'E8(a6)', 'E8(a2)', 'A5', 'A4+A2', 'E8(a3)', 'A4+2A1', 'D5', 'E8(a4)', 'D4+A2', 'E6(a3)', 'D7(a1)', 'E8(b4)', 'D5(a1)+A2', 'A4+A1', 'E6', 'E6(a1)+A1', 'E8(b5)', 'E7', 'D7(a2)'], ['A3+A2', 'A4+A3', 'E7(a5)', 'A5+A1', 'D6', 'E7(a1)', 'A6+A1', 'D6(a1)', 'D6(a2)', 'A7', 'E8(b6)', 'E6+A1', 'E7(a4)', 'E8(a5)', 'E8', 'A4+A2+A1', 'D4(a1)+A2', 'E8(a1)', 'E6(a1)', 'A6', 'D5+A2', 'D7', 'E7(a3)', 'A3+A2+A1', 'E7(a2)', 'D5+A1', '2A3', 'D5(a1)+A1', 'E6(a3)+A1', 'E8(a7)', 'D4+A1', 'D5(a1)', 'E8(a6)', 'E8(a2)', 'A5', 'A4+A2', 'E8(a3)', 'A4+2A1', 'D5', 'E8(a4)', 'D4+A2', 'E6(a3)', 'D7(a1)', 'E8(b4)', 'D5(a1)+A2', 'A4+A1', 'E6', 'A4', 'E6(a1)+A1', 'E8(b5)', 'E7', 'D7(a2)'], ['D4(a1)+A1', 'A4+A3', 'E7(a5)', 'A5+A1', 'D6', 'E7(a1)', 'A6+A1', 'D6(a1)', 'D6(a2)', 'A7', 'E8(b6)', 'E6+A1', 'E7(a4)', 'E8(a5)', 'E8', 'A4+A2+A1', 'D4(a1)+A2', 'E8(a1)', 'E6(a1)', 'A6', 'D5+A2', 'D7', 'E7(a3)', 'A3+A2+A1', 'E7(a2)', 'D5+A1', '2A3', 'D5(a1)+A1', 'A3+A2', 'E6(a3)+A1', 'E8(a7)', 'D4+A1', 'D5(a1)', 'E8(a6)', 'E8(a2)', 'A5', 'A4+A2', 'E8(a3)', 'A4+2A1', 'D5', 'E8(a4)', 'D4+A2', 'E6(a3)', 'D7(a1)', 'E8(b4)', 'D5(a1)+A2', 'A4+A1', 'E6', 'A4', 'E6(a1)+A1', 'E8(b5)', 'E7', 'D7(a2)'], ['A3+2A1', 'A4+A3', 'E7(a5)', 'A5+A1', 'D6', 'E7(a1)', 'A6+A1', 'D6(a1)', 'D6(a2)', 'A7', 'E8(b6)', 'E6+A1', 'E7(a4)', 'E8(a5)', 'E8', 'A4+A2+A1', 'D4(a1)+A2', 'E8(a1)', 'E6(a1)', 'A6', 'D5+A2', 'D7', 'E7(a3)', 'A3+A2+A1', 'E7(a2)', 'D5+A1', '2A3', 'D5(a1)+A1', 'D4(a1)+A1', 'A3+A2', 'E6(a3)+A1', 'E8(a7)', 'D4+A1', 'D5(a1)', 'E8(a6)', 'E8(a2)', 'A5', 'A4+A2', 'E8(a3)', 'A4+2A1', 'D5', 'E8(a4)', 'D4+A2', 'E6(a3)', 'D7(a1)', 'E8(b4)', 'D5(a1)+A2', 'A4+A1', 'E6', 'A4', 'E6(a1)+A1', 'E8(b5)', 'E7', 'D7(a2)'], ['2A2+2A1', 'A4+A3', 'E7(a5)', 'A5+A1', 'A3+2A1', 'D6', 'E7(a1)', 'A6+A1', 'D6(a1)', 'D6(a2)', 'A7', 'E8(b6)', 'E6+A1', 'E7(a4)', 'E8(a5)', 'E8', 'A4+A2+A1', 'D4(a1)+A2', 'E8(a1)', 'E6(a1)', 'A6', 'D5+A2', 'D7', 'E7(a3)', 'A3+A2+A1', 'E7(a2)', 'D5+A1', '2A3', 'D5(a1)+A1', 'D4(a1)+A1', 'A3+A2', 'E6(a3)+A1', 'E8(a7)', 'D4+A1', 'D5(a1)', 'E8(a6)', 'E8(a2)', 'A5', 'A4+A2', 'E8(a3)', 'A4+2A1', 'D5', 'E8(a4)', 'D4+A2', 'E6(a3)', 'D7(a1)', 'E8(b4)', 'D5(a1)+A2', 'A4+A1', 'E6', 'A4', 'E6(a1)+A1', 'E8(b5)', 'E7', 'D7(a2)'], ['D4', 'E7(a5)', 'D6', 'E7(a1)', 'A6+A1', 'D6(a1)', 'D6(a2)', 'A7', 'E8(b6)', 'E6+A1', 'E7(a4)', 'E8(a5)', 'E8', 'E8(a1)', 'E6(a1)', 'A6', 'D5+A2', 'D7', 'E7(a3)', 'E7(a2)', 'D5+A1', 'D5(a1)+A1', 'E6(a3)+A1', 'E8(a7)', 'D4+A1', 'D5(a1)', 'E8(a6)', 'E8(a2)', 'E8(a3)', 'D5', 'E8(a4)', 'D4+A2', 'E6(a3)', 'D7(a1)', 'E8(b4)', 'D5(a1)+A2', 'E6', 'E6(a1)+A1', 'E8(b5)', 'E7', 'D7(a2)'], ['D4(a1)', 'A4+A3', 'E7(a5)', 'A5+A1', 'D6', 'E7(a1)', 'A6+A1', 'D6(a1)', 'D6(a2)', 'A7', 'E8(b6)', 'E6+A1', 'E7(a4)', 'E8(a5)', 'E8', 'A4+A2+A1', 'D4(a1)+A2', 'E8(a1)', 'E6(a1)', 'A6', 'D5+A2', 'D7', 'E7(a3)', 'A3+A2+A1', 'E7(a2)', 'D4', 'D5+A1', '2A3', 'D5(a1)+A1', 'D4(a1)+A1', 'A3+A2', 'E6(a3)+A1', 'E8(a7)', 'D4+A1', 'D5(a1)', 'E8(a6)', 'E8(a2)', 'A5', 'A4+A2', 'E8(a3)', 'A4+2A1', 'D5', 'E8(a4)', 'D4+A2', 'E6(a3)', 'D7(a1)', 'E8(b4)', 'D5(a1)+A2', 'A4+A1', 'E6', 'A4', 'E6(a1)+A1', 'E8(b5)', 'E7', 'D7(a2)'], ['A3+A1', 'A4+A3', 'E7(a5)', 'A5+A1', 'A3+2A1', 'D6', 'E7(a1)', 'A6+A1', 'D6(a1)', 'D6(a2)', 'A7', 'E8(b6)', 'E6+A1', 'E7(a4)', 'E8(a5)', 'E8', 'A4+A2+A1', 'D4(a1)+A2', 'E8(a1)', 'E6(a1)', 'A6', 'D5+A2', 'D7', 'E7(a3)', 'A3+A2+A1', 'E7(a2)', 'D4', 'D5+A1', '2A3', 'D5(a1)+A1', 'D4(a1)', 'D4(a1)+A1', 'A3+A2', 'E6(a3)+A1', 'E8(a7)', 'D4+A1', 'D5(a1)', 'E8(a6)', 'E8(a2)', 'A5', 'A4+A2', 'E8(a3)', 'A4+2A1', 'D5', 'E8(a4)', 'D4+A2', 'E6(a3)', 'D7(a1)', 'E8(b4)', 'D5(a1)+A2', 'A4+A1', 'E6', 'A4', 'E6(a1)+A1', 'E8(b5)', 'E7', 'D7(a2)'], ['2A2+A1', 'A4+A3', 'E7(a5)', 'A5+A1', 'A3+2A1', 'D6', 'E7(a1)', 'A6+A1', 'D6(a1)', 'D6(a2)', 'A7', 'E8(b6)', 'E6+A1', 'E7(a4)', 'E8(a5)', 'E8', 'A4+A2+A1', 'D4(a1)+A2', 'E8(a1)', '2A2+2A1', 'E6(a1)', 'A6', 'D5+A2', 'D7', 'E7(a3)', 'A3+A2+A1', 'E7(a2)', 'D4', 'D5+A1', '2A3', 'D5(a1)+A1', 'D4(a1)', 'D4(a1)+A1', 'A3+A2', 'E6(a3)+A1', 'E8(a7)', 'D4+A1', 'D5(a1)', 'A3+A1', 'E8(a6)', 'E8(a2)', 'A5', 'A4+A2', 'E8(a3)', 'A4+2A1', 'D5', 'E8(a4)', 'D4+A2', 'E6(a3)', 'D7(a1)', 'E8(b4)', 'D5(a1)+A2', 'A4+A1', 'E6', 'A4', 'E6(a1)+A1', 'E8(b5)', 'E7', 'D7(a2)'], ['2A2', 'A4+A3', 'E7(a5)', 'A5+A1', 'A3+2A1', 'D6', 'E7(a1)', 'A6+A1', 'D6(a1)', 'D6(a2)', 'A7', 'E8(b6)', 'E6+A1', 'E7(a4)', 'E8(a5)', 'E8', 'A4+A2+A1', 'D4(a1)+A2', 'E8(a1)', '2A2+2A1', 'E6(a1)', 'A6', 'D5+A2', 'D7', 'E7(a3)', 'A3+A2+A1', 'E7(a2)', '2A2+A1', 'D4', 'D5+A1', '2A3', 'D5(a1)+A1', 'D4(a1)', 'D4(a1)+A1', 'A3+A2', 'E6(a3)+A1', 'E8(a7)', 'D4+A1', 'D5(a1)', 'A3+A1', 'E8(a6)', 'E8(a2)', 'A5', 'A4+A2', 'E8(a3)', 'A4+2A1', 'D5', 'E8(a4)', 'D4+A2', 'E6(a3)', 'D7(a1)', 'E8(b4)', 'D5(a1)+A2', 'A4+A1', 'E6', 'A4', 'E6(a1)+A1', 'E8(b5)', 'E7', 'D7(a2)'], ['A2+3A1', 'A4+A3', 'E7(a5)', 'A5+A1', 'A3+2A1', 'D6', 'E7(a1)', 'A6+A1', 'D6(a1)', 'D6(a2)', 'A7', 'E8(b6)', 'E6+A1', 'E7(a4)', 'E8(a5)', 'E8', 'A4+A2+A1', 'D4(a1)+A2', 'E8(a1)', '2A2+2A1', 'E6(a1)', 'A6', 'D5+A2', 'D7', 'E7(a3)', 'A3+A2+A1', 'E7(a2)', '2A2+A1', 'D4', 'D5+A1', '2A3', 'D5(a1)+A1', 'D4(a1)', 'D4(a1)+A1', 'A3+A2', 'E6(a3)+A1', 'E8(a7)', 'D4+A1', 'D5(a1)', 'A3+A1', 'E8(a6)', 'E8(a2)', 'A5', 'A4+A2', 'E8(a3)', 'A4+2A1', 'D5', 'E8(a4)', 'D4+A2', 'E6(a3)', 'D7(a1)', 'E8(b4)', 'D5(a1)+A2', 'A4+A1', 'E6', 'A4', 'E6(a1)+A1', '2A2', 'E8(b5)', 'E7', 'D7(a2)'], ['A3', 'A4+A3', 'E7(a5)', 'A5+A1', 'A3+2A1', 'D6', 'E7(a1)', 'A6+A1', 'D6(a1)', 'D6(a2)', 'A7', 'E8(b6)', 'E6+A1', 'E7(a4)', 'E8(a5)', 'E8', 'A4+A2+A1', 'D4(a1)+A2', 'E8(a1)', 'E6(a1)', 'A6', 'D5+A2', 'D7', 'E7(a3)', 'A3+A2+A1', 'E7(a2)', 'D4', 'D5+A1', '2A3', 'D5(a1)+A1', 'D4(a1)', 'D4(a1)+A1', 'A3+A2', 'E6(a3)+A1', 'E8(a7)', 'D4+A1', 'D5(a1)', 'A3+A1', 'E8(a6)', 'E8(a2)', 'A5', 'A4+A2', 'E8(a3)', 'A4+2A1', 'D5', 'E8(a4)', 'D4+A2', 'E6(a3)', 'D7(a1)', 'E8(b4)', 'D5(a1)+A2', 'A4+A1', 'E6', 'A4', 'E6(a1)+A1', 'E8(b5)', 'E7', 'D7(a2)'], ['A2+2A1', 'A4+A3', 'E7(a5)', 'A5+A1', 'A3+2A1', 'D6', 'E7(a1)', 'A6+A1', 'D6(a1)', 'D6(a2)', 'A7', 'E8(b6)', 'E6+A1', 'E7(a4)', 'E8(a5)', 'E8', 'A4+A2+A1', 'D4(a1)+A2', 'E8(a1)', '2A2+2A1', 'E6(a1)', 'A6', 'D5+A2', 'D7', 'E7(a3)', 'A3+A2+A1', 'E7(a2)', '2A2+A1', 'D4', 'D5+A1', '2A3', 'D5(a1)+A1', 'D4(a1)', 'D4(a1)+A1', 'A3+A2', 'E6(a3)+A1', 'E8(a7)', 'D4+A1', 'D5(a1)', 'A3+A1', 'E8(a6)', 'E8(a2)', 'A5', 'A3', 'A4+A2', 'E8(a3)', 'A4+2A1', 'D5', 'E8(a4)', 'D4+A2', 'E6(a3)', 'D7(a1)', 'E8(b4)', 'D5(a1)+A2', 'A4+A1', 'E6', 'A4', 'E6(a1)+A1', '2A2', 'A2+3A1', 'E8(b5)', 'E7', 'D7(a2)'], ['A2+A1', 'A4+A3', 'E7(a5)', 'A5+A1', 'A3+2A1', 'D6', 'E7(a1)', 'A6+A1', 'D6(a1)', 'D6(a2)', 'A7', 'E8(b6)', 'E6+A1', 'E7(a4)', 'E8(a5)', 'E8', 'A4+A2+A1', 'D4(a1)+A2', 'E8(a1)', '2A2+2A1', 'E6(a1)', 'A6', 'D5+A2', 'D7', 'E7(a3)', 'A2+2A1', 'A3+A2+A1', 'E7(a2)', '2A2+A1', 'D4', 'D5+A1', '2A3', 'D5(a1)+A1', 'D4(a1)', 'D4(a1)+A1', 'A3+A2', 'E6(a3)+A1', 'E8(a7)', 'D4+A1', 'D5(a1)', 'A3+A1', 'E8(a6)', 'E8(a2)', 'A5', 'A3', 'A4+A2', 'E8(a3)', 'A4+2A1', 'D5', 'E8(a4)', 'D4+A2', 'E6(a3)', 'D7(a1)', 'E8(b4)', 'D5(a1)+A2', 'A4+A1', 'E6', 'A4', 'E6(a1)+A1', '2A2', 'A2+3A1', 'E8(b5)', 'E7', 'D7(a2)'], ['4A1', 'A4+A3', 'E7(a5)', 'A5+A1', 'A3+2A1', 'D6', 'E7(a1)', 'A6+A1', 'D6(a1)', 'D6(a2)', 'A7', 'E8(b6)', 'E6+A1', 'E7(a4)', 'E8(a5)', 'E8', 'A4+A2+A1', 'D4(a1)+A2', 'E8(a1)', '2A2+2A1', 'E6(a1)', 'A6', 'D5+A2', 'D7', 'E7(a3)', 'A2+2A1', 'A3+A2+A1', 'E7(a2)', '2A2+A1', 'D4', 'D5+A1', '2A3', 'D5(a1)+A1', 'D4(a1)', 'D4(a1)+A1', 'A3+A2', 'E6(a3)+A1', 'E8(a7)', 'D4+A1', 'D5(a1)', 'A3+A1', 'E8(a6)', 'E8(a2)', 'A5', 'A3', 'A4+A2', 'E8(a3)', 'A4+2A1', 'A2+A1', 'D5', 'E8(a4)', 'D4+A2', 'E6(a3)', 'D7(a1)', 'E8(b4)', 'D5(a1)+A2', 'A4+A1', 'E6', 'A4', 'E6(a1)+A1', '2A2', 'A2+3A1', 'E8(b5)', 'E7', 'D7(a2)'], ['A2', 'A4+A3', 'E7(a5)', 'A5+A1', 'A3+2A1', 'D6', 'E7(a1)', 'A6+A1', 'D6(a1)', 'D6(a2)', 'A7', 'E8(b6)', 'E6+A1', 'E7(a4)', 'E8(a5)', 'E8', 'A4+A2+A1', 'D4(a1)+A2', 'E8(a1)', '2A2+2A1', 'E6(a1)', 'A6', 'D5+A2', 'D7', 'E7(a3)', 'A2+2A1', 'A3+A2+A1', 'E7(a2)', '2A2+A1', 'D4', 'D5+A1', '2A3', 'D5(a1)+A1', 'D4(a1)', 'D4(a1)+A1', 'A3+A2', 'E6(a3)+A1', 'E8(a7)', 'D4+A1', 'D5(a1)', 'A3+A1', 'E8(a6)', 'E8(a2)', 'A5', 'A3', 'A4+A2', 'E8(a3)', 'A4+2A1', 'A2+A1', 'D5', 'E8(a4)', 'D4+A2', 'E6(a3)', 'D7(a1)', 'E8(b4)', 'D5(a1)+A2', 'A4+A1', 'E6', 'A4', 'E6(a1)+A1', '2A2', 'A2+3A1', 'E8(b5)', 'E7', 'D7(a2)'], ['3A1', 'A4+A3', 'E7(a5)', 'A5+A1', 'A3+2A1', 'D6', 'E7(a1)', 'A6+A1', 'D6(a1)', 'D6(a2)', 'A7', 'E8(b6)', 'E6+A1', 'E7(a4)', 'E8(a5)', 'E8', 'A4+A2+A1', 'D4(a1)+A2', 'E8(a1)', '2A2+2A1', 'E6(a1)', 'A6', 'D5+A2', 'D7', 'E7(a3)', 'A2+2A1', 'A3+A2+A1', 'E7(a2)', '2A2+A1', 'D4', 'D5+A1', '2A3', 'D5(a1)+A1', 'D4(a1)', 'D4(a1)+A1', 'A3+A2', 'E6(a3)+A1', 'E8(a7)', 'D4+A1', 'D5(a1)', 'A3+A1', 'E8(a6)', 'E8(a2)', 'A5', 'A3', 'A4+A2', 'E8(a3)', 'A4+2A1', 'A2+A1', 'D5', 'E8(a4)', 'D4+A2', 'E6(a3)', '4A1', 'D7(a1)', 'E8(b4)', 'D5(a1)+A2', 'A4+A1', 'E6', 'A4', 'A2', 'E6(a1)+A1', '2A2', 'A2+3A1', 'E8(b5)', 'E7', 'D7(a2)'], ['2A1', 'A4+A3', 'E7(a5)', 'A5+A1', 'A3+2A1', 'D6', 'E7(a1)', 'A6+A1', 'D6(a1)', 'D6(a2)', 'A7', 'E8(b6)', 'E6+A1', 'E7(a4)', 'E8(a5)', 'E8', 'A4+A2+A1', 'D4(a1)+A2', 'E8(a1)', '2A2+2A1', 'E6(a1)', 'A6', 'D5+A2', 'D7', 'E7(a3)', 'A2+2A1', 'A3+A2+A1', 'E7(a2)', '2A2+A1', 'D4', 'D5+A1', '2A3', 'D5(a1)+A1', 'D4(a1)', 'D4(a1)+A1', 'A3+A2', 'E6(a3)+A1', 'E8(a7)', 'D4+A1', 'D5(a1)', 'A3+A1', 'E8(a6)', 'E8(a2)', 'A5', 'A3', 'A4+A2', 'E8(a3)', 'A4+2A1', 'A2+A1', 'D5', 'E8(a4)', 'D4+A2', 'E6(a3)', '4A1', 'D7(a1)', 'E8(b4)', 'D5(a1)+A2', 'A4+A1', 'E6', 'A4', 'A2', 'E6(a1)+A1', '2A2', '3A1', 'A2+3A1', 'E8(b5)', 'E7', 'D7(a2)'], ['A1', 'A4+A3', 'E7(a5)', 'A5+A1', 'A3+2A1', 'D6', 'E7(a1)', 'A6+A1', 'D6(a1)', 'D6(a2)', 'A7', 'E8(b6)', 'E6+A1', 'E7(a4)', 'E8(a5)', 'E8', 'A4+A2+A1', 'D4(a1)+A2', 'E8(a1)', '2A2+2A1', 'E6(a1)', 'A6', 'D5+A2', 'D7', 'E7(a3)', 'A2+2A1', 'A3+A2+A1', 'E7(a2)', '2A2+A1', 'D4', 'D5+A1', '2A3', 'D5(a1)+A1', 'D4(a1)', 'D4(a1)+A1', 'A3+A2', 'E6(a3)+A1', 'E8(a7)', 'D4+A1', 'D5(a1)', 'A3+A1', 'E8(a6)', 'E8(a2)', 'A5', 'A3', 'A4+A2', '2A1', 'E8(a3)', 'A4+2A1', 'A2+A1', 'D5', 'E8(a4)', 'D4+A2', 'E6(a3)', '4A1', 'D7(a1)', 'E8(b4)', 'D5(a1)+A2', 'A4+A1', 'E6', 'A4', 'A2', 'E6(a1)+A1', '2A2', '3A1', 'A2+3A1', 'E8(b5)', 'E7', 'D7(a2)'], ['1', 'A4+A3', 'E7(a5)', 'A5+A1', 'A3+2A1', 'D6', 'E7(a1)', 'A6+A1', 'D6(a1)', 'D6(a2)', 'A7', 'E8(b6)', 'E6+A1', 'E7(a4)', 'E8(a5)', 'E8', 'A4+A2+A1', 'D4(a1)+A2', 'E8(a1)', '2A2+2A1', 'E6(a1)', 'A6', 'D5+A2', 'D7', 'E7(a3)', 'A2+2A1', 'A3+A2+A1', 'E7(a2)', '2A2+A1', 'D4', 'D5+A1', '2A3', 'D5(a1)+A1', 'D4(a1)', 'D4(a1)+A1', 'A3+A2', 'E6(a3)+A1', 'E8(a7)', 'D4+A1', 'D5(a1)', 'A3+A1', 'E8(a6)', 'E8(a2)', 'A5', 'A3', 'A4+A2', '2A1', 'E8(a3)', 'A1', 'A4+2A1', 'A2+A1', 'D5', 'E8(a4)', 'D4+A2', 'E6(a3)', '4A1', 'D7(a1)', 'E8(b4)', 'D5(a1)+A2', 'A4+A1', 'E6', 'A4', 'A2', 'E6(a1)+A1', '2A2', '3A1', 'A2+3A1', 'E8(b5)', 'E7', 'D7(a2)']]})


When $W_c$ is of the type $E_8$

In [15]:
# Construct W of type E8
W=coxeter("E",8)

# Give the range of cover fold. Since the computation for E8 is slow, one could change the following range to do a piecewise calculation.
for cover_fold in range(1,30):
    
    # Take the minimal Phi_cy sets 
    Phi_cy_Set=[]
    for y1,y2,y3,y4,y5,y6,y7,y8 in itertools.product(range(2),range(2),range(2),range(2),range(2),range(2),range(2),range(2)):
        y=np.array([y1,y2,y3,y4,y5,y6,y7,y8])
        if np.sum(y*np.array(W.roots[119]))<=cover_fold:
            Phi_cy=[]
            if y1==0:
                Phi_cy.append(0)
            if y2==0:
                Phi_cy.append(1)
            if y3==0:
                Phi_cy.append(2)
            if y4==0:
                Phi_cy.append(3)
            if y5==0:
                Phi_cy.append(4)
            if y6==0:
                Phi_cy.append(5)
            if y7==0:
                Phi_cy.append(6)
            if y8==0:
                Phi_cy.append(7)
            if np.sum(y*np.array(W.roots[119]))==cover_fold:
                Phi_cy.append(119)
            Phi_cy_Set.append(Phi_cy)
    min_Phi_cy_Set=minimal_W_cy(Phi_cy_Set)
    
    # In the following, we compute by going through all representations
    Orbit_Set=[]
    Induc_Wcy_W_set=[]
    W_cy_special_rep_set=[]
    for Phi_cy in min_Phi_cy_Set:
        W_cy=reflectionsubgroup(W,Phi_cy)
        W_cy_chartable=pd.DataFrame(chartable(W_cy))
        W_cy_special_rep=W_cy_chartable.loc[W_cy_chartable['b']==W_cy_chartable['a']]['charnames']
        W_cy_special_rep_set.append(W_cy_special_rep)
        induc_Wcy_W=inductiontable(W_cy,W,invchar=(lambda G:chartable(G, chars=False)['b']))
        Induc_Wcy_W=pd.DataFrame(induc_Wcy_W['scalar'],columns=induc_Wcy_W['charH'],index=induc_Wcy_W['charW'])
        Induc_Wcy_W_set.append(Induc_Wcy_W)
    for char in E8_Data.loc[:,'charnames']:
        orbit_set=[]
        for i in range(len(min_Phi_cy_Set)):
            Phi_cy=min_Phi_cy_Set[i]
            W_cy=reflectionsubgroup(W,Phi_cy)
            W_cy_chartable=pd.DataFrame(chartable(W_cy))
            W_cy_special_rep=W_cy_special_rep_set[i]
            Induc_Wcy_W=Induc_Wcy_W_set[i]
            for rep_now in W_cy_special_rep:
                j_induction_now=(Induc_Wcy_W[rep_now][Induc_Wcy_W[rep_now]==1].index)[0]
                if list((E8_Data.loc[E8_Data['charnames']==str(j_induction_now)]).loc[:,'orbit'])[0] in list((E8_Data.loc[E8_Data['charnames']==char]).loc[:,'Georbit'])[0]:
                    if rep_now==("512_a'",):
                        rep_sign=rep_now
                    elif rep_now==("512_a'",str([1,1])):
                        rep_sign=("512_a'",str([2]))
                    elif rep_now==("512_a'",str([2])):
                        rep_sign=("512_a'",str([1,1]))
                    elif rep_now==("4096_x'",):
                        rep_sign=("4096_z",)
                    elif rep_now==("4096_z",):
                        rep_sign=("4096_x'",)
                    else:
                        rep_sign_position=(W_cy_chartable.loc[W_cy_chartable['charnames']==rep_now]).loc[:,'permsgn'].iloc[0]
                        rep_sign=W_cy_chartable.iloc[rep_sign_position,0]
                    j_induction_sign=(Induc_Wcy_W[rep_sign][Induc_Wcy_W[rep_sign]==1].index)[0]
                    orbit_set.append(list((E8_Data.loc[E8_Data['charnames']==str(j_induction_sign)]).loc[:,'orbit'])[0])
        Orbit_Set.append(list(set(orbit_set)))
    Max_Orbit_Set=[]
    for x in Orbit_Set:
            Max_Orbit_Set.append(max_orbit(x,E8_Data))
    
    # Restore the results in E8_Data
    E8_Data[str("E8_E8_")+str(cover_fold)]=Max_Orbit_Set

#I using also traces ... okay now
#I using also traces ... okay now
#I using also traces ... okay now
#I using also traces ... okay now
#I using also traces ... okay now


When $W_c$ is of the type $D_8$

In [16]:
# Construct W of type E8 and Wc of type D8
W1=coxeter("E",8)
W2=coxeter("D",8)

# Record the saturation dictionary
Sat_dict={('[[1, 1, 1, 1], +]',):'4A1',
 ('[[1, 1, 1, 1], -]',):'A2',
 ('[[1, 1, 1], [1, 1, 1, 1, 1]]',):'3A1',
 ('[[1, 1], [1, 1, 1, 1, 1, 1]]',):'2A1',
 ('[[1], [1, 1, 1, 1, 1, 1, 1]]',):'A1',
 ('[[], [1, 1, 1, 1, 1, 1, 1, 1]]',):'1',
 ('[[1, 1, 1], [2, 1, 1, 1]]',):'A2+2A1',
 ('[[1, 1], [2, 1, 1, 1, 1]]',):'A2+A1',
 ('[[1], [2, 1, 1, 1, 1, 1]]',):'A2',
 ('[[], [2, 1, 1, 1, 1, 1, 1]]',):'2A1',
 ('[[1, 1, 1], [2, 2, 1]]',):'2A2+A1',
 ('[[2, 1, 1], +]',):'A3+2A1',
 ('[[2, 1, 1], -]',):'D4(a1)',
 ('[[1, 1], [2, 2, 1, 1]]',):'2A2',
 ('[[2, 1], [2, 1, 1, 1]]',):'A3+A1',
 ('[[1], [2, 2, 1, 1, 1]]',):'A2+2A1',
 ('[[2], [2, 1, 1, 1, 1]]',):'A3',
 ('[[], [2, 2, 1, 1, 1, 1]]',):'A2+A1',
 ('[[1, 1], [2, 2, 2]]',):'2A2+2A1',
 ('[[2, 1], [2, 2, 1]]',):'A3+A2',
 ('[[1], [2, 2, 2, 1]]',):'A2+3A1',
 ('[[2], [2, 2, 1, 1]]',):'A3+2A1',
 ('[[], [2, 2, 2, 1, 1]]',):'4A1',
 ('[[2, 2], +]',):'2A3',
 ('[[2, 2], -]',):'A4',
 ('[[2], [2, 2, 2]]',):'D4(a1)+A1',
 ('[[], [2, 2, 2, 2]]',):'A2+A1',
 ('[[1, 1, 1], [3, 1, 1]]',):'A3+A2',
 ('[[1, 1], [3, 1, 1, 1]]',):'D4(a1)+A1',
 ('[[1], [3, 1, 1, 1, 1]]',):'D4(a1)',
 ('[[], [3, 1, 1, 1, 1, 1]]',):'A3',
 ('[[1, 1], [3, 2, 1]]',):'D4(a1)+A2',
 ('[[2, 1], [3, 1, 1]]',):'A4+A1',
 ('[[1], [3, 2, 1, 1]]',):'A3+A2',
 ('[[2], [3, 1, 1, 1]]',):'A4',
 ('[[], [3, 2, 1, 1, 1]]',):'A4+A1',
 ('[[2, 1], [3, 2]]',):'A4+A2',
 ('[[1], [3, 2, 2]]',):'A3+A2+A1',
 ('[[2], [3, 2, 1]]',):'A4+2A1',
 ('[[], [3, 2, 2, 1]]',):'A3+2A1',
 ('[[1, 1], [3, 3]]',):'A4+2A1',
 ('[[3, 1], +]',):'A5+A1',
 ('[[3, 1], -]',):'E6(a3)',
 ('[[1], [3, 3, 1]]',):'2A3',
 ('[[3], [3, 1, 1]]',):'A5',
 ('[[2], [3, 3]]',):'A4+A3',
 ('[[3], [3, 2]]',):'E6(a3)+A1',
 ('[[1, 1], [4, 1, 1]]',):'D5(a1)+A1',
 ('[[1], [4, 1, 1, 1]]',):'D5(a1)',
 ('[[], [4, 1, 1, 1, 1]]',):'D4',
 ('[[1, 1], [4, 2]]',):'D5(a1)+A2',
 ('[[2, 1], [4, 1]]',):'E7(a5)',
 ('[[1], [4, 2, 1]]',):'D4+A2',
 ('[[2], [4, 1, 1]]',):'D6(a2)',
 ('[[], [4, 2, 1, 1]]',):'D4+A1',
 ('[[2, 2], [4]]',):'E8(a7)',
 ('[[2], [4, 2]]',):'E8(a7)',
 ('[[], [4, 2, 2]]',):'D5(a1)',
 ('[[1], [4, 3]]',):'D6(a2)',
 ('[[3], [4, 1]]',):'A6',
 ('[[4], +]',):'A7',
 ('[[4], -]',):'E6(a1)',
 ('[[], [4, 4]]',):'E8(a7)',
 ('[[1, 1], [5, 1]]',):'E7(a4)',
 ('[[1], [5, 1, 1]]',):'D6(a1)',
 ('[[], [5, 1, 1, 1]]',):'D5',
 ('[[1], [5, 2]]',):'D5+A2',
 ('[[2], [5, 1]]',):'D7(a2)',
 ('[[], [5, 2, 1]]',):'D5+A1',
 ('[[3], [5]]',):'E8(b6)',
 ('[[1], [6, 1]]',):'D7(a1)',
 ('[[], [6, 1, 1]]',):'D6',
 ('[[2], [6]]',):'E8(a6)',
 ('[[], [6, 2]]',):'E7(a3)',
 ('[[1], [7]]',):'E8(a5)',
 ('[[], [7, 1]]',):'D7',
 ('[[], [8]]',):'E8(a4)'}

weight=[3,4,6,5,4,3,2]
drop_weight=2
node_set_1=[list(p) for p in itertools.product(range(drop_weight+1), repeat=len(weight))]
node_set_2=[]
for node in node_set_1:
    c=(-np.sum(np.array(node)*weight)-1)%drop_weight
    node_set_2.append(np.append(node,c))
    if c==0:
        node_set_2.append(np.append(node,drop_weight))

for cover_fold in range(1,16):
    node_set_3=[]
    for node in node_set_2:
        if node[0]+node[1]+2*node[2]+2*node[3]+2*node[4]+2*node[5]+2*node[6]+node[7]<=cover_fold:
            add_point=[]
            for i in range(8):
                if node[i]==0:
                    add_point.append(i)
            if node[0]+node[1]+2*node[2]+2*node[3]+2*node[4]+2*node[5]+2*node[6]+node[7]==cover_fold:
                add_point.append(111)
            node_set_3.append(add_point)

    
    node_set_3=[list(tpl) for tpl in {tuple(item) for item in node_set_3}]
    minimal_node_set_W2=minimal_W_cy(node_set_3)
    
    dict={0:1,1:2,2:3,3:4,4:5,5:6,6:7,7:239}
    minimal_node_set_W1=[]
    for node in minimal_node_set_W2:
        minimal_node_set_W1.append([convert_fuc(W1.roots,W2.roots,Index,dict) for Index in node])

    Target=[]
    Induc_W1cy_W1_set=[]
    Induc_W2cy_W2_set=[]
    W1_cy_special_rep_set=[]
    for i in range(len(minimal_node_set_W1)):
        Phi1_cy=minimal_node_set_W1[i]
        Phi2_cy=minimal_node_set_W2[i]
        W1_cy=reflectionsubgroup(W1,Phi1_cy)
        W2_cy=reflectionsubgroup(W2,Phi2_cy)
        W1_cy_chartable=pd.DataFrame(chartable(W1_cy))
        W1_cy_special_rep=W1_cy_chartable.loc[W1_cy_chartable['b']==W1_cy_chartable['a']]['charnames']
        W1_cy_special_rep_set.append(W1_cy_special_rep)
        induc_W1cy_W1=inductiontable(W1_cy,W1,invchar=(lambda G:chartable(G, chars=False)['b']))
        Induc_W1cy_W1=pd.DataFrame(induc_W1cy_W1['scalar'],columns=induc_W1cy_W1['charH'],index=induc_W1cy_W1['charW'])
        Induc_W1cy_W1_set.append(Induc_W1cy_W1)
        induc_W2cy_W2=inductiontable(W2_cy,W2,invchar=(lambda G:chartable(G, chars=False)['b']))
        Induc_W2cy_W2=pd.DataFrame(induc_W2cy_W2['scalar'],columns=induc_W2cy_W2['charH'],index=induc_W2cy_W2['charW'])
        Induc_W2cy_W2_set.append(Induc_W2cy_W2)
    for char in E8_Data.loc[:,'charnames']:
        j_induction_sign_set=[]
        for i in range(len(minimal_node_set_W1)):
            Phi1_cy=minimal_node_set_W1[i]
            Phi2_cy=minimal_node_set_W2[i]
            W1_cy=reflectionsubgroup(W1,Phi1_cy)
            W2_cy=reflectionsubgroup(W2,Phi2_cy)
            W1_cy_chartable=pd.DataFrame(chartable(W1_cy))
            W1_cy_special_rep=W1_cy_special_rep_set[i]
            Induc_W1cy_W1=Induc_W1cy_W1_set[i]
            Induc_W2cy_W2=Induc_W2cy_W2_set[i]
            for rep_now in W1_cy_special_rep:
                j_induction_now=(Induc_W1cy_W1[rep_now][Induc_W1cy_W1[rep_now]==1].index)[0]
                if list((E8_Data.loc[E8_Data['charnames']==str(j_induction_now)]).loc[:,'orbit'])[0] in list((E8_Data.loc[E8_Data['charnames']==char]).loc[:,'Georbit'])[0]:
                    rep_sign_position=(W1_cy_chartable.loc[W1_cy_chartable['charnames']==rep_now]).loc[:,'permsgn'].iloc[0]
                    rep_sign=W1_cy_chartable.iloc[rep_sign_position,0]
                    if rep_sign not in Induc_W2cy_W2.columns:
                        rep_sign=switch_A3_D3(rep_sign,Induc_W2cy_W2.columns)
                    j_induction_sign=(Induc_W2cy_W2[rep_sign][Induc_W2cy_W2[rep_sign]==1].index)[0]
                    j_induction_sign_set.append(j_induction_sign)
        Target.append(max_orbit(list(set([Sat_dict.get(x) for x in set(j_induction_sign_set)])),E8_Data))
    E8_Data["E8_D8_"+str(cover_fold)]=Target

#I using also traces ... okay now
#I using also traces ... okay now
#I using also traces ... okay now
#I using also traces ... okay now


When $W_c$ is of the type $A_1+A_7$:

In [17]:
# Construct W of type E8 and Wc of type A1+A7
W1=coxeter("E",8)
W2=coxeter([[2, 0, 0, 0, 0, 0, 0, 0], [0, 2, -1, 0, 0, 0, 0, 0], [0, -1, 2, -1, 0, 0, 0, 0], [0, 0, -1, 2, -1, 0, 0, 0], [0, 0, 0, -1, 2, -1, 0, 0], [0, 0, 0, 0, -1, 2, -1, 0], [0, 0, 0, 0, 0, -1, 2, -1], [0, 0, 0, 0, 0, 0, -1, 2]])

# Record the saturation dictionary
Sat_dict={('[1, 1, 1, 1, 1, 1, 1, 1]', '[1, 1]'):'1',
 ('[1, 1, 1, 1, 1, 1, 1, 1]', '[2]'):'A1',
 ('[2, 1, 1, 1, 1, 1, 1]', '[1, 1]'):'A1',
 ('[2, 1, 1, 1, 1, 1, 1]', '[2]'):'2A1',
 ('[2, 2, 1, 1, 1, 1]', '[1, 1]'):'2A1',
 ('[2, 2, 1, 1, 1, 1]', '[2]'):'3A1',
 ('[2, 2, 2, 1, 1]', '[1, 1]'):'3A1',
 ('[2, 2, 2, 1, 1]', '[2]'):'4A1',
 ('[2, 2, 2, 2]', '[1, 1]'):'A2',
 ('[2, 2, 2, 2]', '[2]'):'A2+A1',
 ('[3, 1, 1, 1, 1, 1]', '[1, 1]'):'A2',
 ('[3, 1, 1, 1, 1, 1]', '[2]'):'A2+A1',
 ('[3, 2, 1, 1, 1]', '[1, 1]'):'A2+A1',
 ('[3, 2, 1, 1, 1]', '[2]'):'A2+2A1',
 ('[3, 2, 2, 1]', '[1, 1]'):'A2+2A1',
 ('[3, 2, 2, 1]', '[2]'):'A2+3A1',
 ('[3, 3, 1, 1]', '[1, 1]'):'2A2',
 ('[3, 3, 1, 1]', '[2]'):'2A2+A1',
 ('[3, 3, 2]', '[1, 1]'):'2A2+A1',
 ('[3, 3, 2]', '[2]'):'2A2+2A1',
 ('[4, 1, 1, 1, 1]', '[1, 1]'):'A3',
 ('[4, 1, 1, 1, 1]', '[2]'):'A3+A1',
 ('[4, 2, 1, 1]', '[1, 1]'):'A3+A1',
 ('[4, 2, 1, 1]', '[2]'):'A3+2A1',
 ('[4, 2, 2]', '[1, 1]'):'D4(a1)',
 ('[4, 2, 2]', '[2]'):'D4(a1)+A1',
 ('[4, 3, 1]', '[1, 1]'):'A3+A2',
 ('[4, 3, 1]', '[2]'):'A3+A2+A1',
 ('[4, 4]', '[1, 1]'):'A4',
 ('[4, 4]', '[2]'):'A4+A1',
 ('[5, 1, 1, 1]', '[1, 1]'):'A4',
 ('[5, 1, 1, 1]', '[2]'):'A4+A1',
 ('[5, 2, 1]', '[1, 1]'):'A4+A1',
 ('[5, 2, 1]', '[2]'):'A4+2A1',
 ('[5, 3]', '[1, 1]'):'A4+A2',
 ('[5, 3]', '[2]'):'A4+A2+A1',
 ('[6, 1, 1]', '[1, 1]'):'A5',
 ('[6, 1, 1]', '[2]'):'A5+A1',
 ('[6, 2]', '[1, 1]'):'E6(a3)',
 ('[6, 2]', '[2]'):'E6(a3)+A1',
 ('[7, 1]', '[1, 1]'):'A6',
 ('[7, 1]', '[2]'):'A6+A1',
 ('[8]', '[1, 1]'):'E6(a1)',
 ('[8]', '[2]'):'E6(a1)+A1'}

weight=[2,3,6,5,4,3,2]
drop_weight=4
node_set_1=[list(p) for p in itertools.product(range(drop_weight+1), repeat=len(weight))]
node_set_2=[]
for node in node_set_1:
    c=(-np.sum(np.array(node)*weight)-1)%drop_weight
    node_set_2.append(np.append(node,c))
    if c==0:
        node_set_2.append(np.append(node,drop_weight))

for cover_fold in range(1,10):
    node_set_3=[]
    for node in node_set_2:
        if node[0]<=cover_fold and node[1]+node[2]+node[3]+node[4]+node[5]+node[6]+node[7]<=cover_fold:
            add_point=[]
            for i in range(8):
                if node[i]==0:
                    add_point.append(i)
            if node[0]==cover_fold:
                add_point.append(0)
            if node[1]+node[2]+node[3]+node[4]+node[5]+node[6]+node[7]==cover_fold:
                add_point.append(57)
            node_set_3.append(add_point)

    
    node_set_3=[list(tpl) for tpl in {tuple(item) for item in node_set_3}]
    minimal_node_set_W2=minimal_W_cy(node_set_3)
    
    dict={0:0,1:1,2:3,3:4,4:5,5:6,6:7,7:239}
    minimal_node_set_W1=[]
    for node in minimal_node_set_W2:
        minimal_node_set_W1.append([convert_fuc(W1.roots,W2.roots,Index,dict) for Index in node])

    Target=[]
    Induc_W1cy_W1_set=[]
    Induc_W2cy_W2_set=[]
    W1_cy_special_rep_set=[]
    for i in range(len(minimal_node_set_W1)):
        Phi1_cy=minimal_node_set_W1[i]
        Phi2_cy=minimal_node_set_W2[i]
        W1_cy=reflectionsubgroup(W1,Phi1_cy)
        W2_cy=reflectionsubgroup(W2,Phi2_cy)
        W1_cy_chartable=pd.DataFrame(chartable(W1_cy))
        W1_cy_special_rep=W1_cy_chartable.loc[W1_cy_chartable['b']==W1_cy_chartable['a']]['charnames']
        W1_cy_special_rep_set.append(W1_cy_special_rep)
        induc_W1cy_W1=inductiontable(W1_cy,W1,invchar=(lambda G:chartable(G, chars=False)['b']))
        Induc_W1cy_W1=pd.DataFrame(induc_W1cy_W1['scalar'],columns=induc_W1cy_W1['charH'],index=induc_W1cy_W1['charW'])
        Induc_W1cy_W1_set.append(Induc_W1cy_W1)
        induc_W2cy_W2=inductiontable(W2_cy,W2,invchar=(lambda G:chartable(G, chars=False)['b']))
        Induc_W2cy_W2=pd.DataFrame(induc_W2cy_W2['scalar'],columns=induc_W2cy_W2['charH'],index=induc_W2cy_W2['charW'])
        Induc_W2cy_W2_set.append(Induc_W2cy_W2)
    for char in E8_Data.loc[:,'charnames']:
        j_induction_sign_set=[]
        for i in range(len(minimal_node_set_W1)):
            Phi1_cy=minimal_node_set_W1[i]
            Phi2_cy=minimal_node_set_W2[i]
            W1_cy=reflectionsubgroup(W1,Phi1_cy)
            W2_cy=reflectionsubgroup(W2,Phi2_cy)
            W1_cy_chartable=pd.DataFrame(chartable(W1_cy))
            W1_cy_special_rep=W1_cy_special_rep_set[i]
            Induc_W1cy_W1=Induc_W1cy_W1_set[i]
            Induc_W2cy_W2=Induc_W2cy_W2_set[i]
            for rep_now in W1_cy_special_rep:
                j_induction_now=(Induc_W1cy_W1[rep_now][Induc_W1cy_W1[rep_now]==1].index)[0]
                if list((E8_Data.loc[E8_Data['charnames']==str(j_induction_now)]).loc[:,'orbit'])[0] in list((E8_Data.loc[E8_Data['charnames']==char]).loc[:,'Georbit'])[0]:
                    rep_sign_position=(W1_cy_chartable.loc[W1_cy_chartable['charnames']==rep_now]).loc[:,'permsgn'].iloc[0]
                    rep_sign=W1_cy_chartable.iloc[rep_sign_position,0]
                    if rep_sign not in Induc_W2cy_W2.columns:
                        rep_sign=switch_A3_D3(rep_sign,Induc_W2cy_W2.columns)
                    j_induction_sign=(Induc_W2cy_W2[rep_sign][Induc_W2cy_W2[rep_sign]==1].index)[0]
                    j_induction_sign_set.append(j_induction_sign)
        Target.append(max_orbit(list(set([Sat_dict.get(x) for x in set(j_induction_sign_set)])),E8_Data))
    E8_Data["E8_A1+A7_"+str(cover_fold)]=Target

When $W_c$ is of the type $A_2+A_1+A_5$:

In [18]:
# Construct W of type E8 and Wc of type A2+A1+A5
W1=coxeter("E",8)
W2=coxeter([[2,-1,0,0,0,0,0,0],[-1,2,0,0,0,0,0,0],[0,0,2,0,0,0,0,0],[0,0,0,2,-1,0,0,0],[0,0,0,-1,2,-1,0,0],[0,0,0,0,-1,2,-1,0],[0,0,0,0,0,-1,2,-1],[0,0,0,0,0,0,-1,2]])
# Record the saturation dictionary
Sat_dict={('[1, 1, 1, 1, 1, 1]', '[1, 1, 1]', '[1, 1]'):'1',
 ('[1, 1, 1, 1, 1, 1]', '[1, 1, 1]', '[2]'):'A1',
 ('[1, 1, 1, 1, 1, 1]', '[2, 1]', '[1, 1]'):'A1',
 ('[1, 1, 1, 1, 1, 1]', '[2, 1]', '[2]'):'2A1',
 ('[1, 1, 1, 1, 1, 1]', '[3]', '[1, 1]'):'A2',
 ('[1, 1, 1, 1, 1, 1]', '[3]', '[2]'):'A2+A1',
 ('[2, 1, 1, 1, 1]', '[1, 1, 1]', '[1, 1]'):'A1',
 ('[2, 1, 1, 1, 1]', '[1, 1, 1]', '[2]'):'2A1',
 ('[2, 1, 1, 1, 1]', '[2, 1]', '[1, 1]'):'2A1',
 ('[2, 1, 1, 1, 1]', '[2, 1]', '[2]'):'3A1',
 ('[2, 1, 1, 1, 1]', '[3]', '[1, 1]'):'A2+A1',
 ('[2, 1, 1, 1, 1]', '[3]', '[2]'):'A2+2A1',
 ('[2, 2, 1, 1]', '[1, 1, 1]', '[1, 1]'):'2A1',
 ('[2, 2, 1, 1]', '[1, 1, 1]', '[2]'):'3A1',
 ('[2, 2, 1, 1]', '[2, 1]', '[1, 1]'):'3A1',
 ('[2, 2, 1, 1]', '[2, 1]', '[2]'):'4A1',
 ('[2, 2, 1, 1]', '[3]', '[1, 1]'):'A2+2A1',
 ('[2, 2, 1, 1]', '[3]', '[2]'):'A2+3A1',
 ('[2, 2, 2]', '[1, 1, 1]', '[1, 1]'):'3A1',
 ('[2, 2, 2]', '[1, 1, 1]', '[2]'):'A2',
 ('[2, 2, 2]', '[2, 1]', '[1, 1]'):'4A1',
 ('[2, 2, 2]', '[2, 1]', '[2]'):'A2+A1',
 ('[2, 2, 2]', '[3]', '[1, 1]'):'A2+3A1',
 ('[2, 2, 2]', '[3]', '[2]'):'2A2',
 ('[3, 1, 1, 1]', '[1, 1, 1]', '[1, 1]'):'A2',
 ('[3, 1, 1, 1]', '[1, 1, 1]', '[2]'):'A2+A1',
 ('[3, 1, 1, 1]', '[2, 1]', '[1, 1]'):'A2+A1',
 ('[3, 1, 1, 1]', '[2, 1]', '[2]'):'A2+2A1',
 ('[3, 1, 1, 1]', '[3]', '[1, 1]'):'2A2',
 ('[3, 1, 1, 1]', '[3]', '[2]'):'2A2+A1',
 ('[3, 2, 1]', '[1, 1, 1]', '[1, 1]'):'A2+A1',
 ('[3, 2, 1]', '[1, 1, 1]', '[2]'):'A2+2A1',
 ('[3, 2, 1]', '[2, 1]', '[1, 1]'):'A2+2A1',
 ('[3, 2, 1]', '[2, 1]', '[2]'):'A2+3A1',
 ('[3, 2, 1]', '[3]', '[1, 1]'):'2A2+A1',
 ('[3, 2, 1]', '[3]', '[2]'):'2A2+2A1',
 ('[3, 3]', '[1, 1, 1]', '[1, 1]'):'2A2',
 ('[3, 3]', '[1, 1, 1]', '[2]'):'2A2+A1',
 ('[3, 3]', '[2, 1]', '[1, 1]'):'2A2+A1',
 ('[3, 3]', '[2, 1]', '[2]'):'2A2+2A1',
 ('[3, 3]', '[3]', '[1, 1]'):'D4(a1)',
 ('[3, 3]', '[3]', '[2]'):'D4(a1)+A1',
 ('[4, 1, 1]', '[1, 1, 1]', '[1, 1]'):'A3',
 ('[4, 1, 1]', '[1, 1, 1]', '[2]'):'A3+A1',
 ('[4, 1, 1]', '[2, 1]', '[1, 1]'):'A3+A1',
 ('[4, 1, 1]', '[2, 1]', '[2]'):'A3+2A1',
 ('[4, 1, 1]', '[3]', '[1, 1]'):'A3+A2',
 ('[4, 1, 1]', '[3]', '[2]'):'A3+A2+A1',
 ('[4, 2]', '[1, 1, 1]', '[1, 1]'):'A3+A1',
 ('[4, 2]', '[1, 1, 1]', '[2]'):'D4(a1)',
 ('[4, 2]', '[2, 1]', '[1, 1]'):'A3+2A1',
 ('[4, 2]', '[2, 1]', '[2]'):'D4(a1)+A1',
 ('[4, 2]', '[3]', '[1, 1]'):'A3+A2+A1',
 ('[4, 2]', '[3]', '[2]'):'D4(a1)+A2',
 ('[5, 1]', '[1, 1, 1]', '[1, 1]'):'A4',
 ('[5, 1]', '[1, 1, 1]', '[2]'):'A4+A1',
 ('[5, 1]', '[2, 1]', '[1, 1]'):'A4+A1',
 ('[5, 1]', '[2, 1]', '[2]'):'A4+2A1',
 ('[5, 1]', '[3]', '[1, 1]'):'A4+A2',
 ('[5, 1]', '[3]', '[2]'):'A4+A2+A1',
 ('[6]', '[1, 1, 1]', '[1, 1]'):'A5',
 ('[6]', '[1, 1, 1]', '[2]'):'E6(a3)',
 ('[6]', '[2, 1]', '[1, 1]'):'A5+A1',
 ('[6]', '[2, 1]', '[2]'):'E6(a3)+A1',
 ('[6]', '[3]', '[1, 1]'):'E7(a5)',
 ('[6]', '[3]', '[2]'):'E8(a7)'}

weight=[2,4,3,5,4,3,2]
drop_weight=6
node_set_1=[list(p) for p in itertools.product(range(drop_weight+1), repeat=len(weight))]
node_set_2=[]
for node in node_set_1:
    c=(-np.sum(np.array(node)*weight)-1)%drop_weight
    node_set_2.append(np.append(node,c))
    if c==0:
        node_set_2.append(np.append(node,drop_weight))

for cover_fold in range(1,7):
    node_set_3=[]
    for node in node_set_2:
        if node[0]+node[1]<=cover_fold and node[2]<=cover_fold and node[3]+node[4]+node[5]+node[6]+node[7]<=cover_fold:
            add_point=[]
            for i in range(8):
                if node[i]==0:
                    add_point.append(i)
            if node[0]+node[1]==cover_fold:
                add_point.append(8)
            if node[2]==cover_fold:
                add_point.append(2)
            if node[3]+node[4]+node[5]+node[6]+node[7]==cover_fold:
                add_point.append(37)
            node_set_3.append(add_point)

    
    node_set_3=[list(tpl) for tpl in {tuple(item) for item in node_set_3}]
    minimal_node_set_W2=minimal_W_cy(node_set_3)
    
    dict={0:0,1:2,2:1,3:4,4:5,5:6,6:7,7:239}
    minimal_node_set_W1=[]
    for node in minimal_node_set_W2:
        minimal_node_set_W1.append([convert_fuc(W1.roots,W2.roots,Index,dict) for Index in node])

    Target=[]
    Induc_W1cy_W1_set=[]
    Induc_W2cy_W2_set=[]
    W1_cy_special_rep_set=[]
    for i in range(len(minimal_node_set_W1)):
        Phi1_cy=minimal_node_set_W1[i]
        Phi2_cy=minimal_node_set_W2[i]
        W1_cy=reflectionsubgroup(W1,Phi1_cy)
        W2_cy=reflectionsubgroup(W2,Phi2_cy)
        W1_cy_chartable=pd.DataFrame(chartable(W1_cy))
        W1_cy_special_rep=W1_cy_chartable.loc[W1_cy_chartable['b']==W1_cy_chartable['a']]['charnames']
        W1_cy_special_rep_set.append(W1_cy_special_rep)
        induc_W1cy_W1=inductiontable(W1_cy,W1,invchar=(lambda G:chartable(G, chars=False)['b']))
        Induc_W1cy_W1=pd.DataFrame(induc_W1cy_W1['scalar'],columns=induc_W1cy_W1['charH'],index=induc_W1cy_W1['charW'])
        Induc_W1cy_W1_set.append(Induc_W1cy_W1)
        induc_W2cy_W2=inductiontable(W2_cy,W2,invchar=(lambda G:chartable(G, chars=False)['b']))
        Induc_W2cy_W2=pd.DataFrame(induc_W2cy_W2['scalar'],columns=induc_W2cy_W2['charH'],index=induc_W2cy_W2['charW'])
        Induc_W2cy_W2_set.append(Induc_W2cy_W2)
    for char in E8_Data.loc[:,'charnames']:
        j_induction_sign_set=[]
        for i in range(len(minimal_node_set_W1)):
            Phi1_cy=minimal_node_set_W1[i]
            Phi2_cy=minimal_node_set_W2[i]
            W1_cy=reflectionsubgroup(W1,Phi1_cy)
            W2_cy=reflectionsubgroup(W2,Phi2_cy)
            W1_cy_chartable=pd.DataFrame(chartable(W1_cy))
            W1_cy_special_rep=W1_cy_special_rep_set[i]
            Induc_W1cy_W1=Induc_W1cy_W1_set[i]
            Induc_W2cy_W2=Induc_W2cy_W2_set[i]
            for rep_now in W1_cy_special_rep:
                j_induction_now=(Induc_W1cy_W1[rep_now][Induc_W1cy_W1[rep_now]==1].index)[0]
                if list((E8_Data.loc[E8_Data['charnames']==str(j_induction_now)]).loc[:,'orbit'])[0] in list((E8_Data.loc[E8_Data['charnames']==char]).loc[:,'Georbit'])[0]:
                    rep_sign_position=(W1_cy_chartable.loc[W1_cy_chartable['charnames']==rep_now]).loc[:,'permsgn'].iloc[0]
                    rep_sign=W1_cy_chartable.iloc[rep_sign_position,0]
                    if rep_sign not in Induc_W2cy_W2.columns:
                        rep_sign=switch_A3_D3(rep_sign,Induc_W2cy_W2.columns)
                    j_induction_sign=(Induc_W2cy_W2[rep_sign][Induc_W2cy_W2[rep_sign]==1].index)[0]
                    j_induction_sign_set.append(j_induction_sign)
        Target.append(max_orbit(list(set([Sat_dict.get(x) for x in set(j_induction_sign_set)])),E8_Data))
    E8_Data["E8_A2+A1+A5_"+str(cover_fold)]=Target

#I using also traces ... okay now
#I using also traces ... okay now
#I using also traces ... okay now
#I using also traces ... okay now
#I using also traces ... okay now
#I using also traces ... okay now


When $W_c$ is of the type $A_8$:

In [19]:
# Construct W of type E8 and Wc of type A8
W1=coxeter("E",8)
W2=coxeter("A",8)
# Record the saturation dictionary
Sat_dict={('[1, 1, 1, 1, 1, 1, 1, 1, 1]',):'1',
 ('[2, 1, 1, 1, 1, 1, 1, 1]',):'A1',
 ('[2, 2, 1, 1, 1, 1, 1]',):'2A1',
 ('[2, 2, 2, 1, 1, 1]',):'3A1',
 ('[2, 2, 2, 2, 1]',):'4A1',
 ('[3, 1, 1, 1, 1, 1, 1]',):'A2',
 ('[3, 2, 1, 1, 1, 1]',):'A2+A1',
 ('[3, 2, 2, 1, 1]',):'A2+2A1',
 ('[3, 2, 2, 2]',):'A2+3A1',
 ('[3, 3, 1, 1, 1]',):'2A2',
 ('[3, 3, 2, 1]',):'2A2+A1',
 ('[3, 3, 3]',):'D4(a1)',
 ('[4, 1, 1, 1, 1, 1]',):'A3',
 ('[4, 2, 1, 1, 1]',):'A3+A1',
 ('[4, 2, 2, 1]',):'A3+2A1',
 ('[4, 3, 1, 1]',):'A3+A2',
 ('[4, 3, 2]',):'A3+A2+A1',
 ('[4, 4, 1]',):'2A3',
 ('[5, 1, 1, 1, 1]',):'A4',
 ('[5, 2, 1, 1]',):'A4+A1',
 ('[5, 2, 2]',):'A4+2A1',
 ('[5, 3, 1]',):'A4+A2',
 ('[5, 4]',):'A4+A3',
 ('[6, 1, 1, 1]',):'A5',
 ('[6, 2, 1]',):'A5+A1',
 ('[6, 3]',):'E7(a5)',
 ('[7, 1, 1]',):'A6',
 ('[7, 2]',):'A6+A1',
 ('[8, 1]',):'A7',
 ('[9]',):'E8(a6)'}

weight=[2,4,6,5,4,3,2]
drop_weight=3
node_set_1=[list(p) for p in itertools.product(range(drop_weight+1), repeat=len(weight))]
node_set_2=[]
for node in node_set_1:
    c=(-np.sum(np.array(node)*weight)-1)%drop_weight
    node_set_2.append(np.append(node,c))
    if c==0:
        node_set_2.append(np.append(node,drop_weight))

for cover_fold in range(1,11):
    node_set_3=[]
    for node in node_set_2:
        if node[0]+node[1]+node[2]+node[3]+node[4]+node[5]+node[6]+node[7]<=cover_fold:
            add_point=[]
            for i in range(8):
                if node[i]==0:
                    add_point.append(i)
            if node[0]+node[1]+node[2]+node[3]+node[4]+node[5]+node[6]+node[7]==cover_fold:
                add_point.append(71)
            node_set_3.append(add_point)

    
    node_set_3=[list(tpl) for tpl in {tuple(item) for item in node_set_3}]
    minimal_node_set_W2=minimal_W_cy(node_set_3)
    
    dict={0:0,1:2,2:3,3:4,4:5,5:6,6:7,7:239}
    minimal_node_set_W1=[]
    for node in minimal_node_set_W2:
        minimal_node_set_W1.append([convert_fuc(W1.roots,W2.roots,Index,dict) for Index in node])

    Target=[]
    Induc_W1cy_W1_set=[]
    Induc_W2cy_W2_set=[]
    W1_cy_special_rep_set=[]
    for i in range(len(minimal_node_set_W1)):
        Phi1_cy=minimal_node_set_W1[i]
        Phi2_cy=minimal_node_set_W2[i]
        W1_cy=reflectionsubgroup(W1,Phi1_cy)
        W2_cy=reflectionsubgroup(W2,Phi2_cy)
        W1_cy_chartable=pd.DataFrame(chartable(W1_cy))
        W1_cy_special_rep=W1_cy_chartable.loc[W1_cy_chartable['b']==W1_cy_chartable['a']]['charnames']
        W1_cy_special_rep_set.append(W1_cy_special_rep)
        induc_W1cy_W1=inductiontable(W1_cy,W1,invchar=(lambda G:chartable(G, chars=False)['b']))
        Induc_W1cy_W1=pd.DataFrame(induc_W1cy_W1['scalar'],columns=induc_W1cy_W1['charH'],index=induc_W1cy_W1['charW'])
        Induc_W1cy_W1_set.append(Induc_W1cy_W1)
        induc_W2cy_W2=inductiontable(W2_cy,W2,invchar=(lambda G:chartable(G, chars=False)['b']))
        Induc_W2cy_W2=pd.DataFrame(induc_W2cy_W2['scalar'],columns=induc_W2cy_W2['charH'],index=induc_W2cy_W2['charW'])
        Induc_W2cy_W2_set.append(Induc_W2cy_W2)
    for char in E8_Data.loc[:,'charnames']:
        j_induction_sign_set=[]
        for i in range(len(minimal_node_set_W1)):
            Phi1_cy=minimal_node_set_W1[i]
            Phi2_cy=minimal_node_set_W2[i]
            W1_cy=reflectionsubgroup(W1,Phi1_cy)
            W2_cy=reflectionsubgroup(W2,Phi2_cy)
            W1_cy_chartable=pd.DataFrame(chartable(W1_cy))
            W1_cy_special_rep=W1_cy_special_rep_set[i]
            Induc_W1cy_W1=Induc_W1cy_W1_set[i]
            Induc_W2cy_W2=Induc_W2cy_W2_set[i]
            for rep_now in W1_cy_special_rep:
                j_induction_now=(Induc_W1cy_W1[rep_now][Induc_W1cy_W1[rep_now]==1].index)[0]
                if list((E8_Data.loc[E8_Data['charnames']==str(j_induction_now)]).loc[:,'orbit'])[0] in list((E8_Data.loc[E8_Data['charnames']==char]).loc[:,'Georbit'])[0]:
                    rep_sign_position=(W1_cy_chartable.loc[W1_cy_chartable['charnames']==rep_now]).loc[:,'permsgn'].iloc[0]
                    rep_sign=W1_cy_chartable.iloc[rep_sign_position,0]
                    if rep_sign not in Induc_W2cy_W2.columns:
                        rep_sign=switch_A3_D3(rep_sign,Induc_W2cy_W2.columns)
                    j_induction_sign=(Induc_W2cy_W2[rep_sign][Induc_W2cy_W2[rep_sign]==1].index)[0]
                    j_induction_sign_set.append(j_induction_sign)
        Target.append(max_orbit(list(set([Sat_dict.get(x) for x in set(j_induction_sign_set)])),E8_Data))
    E8_Data["E8_A8_"+str(cover_fold)]=Target

When $W_c$ is of the type $2A_4$:

In [20]:
# Construct W of type E8 and Wc of type 2A4
W1=coxeter("E",8)
W2=coxeter([[2,-1,0,0,0,0,0,0],[-1,2,-1,0,0,0,0,0],[0,-1,2,-1,0,0,0,0],[0,0,-1,2,0,0,0,0],[0,0,0,0,2,-1,0,0],[0,0,0,0,-1,2,-1,0],[0,0,0,0,0,-1,2,-1],[0,0,0,0,0,0,-1,2]])
# Record the saturation dictionary
Sat_dict={('[1, 1, 1, 1, 1]', '[1, 1, 1, 1, 1]'):'1',
 ('[1, 1, 1, 1, 1]', '[2, 1, 1, 1]'):'A1',
 ('[1, 1, 1, 1, 1]', '[2, 2, 1]'):'2A1',
 ('[1, 1, 1, 1, 1]', '[3, 1, 1]'):'A2',
 ('[1, 1, 1, 1, 1]', '[3, 2]'):'A2+A1',
 ('[1, 1, 1, 1, 1]', '[4, 1]'):'A3',
 ('[1, 1, 1, 1, 1]', '[5]'):'A4',
 ('[2, 1, 1, 1]', '[1, 1, 1, 1, 1]'):'A1',
 ('[2, 1, 1, 1]', '[2, 1, 1, 1]'):'2A1',
 ('[2, 1, 1, 1]', '[2, 2, 1]'):'3A1',
 ('[2, 1, 1, 1]', '[3, 1, 1]'):'A2+A1',
 ('[2, 1, 1, 1]', '[3, 2]'):'A2+2A1',
 ('[2, 1, 1, 1]', '[4, 1]'):'A3+A1',
 ('[2, 1, 1, 1]', '[5]'):'A4+A1',
 ('[2, 2, 1]', '[1, 1, 1, 1, 1]'):'2A1',
 ('[2, 2, 1]', '[2, 1, 1, 1]'):'3A1',
 ('[2, 2, 1]', '[2, 2, 1]'):'4A1',
 ('[2, 2, 1]', '[3, 1, 1]'):'A2+2A1',
 ('[2, 2, 1]', '[3, 2]'):'A2+3A1',
 ('[2, 2, 1]', '[4, 1]'):'A3+2A1',
 ('[2, 2, 1]', '[5]'):'A4+2A1',
 ('[3, 1, 1]', '[1, 1, 1, 1, 1]'):'A2',
 ('[3, 1, 1]', '[2, 1, 1, 1]'):'A2+A1',
 ('[3, 1, 1]', '[2, 2, 1]'):'A2+2A1',
 ('[3, 1, 1]', '[3, 1, 1]'):'2A2',
 ('[3, 1, 1]', '[3, 2]'):'2A2+A1',
 ('[3, 1, 1]', '[4, 1]'):'A3+A2',
 ('[3, 1, 1]', '[5]'):'A4+A2',
 ('[3, 2]', '[1, 1, 1, 1, 1]'):'A2+A1',
 ('[3, 2]', '[2, 1, 1, 1]'):'A2+2A1',
 ('[3, 2]', '[2, 2, 1]'):'A2+3A1',
 ('[3, 2]', '[3, 1, 1]'):'2A2+A1',
 ('[3, 2]', '[3, 2]'):'2A2+2A1',
 ('[3, 2]', '[4, 1]'):'A3+A2+A1',
 ('[3, 2]', '[5]'):'A4+A2+A1',
 ('[4, 1]', '[1, 1, 1, 1, 1]'):'A3',
 ('[4, 1]', '[2, 1, 1, 1]'):'A3+A1',
 ('[4, 1]', '[2, 2, 1]'):'A3+2A1',
 ('[4, 1]', '[3, 1, 1]'):'A3+A2',
 ('[4, 1]', '[3, 2]'):'A3+A2+A1',
 ('[4, 1]', '[4, 1]'):'2A3',
 ('[4, 1]', '[5]'):'A4+A3',
 ('[5]', '[1, 1, 1, 1, 1]'):'A4',
 ('[5]', '[2, 1, 1, 1]'):'A4+A1',
 ('[5]', '[2, 2, 1]'):'A4+2A1',
 ('[5]', '[3, 1, 1]'):'A4+A2',
 ('[5]', '[3, 2]'):'A4+A2+A1',
 ('[5]', '[4, 1]'):'A4+A3',
 ('[5]', '[5]'):'E8(a7)'}

weight=[2,4,6,3,4,3,2]
drop_weight=5
node_set_1=[list(p) for p in itertools.product(range(drop_weight+1), repeat=len(weight))]
node_set_2=[]
for node in node_set_1:
    c=(-np.sum(np.array(node)*weight)-1)%drop_weight
    node_set_2.append(np.append(node,c))
    if c==0:
        node_set_2.append(np.append(node,drop_weight))

for cover_fold in range(1,7):
    node_set_3=[]
    for node in node_set_2:
        if node[0]+node[1]+node[2]+node[3]<=cover_fold and node[4]+node[5]+node[6]+node[7]<=cover_fold:
            add_point=[]
            for i in range(8):
                if node[i]==0:
                    add_point.append(i)
            if node[0]+node[1]+node[2]+node[3]==cover_fold:
                add_point.append(38)
            if node[4]+node[5]+node[6]+node[7]==cover_fold:
                add_point.append(39)
            node_set_3.append(add_point)

    
    node_set_3=[list(tpl) for tpl in {tuple(item) for item in node_set_3}]
    minimal_node_set_W2=minimal_W_cy(node_set_3)
    
    dict={0:0,1:2,2:3,3:1,4:5,5:6,6:7,7:239}
    minimal_node_set_W1=[]
    for node in minimal_node_set_W2:
        minimal_node_set_W1.append([convert_fuc(W1.roots,W2.roots,Index,dict) for Index in node])

    Target=[]
    Induc_W1cy_W1_set=[]
    Induc_W2cy_W2_set=[]
    W1_cy_special_rep_set=[]
    for i in range(len(minimal_node_set_W1)):
        Phi1_cy=minimal_node_set_W1[i]
        Phi2_cy=minimal_node_set_W2[i]
        W1_cy=reflectionsubgroup(W1,Phi1_cy)
        W2_cy=reflectionsubgroup(W2,Phi2_cy)
        W1_cy_chartable=pd.DataFrame(chartable(W1_cy))
        W1_cy_special_rep=W1_cy_chartable.loc[W1_cy_chartable['b']==W1_cy_chartable['a']]['charnames']
        W1_cy_special_rep_set.append(W1_cy_special_rep)
        induc_W1cy_W1=inductiontable(W1_cy,W1,invchar=(lambda G:chartable(G, chars=False)['b']))
        Induc_W1cy_W1=pd.DataFrame(induc_W1cy_W1['scalar'],columns=induc_W1cy_W1['charH'],index=induc_W1cy_W1['charW'])
        Induc_W1cy_W1_set.append(Induc_W1cy_W1)
        induc_W2cy_W2=inductiontable(W2_cy,W2,invchar=(lambda G:chartable(G, chars=False)['b']))
        Induc_W2cy_W2=pd.DataFrame(induc_W2cy_W2['scalar'],columns=induc_W2cy_W2['charH'],index=induc_W2cy_W2['charW'])
        Induc_W2cy_W2_set.append(Induc_W2cy_W2)
    for char in E8_Data.loc[:,'charnames']:
        j_induction_sign_set=[]
        for i in range(len(minimal_node_set_W1)):
            Phi1_cy=minimal_node_set_W1[i]
            Phi2_cy=minimal_node_set_W2[i]
            W1_cy=reflectionsubgroup(W1,Phi1_cy)
            W2_cy=reflectionsubgroup(W2,Phi2_cy)
            W1_cy_chartable=pd.DataFrame(chartable(W1_cy))
            W1_cy_special_rep=W1_cy_special_rep_set[i]
            Induc_W1cy_W1=Induc_W1cy_W1_set[i]
            Induc_W2cy_W2=Induc_W2cy_W2_set[i]
            for rep_now in W1_cy_special_rep:
                j_induction_now=(Induc_W1cy_W1[rep_now][Induc_W1cy_W1[rep_now]==1].index)[0]
                if list((E8_Data.loc[E8_Data['charnames']==str(j_induction_now)]).loc[:,'orbit'])[0] in list((E8_Data.loc[E8_Data['charnames']==char]).loc[:,'Georbit'])[0]:
                    rep_sign_position=(W1_cy_chartable.loc[W1_cy_chartable['charnames']==rep_now]).loc[:,'permsgn'].iloc[0]
                    rep_sign=W1_cy_chartable.iloc[rep_sign_position,0]
                    if rep_sign not in Induc_W2cy_W2.columns:
                        rep_sign=switch_A3_D3(rep_sign,Induc_W2cy_W2.columns)
                    j_induction_sign=(Induc_W2cy_W2[rep_sign][Induc_W2cy_W2[rep_sign]==1].index)[0]
                    j_induction_sign_set.append(j_induction_sign)
        Target.append(max_orbit(list(set([Sat_dict.get(x) for x in set(j_induction_sign_set)])),E8_Data))
    E8_Data["E8_2A4_"+str(cover_fold)]=Target

When $W_c$ is of the type $D_5+A_3$

In [21]:
# Construct W of type E8 and Wc of type D5+A3
W1=coxeter("E",8)
W2=coxeter([[2, 0, -1, 0, 0,0,0,0],[0, 2, -1, 0, 0,0,0,0],[-1, -1, 2, -1, 0,0,0,0],[0, 0, -1, 2, -1,0,0,0],[0, 0, 0, -1, 2,0,0,0],[0,0,0,0,0,2,-1,0],[0,0,0,0,0,-1,2,-1],[0,0,0,0,0,0,-1,2]])
# Record the saturation dictionary
Sat_dict={('[[1, 1], [1, 1, 1]]', '[1, 1, 1, 1]'):'2A1',
 ('[[1, 1], [1, 1, 1]]', '[2, 1, 1]'):'3A1',
 ('[[1, 1], [1, 1, 1]]', '[2, 2]'):'4A1',
 ('[[1, 1], [1, 1, 1]]', '[3, 1]'):'A2+2A1',
 ('[[1, 1], [1, 1, 1]]', '[4]'):'A3+2A1',
 ('[[1], [1, 1, 1, 1]]', '[1, 1, 1, 1]'):'A1',
 ('[[1], [1, 1, 1, 1]]', '[2, 1, 1]'):'2A1',
 ('[[1], [1, 1, 1, 1]]', '[2, 2]'):'3A1',
 ('[[1], [1, 1, 1, 1]]', '[3, 1]'):'A2+A1',
 ('[[1], [1, 1, 1, 1]]', '[4]'):'A3+A1',
 ('[[], [1, 1, 1, 1, 1]]', '[1, 1, 1, 1]'):'1',
 ('[[], [1, 1, 1, 1, 1]]', '[2, 1, 1]'):'A1',
 ('[[], [1, 1, 1, 1, 1]]', '[2, 2]'):'2A1',
 ('[[], [1, 1, 1, 1, 1]]', '[3, 1]'):'A2',
 ('[[], [1, 1, 1, 1, 1]]', '[4]'):'A3',
 ('[[1, 1], [2, 1]]', '[1, 1, 1, 1]'):'A2+A1',
 ('[[1, 1], [2, 1]]', '[2, 1, 1]'):'A2+2A1',
 ('[[1, 1], [2, 1]]', '[2, 2]'):'A2+3A1',
 ('[[1, 1], [2, 1]]', '[3, 1]'):'2A2',
 ('[[1, 1], [2, 1]]', '[4]'):'A3+A2+A1',
 ('[[1], [2, 1, 1]]', '[1, 1, 1, 1]'):'A2',
 ('[[1], [2, 1, 1]]', '[2, 1, 1]'):'A2+A1',
 ('[[1], [2, 1, 1]]', '[2, 2]'):'A2+2A1',
 ('[[1], [2, 1, 1]]', '[3, 1]'):'2A2',
 ('[[1], [2, 1, 1]]', '[4]'):'A3+A2',
 ('[[], [2, 1, 1, 1]]', '[1, 1, 1, 1]'):'2A1',
 ('[[], [2, 1, 1, 1]]', '[2, 1, 1]'):'3A1',
 ('[[], [2, 1, 1, 1]]', '[2, 2]'):'A2',
 ('[[], [2, 1, 1, 1]]', '[3, 1]'):'A2+2A1',
 ('[[], [2, 1, 1, 1]]', '[4]'):'D4(a1)',
 ('[[1], [2, 2]]', '[1, 1, 1, 1]'):'A2+2A1',
 ('[[1], [2, 2]]', '[2, 1, 1]'):'A2+3A1',
 ('[[1], [2, 2]]', '[2, 2]'):'2A2',
 ('[[1], [2, 2]]', '[3, 1]'):'2A2+2A1',
 ('[[1], [2, 2]]', '[4]'):'D4(a1)+A2',
 ('[[2], [2, 1]]', '[1, 1, 1, 1]'):'A3',
 ('[[2], [2, 1]]', '[2, 1, 1]'):'A3+A1',
 ('[[2], [2, 1]]', '[2, 2]'):'A3+2A1',
 ('[[2], [2, 1]]', '[3, 1]'):'A3+A2',
 ('[[2], [2, 1]]', '[4]'):'2A3',
 ('[[], [2, 2, 1]]', '[1, 1, 1, 1]'):'3A1',
 ('[[], [2, 2, 1]]', '[2, 1, 1]'):'4A1',
 ('[[], [2, 2, 1]]', '[2, 2]'):'A2+A1',
 ('[[], [2, 2, 1]]', '[3, 1]'):'A2+3A1',
 ('[[], [2, 2, 1]]', '[4]'):'D4(a1)+A1',
 ('[[1], [3, 1]]', '[1, 1, 1, 1]'):'D4(a1)',
 ('[[1], [3, 1]]', '[2, 1, 1]'):'D4(a1)+A1',
 ('[[1], [3, 1]]', '[2, 2]'):'A3+A2',
 ('[[1], [3, 1]]', '[3, 1]'):'D4(a1)+A2',
 ('[[1], [3, 1]]', '[4]'):'A4+2A1',
 ('[[], [3, 1, 1]]', '[1, 1, 1, 1]'):'A3',
 ('[[], [3, 1, 1]]', '[2, 1, 1]'):'A3+A1',
 ('[[], [3, 1, 1]]', '[2, 2]'):'D4(a1)',
 ('[[], [3, 1, 1]]', '[3, 1]'):'A3+A2',
 ('[[], [3, 1, 1]]', '[4]'):'A4',
 ('[[2], [3]]', '[1, 1, 1, 1]'):'A4',
 ('[[2], [3]]', '[2, 1, 1]'):'A4+A1',
 ('[[2], [3]]', '[2, 2]'):'A4+2A1',
 ('[[2], [3]]', '[3, 1]'):'A4+A2',
 ('[[2], [3]]', '[4]'):'A4+A3',
 ('[[], [3, 2]]', '[1, 1, 1, 1]'):'A3+A1',
 ('[[], [3, 2]]', '[2, 1, 1]'):'A3+2A1',
 ('[[], [3, 2]]', '[2, 2]'):'D4(a1)+A1',
 ('[[], [3, 2]]', '[3, 1]'):'A3+A2',
 ('[[], [3, 2]]', '[4]'):'A4+A1',
 ('[[1], [4]]', '[1, 1, 1, 1]'):'D5(a1)',
 ('[[1], [4]]', '[2, 1, 1]'):'D5(a1)+A1',
 ('[[1], [4]]', '[2, 2]'):'D4+A2',
 ('[[1], [4]]', '[3, 1]'):'D5(a1)+A2',
 ('[[1], [4]]', '[4]'):'E8(a7)',
 ('[[], [4, 1]]', '[1, 1, 1, 1]'):'D4',
 ('[[], [4, 1]]', '[2, 1, 1]'):'D4+A1',
 ('[[], [4, 1]]', '[2, 2]'):'D5(a1)',
 ('[[], [4, 1]]', '[3, 1]'):'D4+A2',
 ('[[], [4, 1]]', '[4]'):'D6(a2)',
 ('[[], [5]]', '[1, 1, 1, 1]'):'D5',
 ('[[], [5]]', '[2, 1, 1]'):'D5+A1',
 ('[[], [5]]', '[2, 2]'):'D6(a1)',
 ('[[], [5]]', '[3, 1]'):'D5+A2',
 ('[[], [5]]', '[4]'):'D7(a2)'}

weight=[3,5,6,4,2,3,2]
drop_weight=4
node_set_1=[list(p) for p in itertools.product(range(drop_weight+1), repeat=len(weight))]
node_set_2=[]
for node in node_set_1:
    c=(-np.sum(np.array(node)*weight)-1)%drop_weight
    node_set_2.append(np.append(node,c))
    if c==0:
        node_set_2.append(np.append(node,drop_weight))

for cover_fold in range(1,9):
    node_set_3=[]
    for node in node_set_2:
        if node[0]+node[1]+2*node[2]+2*node[3]+node[4]<=cover_fold and node[5]+node[6]+node[7]<=cover_fold:
            add_point=[]
            for i in range(8):
                if node[i]==0:
                    add_point.append(i)
            if node[0]+node[1]+2*node[2]+2*node[3]+node[4]==cover_fold:
                add_point.append(25)
            if node[5]+node[6]+node[7]==cover_fold:
                add_point.append(18)
            node_set_3.append(add_point)

    
    node_set_3=[list(tpl) for tpl in {tuple(item) for item in node_set_3}]
    minimal_node_set_W2=minimal_W_cy(node_set_3)
    
    dict={0:1,1:4,2:3,3:2,4:0,5:6,6:7,7:239}
    minimal_node_set_W1=[]
    for node in minimal_node_set_W2:
        minimal_node_set_W1.append([convert_fuc(W1.roots,W2.roots,Index,dict) for Index in node])

    Target=[]
    Induc_W1cy_W1_set=[]
    Induc_W2cy_W2_set=[]
    W1_cy_special_rep_set=[]
    for i in range(len(minimal_node_set_W1)):
        Phi1_cy=minimal_node_set_W1[i]
        Phi2_cy=minimal_node_set_W2[i]
        W1_cy=reflectionsubgroup(W1,Phi1_cy)
        W2_cy=reflectionsubgroup(W2,Phi2_cy)
        W1_cy_chartable=pd.DataFrame(chartable(W1_cy))
        W1_cy_special_rep=W1_cy_chartable.loc[W1_cy_chartable['b']==W1_cy_chartable['a']]['charnames']
        W1_cy_special_rep_set.append(W1_cy_special_rep)
        induc_W1cy_W1=inductiontable(W1_cy,W1,invchar=(lambda G:chartable(G, chars=False)['b']))
        Induc_W1cy_W1=pd.DataFrame(induc_W1cy_W1['scalar'],columns=induc_W1cy_W1['charH'],index=induc_W1cy_W1['charW'])
        Induc_W1cy_W1_set.append(Induc_W1cy_W1)
        induc_W2cy_W2=inductiontable(W2_cy,W2,invchar=(lambda G:chartable(G, chars=False)['b']))
        Induc_W2cy_W2=pd.DataFrame(induc_W2cy_W2['scalar'],columns=induc_W2cy_W2['charH'],index=induc_W2cy_W2['charW'])
        Induc_W2cy_W2_set.append(Induc_W2cy_W2)
    for char in E8_Data.loc[:,'charnames']:
        j_induction_sign_set=[]
        for i in range(len(minimal_node_set_W1)):
            Phi1_cy=minimal_node_set_W1[i]
            Phi2_cy=minimal_node_set_W2[i]
            W1_cy=reflectionsubgroup(W1,Phi1_cy)
            W2_cy=reflectionsubgroup(W2,Phi2_cy)
            W1_cy_chartable=pd.DataFrame(chartable(W1_cy))
            W1_cy_special_rep=W1_cy_special_rep_set[i]
            Induc_W1cy_W1=Induc_W1cy_W1_set[i]
            Induc_W2cy_W2=Induc_W2cy_W2_set[i]
            for rep_now in W1_cy_special_rep:
                j_induction_now=(Induc_W1cy_W1[rep_now][Induc_W1cy_W1[rep_now]==1].index)[0]
                if list((E8_Data.loc[E8_Data['charnames']==str(j_induction_now)]).loc[:,'orbit'])[0] in list((E8_Data.loc[E8_Data['charnames']==char]).loc[:,'Georbit'])[0]:
                    rep_sign_position=(W1_cy_chartable.loc[W1_cy_chartable['charnames']==rep_now]).loc[:,'permsgn'].iloc[0]
                    rep_sign=W1_cy_chartable.iloc[rep_sign_position,0]
                    if rep_sign not in Induc_W2cy_W2.columns:
                        rep_sign=switch_A3_D3(rep_sign,Induc_W2cy_W2.columns)
                    j_induction_sign=(Induc_W2cy_W2[rep_sign][Induc_W2cy_W2[rep_sign]==1].index)[0]
                    j_induction_sign_set.append(j_induction_sign)
        Target.append(max_orbit(list(set([Sat_dict.get(x) for x in set(j_induction_sign_set)])),E8_Data))
    E8_Data["E8_D5+A3_"+str(cover_fold)]=Target

#I using also traces ... okay now
#I using also traces ... okay now
#I using also traces ... okay now
#I using also traces ... okay now


When $W_c$ is of the type $E_6+A_2$

In [22]:
# Construct W of type E8 and Wc of type E6+A2
W1=coxeter("E",8)
W2=coxeter([[2, 0, -1, 0, 0, 0, 0, 0],[0, 2, 0, -1, 0, 0, 0, 0],[-1, 0, 2, -1, 0, 0, 0, 0],[0, -1, -1, 2, -1, 0, 0, 0],[0, 0, 0, -1, 2, -1, 0, 0],[0, 0, 0, 0, -1, 2, 0, 0],[0,0,0,0,0,0,2,-1],[0,0,0,0,0,0,-1,2]])
Sat_dict={('1_p', '[1, 1, 1]'):'E6',
 ('1_p', '[2, 1]'):'E6+A1',
 ('1_p', '[3]'):'E8(b5)',
 ("1_p'", '[1, 1, 1]'):'1',
 ("1_p'", '[2, 1]'):'A1',
 ("1_p'", '[3]'):'A2',
 ('10_s', '[1, 1, 1]'):'2A2+A1',
 ('10_s', '[2, 1]'):'2A2+2A1',
 ('10_s', '[3]'):'D4(a1)+A1',
 ('6_p', '[1, 1, 1]'):'E6(a1)',
 ('6_p', '[2, 1]'):'E6(a1)+A1',
 ('6_p', '[3]'):'E8(b6)',
 ("6_p'", '[1, 1, 1]'):'A1',
 ("6_p'", '[2, 1]'):'2A1',
 ("6_p'", '[3]'):'A2+A1',
 ('15_q', '[1, 1, 1]'):'A5',
 ('15_q', '[2, 1]'):'A5+A1',
 ('15_q', '[3]'):'E7(a5)',
 ("15_q'", '[1, 1, 1]'):'3A1',
 ("15_q'", '[2, 1]'):'4A1',
 ("15_q'", '[3]'):'A2+3A1',
 ('20_p', '[1, 1, 1]'):'D5',
 ('20_p', '[2, 1]'):'D5+A1',
 ('20_p', '[3]'):'D5+A2',
 ("20_p'", '[1, 1, 1]'):'2A1',
 ("20_p'", '[2, 1]'):'3A1',
 ("20_p'", '[3]'):'A2+2A1',
 ('24_p', '[1, 1, 1]'):'D4',
 ('24_p', '[2, 1]'):'D4+A1',
 ('24_p', '[3]'):'D4+A2',
 ("24_p'", '[1, 1, 1]'):'2A2',
 ("24_p'", '[2, 1]'):'2A2+A1',
 ("24_p'", '[3]'):'D4(a1)',
 ('30_p', '[1, 1, 1]'):'E6(a3)',
 ('30_p', '[2, 1]'):'E6(a3)+A1',
 ('30_p', '[3]'):'E8(a7)',
 ("30_p'", '[1, 1, 1]'):'A2',
 ("30_p'", '[2, 1]'):'A2+A1',
 ("30_p'", '[3]'):'2A2',
 ('60_s', '[1, 1, 1]'):'A3+A1',
 ('60_s', '[2, 1]'):'A3+2A1',
 ('60_s', '[3]'):'A3+A2+A1',
 ('80_s', '[1, 1, 1]'):'D4(a1)',
 ('80_s', '[2, 1]'):'D4(a1)+A1',
 ('80_s', '[3]'):'D4(a1)+A2',
 ('60_p', '[1, 1, 1]'):'A4+A1',
 ('60_p', '[2, 1]'):'A4+2A1',
 ('60_p', '[3]'):'A4+A2+A1',
 ("60_p'", '[1, 1, 1]'):'A2+2A1',
 ("60_p'", '[2, 1]'):'A2+3A1',
 ("60_p'", '[3]'):'2A2+2A1',
 ('64_p', '[1, 1, 1]'):'D5(a1)',
 ('64_p', '[2, 1]'):'D5(a1)+A1',
 ('64_p', '[3]'):'D5(a1)+A2',
 ("64_p'", '[1, 1, 1]'):'A2+A1',
 ("64_p'", '[2, 1]'):'A2+2A1',
 ("64_p'", '[3]'):'2A2+A1',
 ('81_p', '[1, 1, 1]'):'A4',
 ('81_p', '[2, 1]'):'A4+A1',
 ('81_p', '[3]'):'A4+A2',
 ("81_p'", '[1, 1, 1]'):'A3',
 ("81_p'", '[2, 1]'):'A3+A1',
 ("81_p'", '[3]'):'A3+A2'}

weight=[2,3,4,6,5,4,2]
drop_weight=3
node_set_1=[list(p) for p in itertools.product(range(drop_weight+1), repeat=len(weight))]
node_set_2=[]
for node in node_set_1:
    c=(-np.sum(np.array(node)*weight)-1)%drop_weight
    node_set_2.append(np.append(node,c))
    if c==0:
        node_set_2.append(np.append(node,drop_weight))

for cover_fold in range(1,13):
    node_set_3=[]
    for node in node_set_2:
        if node[0]+2*node[1]+2*node[2]+3*node[3]+2*node[4]+node[5]<=cover_fold and node[6]+node[7]<=cover_fold:
            add_point=[]
            for i in range(8):
                if node[i]==0:
                    add_point.append(i)
            if node[0]+2*node[1]+2*node[2]+3*node[3]+2*node[4]+node[5]==cover_fold:
                add_point.append(38)
            if node[6]+node[7]==cover_fold:
                add_point.append(13)
            node_set_3.append(add_point)
    
    node_set_3=[list(tpl) for tpl in {tuple(item) for item in node_set_3}]
    minimal_node_set_W2=minimal_W_cy(node_set_3)
    
    dict={0:0,1:1,2:2,3:3,4:4,5:5,6:7,7:239}
    minimal_node_set_W1=[]
    for node in minimal_node_set_W2:
        minimal_node_set_W1.append([convert_fuc(W1.roots,W2.roots,Index,dict) for Index in node])

    Target=[]
    Induc_W1cy_W1_set=[]
    Induc_W2cy_W2_set=[]
    W1_cy_special_rep_set=[]
    for i in range(len(minimal_node_set_W1)):
        Phi1_cy=minimal_node_set_W1[i]
        Phi2_cy=minimal_node_set_W2[i]
        W1_cy=reflectionsubgroup(W1,Phi1_cy)
        W2_cy=reflectionsubgroup(W2,Phi2_cy)
        W1_cy_chartable=pd.DataFrame(chartable(W1_cy))
        W1_cy_special_rep=W1_cy_chartable.loc[W1_cy_chartable['b']==W1_cy_chartable['a']]['charnames']
        W1_cy_special_rep_set.append(W1_cy_special_rep)
        induc_W1cy_W1=inductiontable(W1_cy,W1,invchar=(lambda G:chartable(G, chars=False)['b']))
        Induc_W1cy_W1=pd.DataFrame(induc_W1cy_W1['scalar'],columns=induc_W1cy_W1['charH'],index=induc_W1cy_W1['charW'])
        Induc_W1cy_W1_set.append(Induc_W1cy_W1)
        induc_W2cy_W2=inductiontable(W2_cy,W2,invchar=(lambda G:chartable(G, chars=False)['b']))
        Induc_W2cy_W2=pd.DataFrame(induc_W2cy_W2['scalar'],columns=induc_W2cy_W2['charH'],index=induc_W2cy_W2['charW'])
        Induc_W2cy_W2_set.append(Induc_W2cy_W2)
    for char in E8_Data.loc[:,'charnames']:
        j_induction_sign_set=[]
        for i in range(len(minimal_node_set_W1)):
            Phi1_cy=minimal_node_set_W1[i]
            Phi2_cy=minimal_node_set_W2[i]
            W1_cy=reflectionsubgroup(W1,Phi1_cy)
            W2_cy=reflectionsubgroup(W2,Phi2_cy)
            W1_cy_chartable=pd.DataFrame(chartable(W1_cy))
            W1_cy_special_rep=W1_cy_special_rep_set[i]
            Induc_W1cy_W1=Induc_W1cy_W1_set[i]
            Induc_W2cy_W2=Induc_W2cy_W2_set[i]
            for rep_now in W1_cy_special_rep:
                j_induction_now=(Induc_W1cy_W1[rep_now][Induc_W1cy_W1[rep_now]==1].index)[0]
                if list((E8_Data.loc[E8_Data['charnames']==str(j_induction_now)]).loc[:,'orbit'])[0] in list((E8_Data.loc[E8_Data['charnames']==char]).loc[:,'Georbit'])[0]:
                    rep_sign_position=(W1_cy_chartable.loc[W1_cy_chartable['charnames']==rep_now]).loc[:,'permsgn'].iloc[0]
                    rep_sign=W1_cy_chartable.iloc[rep_sign_position,0]
                    if rep_sign not in Induc_W2cy_W2.columns:
                        rep_sign=switch_A3_D3(rep_sign,Induc_W2cy_W2.columns)
                    j_induction_sign=(Induc_W2cy_W2[rep_sign][Induc_W2cy_W2[rep_sign]==1].index)[0]
                    j_induction_sign_set.append(j_induction_sign)
        Target.append(max_orbit(list(set([Sat_dict.get(x) for x in set(j_induction_sign_set)])),E8_Data))
    E8_Data["E8_E6+A2_"+str(cover_fold)]=Target

#I using also traces ... okay now
#I using also traces ... okay now
#I using also traces ... okay now
#I using also traces ... okay now
#I using also traces ... okay now
#I using also traces ... okay now


When $W_c$ is of the type $E_7+A_1$

In [23]:
# Construct W of type E8 and Wc of type E7+A1
W1=coxeter("E",8)
W2=coxeter([[2, 0, -1, 0, 0, 0, 0,0],[0, 2, 0, -1, 0, 0, 0,0],[-1, 0, 2, -1, 0, 0, 0,0],[0, -1, -1, 2, -1, 0, 0,0],[0, 0, 0, -1, 2, -1, 0,0],[0, 0, 0, 0, -1, 2, -1,0],[0, 0, 0, 0, 0, -1, 2,0],[0,0,0,0,0,0,0,2]])
weight=[2,3,4,6,5,4,3]
Sat_dict={('1_a', '[1, 1]'):'E7',
 ('1_a', '[2]'):'E8(a3)',
 ("1_a'", '[1, 1]'):'1',
 ("1_a'", '[2]'):'A1',
 ('7_a', '[1, 1]'):'A1',
 ('7_a', '[2]'):'2A1',
 ("7_a'", '[1, 1]'):'E7(a1)',
 ("7_a'", '[2]'):'E8(b4)',
 ('15_a', '[1, 1]'):'4A1',
 ('15_a', '[2]'):'A2+A1',
 ('21_b', '[1, 1]'):'3A1',
 ('21_b', '[2]'):'A2',
 ("21_b'", '[1, 1]'):'E6',
 ("21_b'", '[2]'):'E6+A1',
 ('27_a', '[1, 1]'):'E7(a2)',
 ('27_a', '[2]'):'E8(b5)',
 ("27_a'", '[1, 1]'):'2A1',
 ("27_a'", '[2]'):'3A1',
 ('35_b', '[1, 1]'):'D6',
 ('35_b', '[2]'):'E7(a3)',
 ("35_b'", '[1, 1]'):'3A1',
 ("35_b'", '[2]'):'4A1',
 ('56_a', '[1, 1]'):'A2',
 ('56_a', '[2]'):'A2+A1',
 ("56_a'", '[1, 1]'):'E7(a3)',
 ("56_a'", '[2]'):'D7(a1)',
 ('70_a', '[1, 1]'):'2A2+A1',
 ('70_a', '[2]'):'2A2+2A1',
 ("70_a'", '[1, 1]'):'A5+A1',
 ("70_a'", '[2]'):'E6(a3)+A1',
 ('84_a', '[1, 1]'):'D4+A1',
 ('84_a', '[2]'):'D5(a1)',
 ('105_b', '[1, 1]'):'A6',
 ('105_b', '[2]'):'A6+A1',
 ("105_b'", '[1, 1]'):'A2+3A1',
 ("105_b'", '[2]'):'2A2',
 ('105_c', '[1, 1]'):'A5',
 ('105_c', '[2]'):'E6(a3)',
 ("105_c'", '[1, 1]'):'D4',
 ("105_c'", '[2]'):'D4+A1',
 ('120_a', '[1, 1]'):'E6(a1)',
 ('120_a', '[2]'):'E6(a1)+A1',
 ("120_a'", '[1, 1]'):'A2+A1',
 ("120_a'", '[2]'):'A2+2A1',
 ('168_a', '[1, 1]'):'D5+A1',
 ('168_a', '[2]'):'D6(a1)',
 ("168_a'", '[1, 1]'):'2A2',
 ("168_a'", '[2]'):'2A2+A1',
 ('189_b', '[1, 1]'):'A2+2A1',
 ('189_b', '[2]'):'A2+3A1',
 ("189_b'", '[1, 1]'):'E7(a4)',
 ("189_b'", '[2]'):'D5+A2',
 ('189_c', '[1, 1]'):'A3+A1',
 ('189_c', '[2]'):'D4(a1)',
 ("189_c'", '[1, 1]'):'D5',
 ("189_c'", '[2]'):'D5+A1',
 ('210_a', '[1, 1]'):'D6(a1)',
 ('210_a', '[2]'):'E7(a4)',
 ("210_a'", '[1, 1]'):'A3',
 ("210_a'", '[2]'):'A3+A1',
 ('210_b', '[1, 1]'):'A4+A2',
 ('210_b', '[2]'):'A4+A2+A1',
 ("210_b'", '[1, 1]'):'A3+A2+A1',
 ("210_b'", '[2]'):'D4(a1)+A2',
 ('216_a', '[1, 1]'):'A3+2A1',
 ('216_a', '[2]'):'D4(a1)+A1',
 ("216_a'", '[1, 1]'):'A5',
 ("216_a'", '[2]'):'A5+A1',
 ('280_b', '[1, 1]'):'D6(a2)',
 ('280_b', '[2]'):'E7(a5)',
 ("280_b'", '[1, 1]'):'A3+A1',
 ("280_b'", '[2]'):'A3+2A1',
 ('315_a', '[1, 1]'):'D4(a1)',
 ('315_a', '[2]'):'D4(a1)+A1',
 ("315_a'", '[1, 1]'):'E7(a5)',
 ("315_a'", '[2]'):'E8(a7)',
 ('378_a', '[1, 1]'):'A3+A2',
 ('378_a', '[2]'):'A3+A2+A1',
 ("378_a'", '[1, 1]'):'D5(a1)+A1',
 ("378_a'", '[2]'):'D4+A2',
 ('405_a', '[1, 1]'):'E6(a3)',
 ('405_a', '[2]'):'E6(a3)+A1',
 ("405_a'", '[1, 1]'):'D4(a1)+A1',
 ("405_a'", '[2]'):'A3+A2',
 ('420_a', '[1, 1]'):'D5(a1)',
 ('420_a', '[2]'):'D5(a1)+A1',
 ("420_a'", '[1, 1]'):'A4',
 ("420_a'", '[2]'):'A4+A1',
 ("512_a'", '[1, 1]'):'A4+A1',
 ("512_a'", '[2]'):'A4+2A1'}

weight=[2,3,4,6,5,4,3]
drop_weight=2
node_set_1=[list(p) for p in itertools.product(range(drop_weight+1), repeat=len(weight))]
node_set_2=[]
for node in node_set_1:
    c=(-np.sum(np.array(node)*weight)-1)%drop_weight
    node_set_2.append(np.append(node,c))
    if c==0:
        node_set_2.append(np.append(node,drop_weight))

for cover_fold in range(1,19):
    node_set_3=[]
    for node in node_set_2:
        if 2*node[0]+2*node[1]+3*node[2]+4*node[3]+3*node[4]+2*node[5]+node[6]<=cover_fold and node[7]<=cover_fold:
            add_point=[]
            for i in range(8):
                if node[i]==0:
                    add_point.append(i)
            if 2*node[0]+2*node[1]+3*node[2]+4*node[3]+3*node[4]+2*node[5]+node[6]==cover_fold:
                add_point.append(63)
            if node[7]==cover_fold:
                add_point.append(7)
            node_set_3.append(add_point)
    
    node_set_3=[list(tpl) for tpl in {tuple(item) for item in node_set_3}]
    minimal_node_set_W2=minimal_W_cy(node_set_3)
    
    dict={0:0,1:1,2:2,3:3,4:4,5:5,6:6,7:239}
    minimal_node_set_W1=[]
    for node in minimal_node_set_W2:
        minimal_node_set_W1.append([convert_fuc(W1.roots,W2.roots,Index,dict) for Index in node])

    Target=[]
    Induc_W1cy_W1_set=[]
    Induc_W2cy_W2_set=[]
    W1_cy_special_rep_set=[]
    for i in range(len(minimal_node_set_W1)):
        Phi1_cy=minimal_node_set_W1[i]
        Phi2_cy=minimal_node_set_W2[i]
        W1_cy=reflectionsubgroup(W1,Phi1_cy)
        W2_cy=reflectionsubgroup(W2,Phi2_cy)
        W1_cy_chartable=pd.DataFrame(chartable(W1_cy))
        W1_cy_special_rep=W1_cy_chartable.loc[W1_cy_chartable['b']==W1_cy_chartable['a']]['charnames']
        W1_cy_special_rep_set.append(W1_cy_special_rep)
        induc_W1cy_W1=inductiontable(W1_cy,W1,invchar=(lambda G:chartable(G, chars=False)['b']))
        Induc_W1cy_W1=pd.DataFrame(induc_W1cy_W1['scalar'],columns=induc_W1cy_W1['charH'],index=induc_W1cy_W1['charW'])
        Induc_W1cy_W1_set.append(Induc_W1cy_W1)
        induc_W2cy_W2=inductiontable(W2_cy,W2,invchar=(lambda G:chartable(G, chars=False)['b']))
        Induc_W2cy_W2=pd.DataFrame(induc_W2cy_W2['scalar'],columns=induc_W2cy_W2['charH'],index=induc_W2cy_W2['charW'])
        Induc_W2cy_W2_set.append(Induc_W2cy_W2)
    for char in E8_Data.loc[:,'charnames']:
        j_induction_sign_set=[]
        for i in range(len(minimal_node_set_W1)):
            Phi1_cy=minimal_node_set_W1[i]
            Phi2_cy=minimal_node_set_W2[i]
            W1_cy=reflectionsubgroup(W1,Phi1_cy)
            W2_cy=reflectionsubgroup(W2,Phi2_cy)
            W1_cy_chartable=pd.DataFrame(chartable(W1_cy))
            W1_cy_special_rep=W1_cy_special_rep_set[i]
            Induc_W1cy_W1=Induc_W1cy_W1_set[i]
            Induc_W2cy_W2=Induc_W2cy_W2_set[i]
            for rep_now in W1_cy_special_rep:
                j_induction_now=(Induc_W1cy_W1[rep_now][Induc_W1cy_W1[rep_now]==1].index)[0]
                if list((E8_Data.loc[E8_Data['charnames']==str(j_induction_now)]).loc[:,'orbit'])[0] in list((E8_Data.loc[E8_Data['charnames']==char]).loc[:,'Georbit'])[0]:
                    if rep_now==("512_a'",):
                        rep_sign=rep_now
                    elif rep_now==("512_a'",str([1,1])):
                        rep_sign=("512_a'",str([2]))
                    elif rep_now==("512_a'",str([2])):
                        rep_sign=("512_a'",str([1,1]))
                    else:
                        rep_sign_position=(W1_cy_chartable.loc[W1_cy_chartable['charnames']==rep_now]).loc[:,'permsgn'].iloc[0]
                        rep_sign=W1_cy_chartable.iloc[rep_sign_position,0]
                    if rep_sign not in Induc_W2cy_W2.columns:
                        rep_sign=switch_A3_D3(rep_sign,Induc_W2cy_W2.columns)
                    j_induction_sign=(Induc_W2cy_W2[rep_sign][Induc_W2cy_W2[rep_sign]==1].index)[0]
                    j_induction_sign_set.append(j_induction_sign)
        Target.append(max_orbit(list(set([Sat_dict.get(x) for x in set(j_induction_sign_set)])),E8_Data))
    E8_Data["E8_E7+A1_"+str(cover_fold)]=Target

#I using also traces ... okay now
#I using also traces ... okay now


C:\Windows\Temp\ipykernel_63272\1959048175.py:174: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  E8_Data["E8_E7+A1_"+str(cover_fold)]=Target
C:\Windows\Temp\ipykernel_63272\1959048175.py:174: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  E8_Data["E8_E7+A1_"+str(cover_fold)]=Target
C:\Windows\Temp\ipykernel_63272\1959048175.py:174: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(a

$F_4$

In [24]:
F4_Data=pd.DataFrame({'charnames':["('1_1',)", "('4_2',)", "('9_1',)", "('8_3',)", "('8_1',)", "('12',)", "('16',)", "('9_2',)", "('6_1',)", "('4_3',)", "('8_4',)", "('8_2',)", "('9_4',)", "('4_5',)", "('2_4',)", "('1_4',)"],
'orbit':['F4', 'F4(a1)', 'F4(a2)', 'C3', 'B3', 'F4(a3)', 'C3(a1)', 'B2', '~A2+A1', 'A2+~A1', 'A2', '~A2', 'A1+~A1', '~A1', 'A1', '1'],
'Georbit':[['F4'], ['F4(a1)', 'F4'], ['F4(a2)', 'F4', 'F4(a1)'], ['C3', 'F4', 'F4(a2)', 'F4(a1)'], ['B3', 'F4', 'F4(a2)', 'F4(a1)'], ['F4(a3)', 'F4', 'F4(a2)', 'F4(a1)', 'B3', 'C3'], ['C3(a1)', 'F4', 'F4(a3)', 'F4(a2)', 'F4(a1)', 'B3', 'C3'], ['B2', 'F4', 'F4(a3)', 'F4(a2)', 'F4(a1)', 'B3', 'C3(a1)', 'C3'], ['~A2+A1', 'F4', 'F4(a3)', 'F4(a2)', 'F4(a1)', 'B3', 'C3(a1)', 'C3'], ['A2+~A1', 'F4', 'F4(a3)', 'F4(a2)', 'F4(a1)', 'B3', 'B2', '~A2+A1', 'C3(a1)', 'C3'], ['A2', 'F4', 'F4(a3)', 'A2+~A1', 'F4(a2)', 'F4(a1)', 'B3', 'B2', '~A2+A1', 'C3(a1)', 'C3'], ['~A2', 'F4', 'F4(a3)', 'F4(a2)', 'F4(a1)', 'B3', '~A2+A1', 'C3(a1)', 'C3'], ['A1+~A1', 'F4', 'F4(a3)', '~A2', 'A2', 'A2+~A1', 'F4(a2)', 'F4(a1)', 'B3', 'B2', '~A2+A1', 'C3(a1)', 'C3'], ['~A1', 'F4', 'F4(a3)', '~A2', 'A2', 'A2+~A1', 'F4(a2)', 'F4(a1)', 'B3', 'B2', 'A1+~A1', '~A2+A1', 'C3(a1)', 'C3'], ['A1', 'F4', 'F4(a3)', '~A2', 'A2', 'A2+~A1', 'F4(a2)', 'F4(a1)', 'B3', 'B2', 'A1+~A1', '~A2+A1', 'C3(a1)', '~A1', 'C3'], ['1', 'F4', 'F4(a3)', '~A2', 'A2', 'A2+~A1', 'F4(a2)', 'F4(a1)', 'B3', 'B2', 'A1+~A1', '~A2+A1', 'C3(a1)', '~A1', 'A1', 'C3']]})

When $W_c$ is of type $F_4$ and $n$ is odd

In [25]:
# Construct W of type F4
W=coxeter("F",4)

# Give the range of cover fold
for cover_fold in range(1,14,2):
    
    # Take the minimal Phi_cy sets 
    Phi_cy_Set=[]
    for y1,y2,y3,y4 in itertools.product(range(2),range(2),range(2),range(2)):
        y=np.array([y1,y2,y3,y4])
        if np.sum(y*np.array(W.roots[23]))<=cover_fold:
            Phi_cy=[]
            if y1==0:
                Phi_cy.append(0)
            if y2==0:
                Phi_cy.append(1)
            if y3==0:
                Phi_cy.append(2)
            if y4==0:
                Phi_cy.append(3)
            if np.sum(y*np.array(W.roots[23]))==cover_fold:
                Phi_cy.append(23)
            Phi_cy_Set.append(Phi_cy)
    min_Phi_cy_Set=minimal_W_cy(Phi_cy_Set)
    
    # In the following, we compute by going through all representations
    Orbit_Set=[]
    Induc_Wcy_W_set=[]
    W_cy_special_rep_set=[]
    for Phi_cy in min_Phi_cy_Set:
        W_cy=reflectionsubgroup(W,Phi_cy)
        W_cy_chartable=pd.DataFrame(chartable(W_cy))
        W_cy_special_rep=W_cy_chartable.loc[W_cy_chartable['b']==W_cy_chartable['a']]['charnames']
        W_cy_special_rep_set.append(W_cy_special_rep)
        induc_Wcy_W=inductiontable(W_cy,W,invchar=(lambda G:chartable(G, chars=False)['b']))
        Induc_Wcy_W=pd.DataFrame(induc_Wcy_W['scalar'],columns=induc_Wcy_W['charH'],index=induc_Wcy_W['charW'])
        Induc_Wcy_W_set.append(Induc_Wcy_W)
    for char in F4_Data.loc[:,'charnames']:
        orbit_set=[]
        for i in range(len(min_Phi_cy_Set)):
            Phi_cy=min_Phi_cy_Set[i]
            W_cy=reflectionsubgroup(W,Phi_cy)
            W_cy_chartable=pd.DataFrame(chartable(W_cy))
            W_cy_special_rep=W_cy_special_rep_set[i]
            Induc_Wcy_W=Induc_Wcy_W_set[i]
            for rep_now in W_cy_special_rep:
                j_induction_now=dual_rep_F4((Induc_Wcy_W[rep_now][Induc_Wcy_W[rep_now]==1].index)[0])
                if list((F4_Data.loc[F4_Data['charnames']==str(j_induction_now)]).loc[:,'orbit'])[0] in list((F4_Data.loc[F4_Data['charnames']==char]).loc[:,'Georbit'])[0]:
                    rep_sign_position=(W_cy_chartable.loc[W_cy_chartable['charnames']==rep_now]).loc[:,'permsgn'].iloc[0]
                    rep_sign=W_cy_chartable.iloc[rep_sign_position,0]
                    j_induction_sign=(Induc_Wcy_W[rep_sign][Induc_Wcy_W[rep_sign]==1].index)[0]
                    orbit_set.append(list((F4_Data.loc[F4_Data['charnames']==str(j_induction_sign)]).loc[:,'orbit'])[0])
        Orbit_Set.append(list(set(orbit_set)))
    Max_Orbit_Set=[]
    for x in Orbit_Set:
            Max_Orbit_Set.append(max_orbit(x,F4_Data))
    
    # Restore the results in F4_Data
    F4_Data[str("F4_F4_")+str(cover_fold)]=Max_Orbit_Set

When $W_c$ is of type $F_4$ and $n$ is even

In [26]:
# Construct W of type F4
W=coxeter("F",4)

# Give the range of cover fold
for cover_fold in range(2,19,2):
    
    # Take the minimal Phi_cy sets 
    Phi_cy_Set=[]
    for y1,y2,y3,y4 in itertools.product(range(2),range(2),range(2),range(2)):
        y=np.array([y1,y2,y3,y4])
        if np.sum(y*np.array([2,4,6,4]))<=cover_fold:
            Phi_cy=[]
            if y1==0:
                Phi_cy.append(3)
            if y2==0:
                Phi_cy.append(2)
            if y3==0:
                Phi_cy.append(1)
            if y4==0:
                Phi_cy.append(0)
            if np.sum(y*np.array([2,4,6,4]))==cover_fold:
                Phi_cy.append(23)
            Phi_cy_Set.append(Phi_cy)
    min_Phi_cy_Set=minimal_W_cy(Phi_cy_Set)
    
    # In the following, we compute by going through all representations
    Orbit_Set=[]
    Induc_Wcy_W_set=[]
    W_cy_special_rep_set=[]
    for Phi_cy in min_Phi_cy_Set:
        W_cy=reflectionsubgroup(W,Phi_cy)
        W_cy_chartable=pd.DataFrame(chartable(W_cy))
        W_cy_special_rep=W_cy_chartable.loc[W_cy_chartable['b']==W_cy_chartable['a']]['charnames']
        W_cy_special_rep_set.append(W_cy_special_rep)
        induc_Wcy_W=inductiontable(W_cy,W,invchar=(lambda G:chartable(G, chars=False)['b']))
        Induc_Wcy_W=pd.DataFrame(induc_Wcy_W['scalar'],columns=induc_Wcy_W['charH'],index=induc_Wcy_W['charW'])
        Induc_Wcy_W_set.append(Induc_Wcy_W)
    for char in F4_Data.loc[:,'charnames']:
        orbit_set=[]
        for i in range(len(min_Phi_cy_Set)):
            Phi_cy=min_Phi_cy_Set[i]
            W_cy=reflectionsubgroup(W,Phi_cy)
            W_cy_chartable=pd.DataFrame(chartable(W_cy))
            W_cy_special_rep=W_cy_special_rep_set[i]
            Induc_Wcy_W=Induc_Wcy_W_set[i]
            for rep_now in W_cy_special_rep:
                j_induction_now=dual_rep_F4((Induc_Wcy_W[rep_now][Induc_Wcy_W[rep_now]==1].index)[0])
                if list((F4_Data.loc[F4_Data['charnames']==str(j_induction_now)]).loc[:,'orbit'])[0] in list((F4_Data.loc[F4_Data['charnames']==char]).loc[:,'Georbit'])[0]:
                    rep_sign_position=(W_cy_chartable.loc[W_cy_chartable['charnames']==rep_now]).loc[:,'permsgn'].iloc[0]
                    rep_sign=W_cy_chartable.iloc[rep_sign_position,0]
                    j_induction_sign=dual_rep_F4((Induc_Wcy_W[rep_sign][Induc_Wcy_W[rep_sign]==1].index)[0])
                    orbit_set.append(list((F4_Data.loc[F4_Data['charnames']==str(j_induction_sign)]).loc[:,'orbit'])[0])
        Orbit_Set.append(list(set(orbit_set)))
    Max_Orbit_Set=[]
    for x in Orbit_Set:
            Max_Orbit_Set.append(max_orbit(x,F4_Data))
    
    # Restore the results in F4_Data
    F4_Data[str("F4_F4_")+str(cover_fold)]=Max_Orbit_Set

When $W_c$ is of the type $A_1+C_3$ and $n$ is odd

In [27]:
# Construct W of type F4 and Wc of type A1+C3
W1=coxeter("F",4)
W2=coxeter([[2, -1, 0,0], [-2, 2, -1,0], [0, -1, 2,0],[0,0,0,2]])
Sat_dict={('[[1, 1, 1], []]', '[1, 1]'):'A1',
 ('[[1, 1, 1], []]', '[2]'):'~A1',
 ('[[1, 1], [1]]', '[1, 1]'):'A1+~A1',
 ('[[1, 1], [1]]', '[2]'):'A2',
 ('[[1], [1, 1]]', '[1, 1]'):'~A1',
 ('[[1], [1, 1]]', '[2]'):'A1+~A1',
 ('[[], [1, 1, 1]]', '[1, 1]'):'1',
 ('[[], [1, 1, 1]]', '[2]'):'A1',
 ('[[2, 1], []]', '[1, 1]'):'B2',
 ('[[2, 1], []]', '[2]'):'C3(a1)',
 ('[[1], [2]]', '[1, 1]'):'~A2',
 ('[[1], [2]]', '[2]'):'~A2+A1',
 ('[[2], [1]]', '[1, 1]'):'C3(a1)',
 ('[[2], [1]]', '[2]'):'F4(a3)',
 ('[[3], []]', '[1, 1]'):'C3',
 ('[[3], []]', '[2]'):'F4(a2)'}

weight=[3,4,2]
drop_weight=2
node_set_1=[list(p) for p in itertools.product(range(drop_weight+1), repeat=len(weight))]
node_set_2=[]
for node in node_set_1:
    c=(-np.sum(np.array(node)*weight)-1)%drop_weight
    node_set_2.append(np.append(node,c))
    if c==0:
        node_set_2.append(np.append(node,drop_weight))

for cover_fold in range(1,8,2):
    node_set_3=[]
    for node in node_set_2:
        if node[0]+2*node[1]+2*node[2]<=cover_fold and node[3]<=cover_fold:
            add_point=[]
            for i in range(4):
                if node[i]==0:
                    add_point.append(i)
            if node[0]+2*node[1]+2*node[2]==cover_fold:
                add_point.append(9)
            if node[3]==cover_fold:
                add_point.append(3)
            node_set_3.append(add_point)
    
    node_set_3=[list(tpl) for tpl in {tuple(item) for item in node_set_3}]
    minimal_node_set_W2=minimal_W_cy(node_set_3)
    
    dict={0:1,1:2,2:3,3:47}
    minimal_node_set_W1=[]
    for node in minimal_node_set_W2:
        minimal_node_set_W1.append([convert_fuc(W1.roots,W2.roots,Index,dict) for Index in node])

    Target=[]
    Induc_W1cy_W1_set=[]
    Induc_W2cy_W2_set=[]
    W1_cy_special_rep_set=[]
    for i in range(len(minimal_node_set_W1)):
        Phi1_cy=minimal_node_set_W1[i]
        Phi2_cy=minimal_node_set_W2[i]
        W1_cy=reflectionsubgroup(W1,Phi1_cy)
        W2_cy=reflectionsubgroup(W2,Phi2_cy)
        W1_cy_chartable=pd.DataFrame(chartable(W1_cy))
        W1_cy_special_rep=W1_cy_chartable.loc[W1_cy_chartable['b']==W1_cy_chartable['a']]['charnames']
        W1_cy_special_rep_set.append(W1_cy_special_rep)
        induc_W1cy_W1=inductiontable(W1_cy,W1,invchar=(lambda G:chartable(G, chars=False)['b']))
        Induc_W1cy_W1=pd.DataFrame(induc_W1cy_W1['scalar'],columns=induc_W1cy_W1['charH'],index=induc_W1cy_W1['charW'])
        Induc_W1cy_W1_set.append(Induc_W1cy_W1)
        induc_W2cy_W2=inductiontable(W2_cy,W2,invchar=(lambda G:chartable(G, chars=False)['b']))
        Induc_W2cy_W2=pd.DataFrame(induc_W2cy_W2['scalar'],columns=induc_W2cy_W2['charH'],index=induc_W2cy_W2['charW'])
        Induc_W2cy_W2_set.append(Induc_W2cy_W2)
    for char in F4_Data.loc[:,'charnames']:
        j_induction_sign_set=[]
        for i in range(len(minimal_node_set_W1)):
            Phi1_cy=minimal_node_set_W1[i]
            Phi2_cy=minimal_node_set_W2[i]
            W1_cy=reflectionsubgroup(W1,Phi1_cy)
            W2_cy=reflectionsubgroup(W2,Phi2_cy)
            W1_cy_chartable=pd.DataFrame(chartable(W1_cy))
            W1_cy_special_rep=W1_cy_special_rep_set[i]
            Induc_W1cy_W1=Induc_W1cy_W1_set[i]
            Induc_W2cy_W2=Induc_W2cy_W2_set[i]
            for rep_now in W1_cy_special_rep:
                j_induction_now=dual_rep_F4((Induc_W1cy_W1[rep_now][Induc_W1cy_W1[rep_now]==1].index)[0])
                if list((F4_Data.loc[F4_Data['charnames']==str(j_induction_now)]).loc[:,'orbit'])[0] in list((F4_Data.loc[F4_Data['charnames']==char]).loc[:,'Georbit'])[0]:
                    rep_sign_position=(W1_cy_chartable.loc[W1_cy_chartable['charnames']==rep_now]).loc[:,'permsgn'].iloc[0]
                    rep_sign=W1_cy_chartable.iloc[rep_sign_position,0]
                    if rep_sign not in Induc_W2cy_W2.columns:
                        rep_sign=switch_A3_D3(rep_sign,Induc_W2cy_W2.columns)
                    j_induction_sign=(Induc_W2cy_W2[rep_sign][Induc_W2cy_W2[rep_sign]==1].index)[0]
                    j_induction_sign_set.append(j_induction_sign)
        Target.append(max_orbit(list(set([Sat_dict.get(x) for x in set(j_induction_sign_set)])),F4_Data))
    F4_Data["F4_A1+C3_"+str(cover_fold)]=Target

When $W_c$ is of the type $A_1+C_3$ and $n$ is even

In [28]:
# Construct W of type F4 and Wc of type A1+C3
W1=coxeter("F",4)
W2=coxeter([[2, -2, 0,0], [-1, 2, -1,0], [0, -1, 2,0],[0,0,0,2]])
Sat_dict={('[[1, 1, 1], []]', '[1, 1]'):'A1',
 ('[[1, 1, 1], []]', '[2]'):'~A1',
 ('[[1, 1], [1]]', '[1, 1]'):'A1+~A1',
 ('[[1, 1], [1]]', '[2]'):'A2',
 ('[[1], [1, 1]]', '[1, 1]'):'~A1',
 ('[[1], [1, 1]]', '[2]'):'A1+~A1',
 ('[[], [1, 1, 1]]', '[1, 1]'):'1',
 ('[[], [1, 1, 1]]', '[2]'):'A1',
 ('[[2, 1], []]', '[1, 1]'):'B2',
 ('[[2, 1], []]', '[2]'):'C3(a1)',
 ('[[1], [2]]', '[1, 1]'):'~A2',
 ('[[1], [2]]', '[2]'):'~A2+A1',
 ('[[2], [1]]', '[1, 1]'):'C3(a1)',
 ('[[2], [1]]', '[2]'):'F4(a3)',
 ('[[3], []]', '[1, 1]'):'C3',
 ('[[3], []]', '[2]'):'F4(a2)'}

weight=[3,4,2]
drop_weight=2
node_set_1=[list(p) for p in itertools.product(range(drop_weight+1), repeat=len(weight))]
node_set_2=[]
for node in node_set_1:
    c=(-np.sum(np.array(node)*weight)-1)%drop_weight
    node_set_2.append(np.append(node,c))
    if c==0:
        node_set_2.append(np.append(node,drop_weight))

for cover_fold in range(2,11,2):
    node_set_3=[]
    for node in node_set_2:
        if 2*node[0]+4*node[1]+2*node[2]<=cover_fold and node[3]<=cover_fold:
            add_point=[]
            for i in range(4):
                if node[i]==0:
                    add_point.append(i)
            if 2*node[0]+4*node[1]+2*node[2]==cover_fold:
                add_point.append(9)
            if node[3]==cover_fold:
                add_point.append(3)
            node_set_3.append(add_point)
    
    node_set_3=[list(tpl) for tpl in {tuple(item) for item in node_set_3}]
    minimal_node_set_W2=minimal_W_cy(node_set_3)
    
    dict={0:2,1:1,2:0,3:44}
    minimal_node_set_W1=[]
    for node in minimal_node_set_W2:
        minimal_node_set_W1.append([convert_fuc(W1.roots,W2.roots,Index,dict) for Index in node])

    Target=[]
    Induc_W1cy_W1_set=[]
    Induc_W2cy_W2_set=[]
    W1_cy_special_rep_set=[]
    for i in range(len(minimal_node_set_W1)):
        Phi1_cy=minimal_node_set_W1[i]
        Phi2_cy=minimal_node_set_W2[i]
        W1_cy=reflectionsubgroup(W1,Phi1_cy)
        W2_cy=reflectionsubgroup(W2,Phi2_cy)
        W1_cy_chartable=pd.DataFrame(chartable(W1_cy))
        W1_cy_special_rep=W1_cy_chartable.loc[W1_cy_chartable['b']==W1_cy_chartable['a']]['charnames']
        W1_cy_special_rep_set.append(W1_cy_special_rep)
        induc_W1cy_W1=inductiontable(W1_cy,W1,invchar=(lambda G:chartable(G, chars=False)['b']))
        Induc_W1cy_W1=pd.DataFrame(induc_W1cy_W1['scalar'],columns=induc_W1cy_W1['charH'],index=induc_W1cy_W1['charW'])
        Induc_W1cy_W1_set.append(Induc_W1cy_W1)
        induc_W2cy_W2=inductiontable(W2_cy,W2,invchar=(lambda G:chartable(G, chars=False)['b']))
        Induc_W2cy_W2=pd.DataFrame(induc_W2cy_W2['scalar'],columns=induc_W2cy_W2['charH'],index=induc_W2cy_W2['charW'])
        Induc_W2cy_W2_set.append(Induc_W2cy_W2)
    for char in F4_Data.loc[:,'charnames']:
        j_induction_sign_set=[]
        for i in range(len(minimal_node_set_W1)):
            Phi1_cy=minimal_node_set_W1[i]
            Phi2_cy=minimal_node_set_W2[i]
            W1_cy=reflectionsubgroup(W1,Phi1_cy)
            W2_cy=reflectionsubgroup(W2,Phi2_cy)
            W1_cy_chartable=pd.DataFrame(chartable(W1_cy))
            W1_cy_special_rep=W1_cy_special_rep_set[i]
            Induc_W1cy_W1=Induc_W1cy_W1_set[i]
            Induc_W2cy_W2=Induc_W2cy_W2_set[i]
            for rep_now in W1_cy_special_rep:
                j_induction_now=dual_rep_F4((Induc_W1cy_W1[rep_now][Induc_W1cy_W1[rep_now]==1].index)[0])
                if list((F4_Data.loc[F4_Data['charnames']==str(j_induction_now)]).loc[:,'orbit'])[0] in list((F4_Data.loc[F4_Data['charnames']==char]).loc[:,'Georbit'])[0]:
                    rep_sign_position=(W1_cy_chartable.loc[W1_cy_chartable['charnames']==rep_now]).loc[:,'permsgn'].iloc[0]
                    rep_sign=W1_cy_chartable.iloc[rep_sign_position,0]
                    if rep_sign not in Induc_W2cy_W2.columns:
                        rep_sign=switch_A3_D3(rep_sign,Induc_W2cy_W2.columns)
                    j_induction_sign=(Induc_W2cy_W2[rep_sign][Induc_W2cy_W2[rep_sign]==1].index)[0]
                    j_induction_sign_set.append(j_induction_sign)
        Target.append(max_orbit(list(set([Sat_dict.get(x) for x in set(j_induction_sign_set)])),F4_Data))
    F4_Data["F4_A1+C3_"+str(cover_fold)]=Target

When $W_c$ is of the type $A_2+\tilde{A_2}$ and $n$ is odd

In [29]:
# Construct W of type F4 and Wc of type A2+~A2
W1=coxeter("F",4)
W2=coxeter([[2, -1, 0,0], [-1, 2, 0,0], [0, 0, 2,-1],[0,0,-1,2]])
Sat_dict={('[1, 1, 1]', '[1, 1, 1]'):'1',
 ('[1, 1, 1]', '[2, 1]'):'A1',
 ('[1, 1, 1]', '[3]'):'A2',
 ('[2, 1]', '[1, 1, 1]'):'~A1',
 ('[2, 1]', '[2, 1]'):'A1+~A1',
 ('[2, 1]', '[3]'):'A2+~A1',
 ('[3]', '[1, 1, 1]'):'~A2',
 ('[3]', '[2, 1]'):'~A2+A1',
 ('[3]', '[3]'):'F4(a3)'}

weight=[4,2,2]
drop_weight=3
node_set_1=[list(p) for p in itertools.product(range(drop_weight+1), repeat=len(weight))]
node_set_2=[]
for node in node_set_1:
    c=(-np.sum(np.array(node)*weight)-1)%drop_weight
    node_set_2.append(np.append(node,c))
    if c==0:
        node_set_2.append(np.append(node,drop_weight))

for cover_fold in range(1,6,2):
    node_set_3=[]
    for node in node_set_2:
        if node[0]+node[1]<=cover_fold and node[2]+node[3]<=cover_fold:
            add_point=[]
            for i in range(4):
                if node[i]==0:
                    add_point.append(i)
            if node[0]+node[1]==cover_fold:
                add_point.append(4)
            if node[2]+node[3]==cover_fold:
                add_point.append(5)
            node_set_3.append(add_point)
    
    node_set_3=[list(tpl) for tpl in {tuple(item) for item in node_set_3}]
    minimal_node_set_W2=minimal_W_cy(node_set_3)
    
    dict={0:2,1:3,2:0,3:47}
    minimal_node_set_W1=[]
    for node in minimal_node_set_W2:
        minimal_node_set_W1.append([convert_fuc(W1.roots,W2.roots,Index,dict) for Index in node])

    Target=[]
    Induc_W1cy_W1_set=[]
    Induc_W2cy_W2_set=[]
    W1_cy_special_rep_set=[]
    for i in range(len(minimal_node_set_W1)):
        Phi1_cy=minimal_node_set_W1[i]
        Phi2_cy=minimal_node_set_W2[i]
        W1_cy=reflectionsubgroup(W1,Phi1_cy)
        W2_cy=reflectionsubgroup(W2,Phi2_cy)
        W1_cy_chartable=pd.DataFrame(chartable(W1_cy))
        W1_cy_special_rep=W1_cy_chartable.loc[W1_cy_chartable['b']==W1_cy_chartable['a']]['charnames']
        W1_cy_special_rep_set.append(W1_cy_special_rep)
        induc_W1cy_W1=inductiontable(W1_cy,W1,invchar=(lambda G:chartable(G, chars=False)['b']))
        Induc_W1cy_W1=pd.DataFrame(induc_W1cy_W1['scalar'],columns=induc_W1cy_W1['charH'],index=induc_W1cy_W1['charW'])
        Induc_W1cy_W1_set.append(Induc_W1cy_W1)
        induc_W2cy_W2=inductiontable(W2_cy,W2,invchar=(lambda G:chartable(G, chars=False)['b']))
        Induc_W2cy_W2=pd.DataFrame(induc_W2cy_W2['scalar'],columns=induc_W2cy_W2['charH'],index=induc_W2cy_W2['charW'])
        Induc_W2cy_W2_set.append(Induc_W2cy_W2)
    for char in F4_Data.loc[:,'charnames']:
        j_induction_sign_set=[]
        for i in range(len(minimal_node_set_W1)):
            Phi1_cy=minimal_node_set_W1[i]
            Phi2_cy=minimal_node_set_W2[i]
            W1_cy=reflectionsubgroup(W1,Phi1_cy)
            W2_cy=reflectionsubgroup(W2,Phi2_cy)
            W1_cy_chartable=pd.DataFrame(chartable(W1_cy))
            W1_cy_special_rep=W1_cy_special_rep_set[i]
            Induc_W1cy_W1=Induc_W1cy_W1_set[i]
            Induc_W2cy_W2=Induc_W2cy_W2_set[i]
            for rep_now in W1_cy_special_rep:
                j_induction_now=dual_rep_F4((Induc_W1cy_W1[rep_now][Induc_W1cy_W1[rep_now]==1].index)[0])
                if list((F4_Data.loc[F4_Data['charnames']==str(j_induction_now)]).loc[:,'orbit'])[0] in list((F4_Data.loc[F4_Data['charnames']==char]).loc[:,'Georbit'])[0]:
                    rep_sign_position=(W1_cy_chartable.loc[W1_cy_chartable['charnames']==rep_now]).loc[:,'permsgn'].iloc[0]
                    rep_sign=W1_cy_chartable.iloc[rep_sign_position,0]
                    if rep_sign not in Induc_W2cy_W2.columns:
                        rep_sign=switch_A3_D3(rep_sign,Induc_W2cy_W2.columns)
                    j_induction_sign=(Induc_W2cy_W2[rep_sign][Induc_W2cy_W2[rep_sign]==1].index)[0]
                    j_induction_sign_set.append(j_induction_sign)
        Target.append(max_orbit(list(set([Sat_dict.get(x) for x in set(j_induction_sign_set)])),F4_Data))
    F4_Data["F4_A2+~A2_"+str(cover_fold)]=Target

When $W_c$ is of the type $A_2+\tilde{A_2}$ and $n$ is even

In [30]:
# Construct W of type F4 and Wc of type A2+~A2
W1=coxeter("F",4)
W2=coxeter([[2, -1, 0,0], [-1, 2, 0,0], [0, 0, 2,-1],[0,0,-1,2]])
Sat_dict={('[1, 1, 1]', '[1, 1, 1]'):'1',
 ('[1, 1, 1]', '[2, 1]'):'A1',
 ('[1, 1, 1]', '[3]'):'A2',
 ('[2, 1]', '[1, 1, 1]'):'~A1',
 ('[2, 1]', '[2, 1]'):'A1+~A1',
 ('[2, 1]', '[3]'):'A2+~A1',
 ('[3]', '[1, 1, 1]'):'~A2',
 ('[3]', '[2, 1]'):'~A2+A1',
 ('[3]', '[3]'):'F4(a3)'}

weight=[4,2,2]
drop_weight=3
node_set_1=[list(p) for p in itertools.product(range(drop_weight+1), repeat=len(weight))]
node_set_2=[]
for node in node_set_1:
    c=(-np.sum(np.array(node)*weight)-1)%drop_weight
    node_set_2.append(np.append(node,c))
    if c==0:
        node_set_2.append(np.append(node,drop_weight))

for cover_fold in range(2,7,2):
    node_set_3=[]
    for node in node_set_2:
        if 2*node[0]+2*node[1]<=cover_fold and node[2]+node[3]<=cover_fold:
            add_point=[]
            for i in range(4):
                if node[i]==0:
                    add_point.append(i)
            if 2*node[0]+2*node[1]==cover_fold:
                add_point.append(4)
            if node[2]+node[3]==cover_fold:
                add_point.append(5)
            node_set_3.append(add_point)
    
    node_set_3=[list(tpl) for tpl in {tuple(item) for item in node_set_3}]
    minimal_node_set_W2=minimal_W_cy(node_set_3)
    
    dict={0:1,1:0,2:3,3:44}
    minimal_node_set_W1=[]
    for node in minimal_node_set_W2:
        minimal_node_set_W1.append([convert_fuc(W1.roots,W2.roots,Index,dict) for Index in node])

    Target=[]
    Induc_W1cy_W1_set=[]
    Induc_W2cy_W2_set=[]
    W1_cy_special_rep_set=[]
    for i in range(len(minimal_node_set_W1)):
        Phi1_cy=minimal_node_set_W1[i]
        Phi2_cy=minimal_node_set_W2[i]
        W1_cy=reflectionsubgroup(W1,Phi1_cy)
        W2_cy=reflectionsubgroup(W2,Phi2_cy)
        W1_cy_chartable=pd.DataFrame(chartable(W1_cy))
        W1_cy_special_rep=W1_cy_chartable.loc[W1_cy_chartable['b']==W1_cy_chartable['a']]['charnames']
        W1_cy_special_rep_set.append(W1_cy_special_rep)
        induc_W1cy_W1=inductiontable(W1_cy,W1,invchar=(lambda G:chartable(G, chars=False)['b']))
        Induc_W1cy_W1=pd.DataFrame(induc_W1cy_W1['scalar'],columns=induc_W1cy_W1['charH'],index=induc_W1cy_W1['charW'])
        Induc_W1cy_W1_set.append(Induc_W1cy_W1)
        induc_W2cy_W2=inductiontable(W2_cy,W2,invchar=(lambda G:chartable(G, chars=False)['b']))
        Induc_W2cy_W2=pd.DataFrame(induc_W2cy_W2['scalar'],columns=induc_W2cy_W2['charH'],index=induc_W2cy_W2['charW'])
        Induc_W2cy_W2_set.append(Induc_W2cy_W2)
    for char in F4_Data.loc[:,'charnames']:
        j_induction_sign_set=[]
        for i in range(len(minimal_node_set_W1)):
            Phi1_cy=minimal_node_set_W1[i]
            Phi2_cy=minimal_node_set_W2[i]
            W1_cy=reflectionsubgroup(W1,Phi1_cy)
            W2_cy=reflectionsubgroup(W2,Phi2_cy)
            W1_cy_chartable=pd.DataFrame(chartable(W1_cy))
            W1_cy_special_rep=W1_cy_special_rep_set[i]
            Induc_W1cy_W1=Induc_W1cy_W1_set[i]
            Induc_W2cy_W2=Induc_W2cy_W2_set[i]
            for rep_now in W1_cy_special_rep:
                j_induction_now=dual_rep_F4((Induc_W1cy_W1[rep_now][Induc_W1cy_W1[rep_now]==1].index)[0])
                if list((F4_Data.loc[F4_Data['charnames']==str(j_induction_now)]).loc[:,'orbit'])[0] in list((F4_Data.loc[F4_Data['charnames']==char]).loc[:,'Georbit'])[0]:
                    rep_sign_position=(W1_cy_chartable.loc[W1_cy_chartable['charnames']==rep_now]).loc[:,'permsgn'].iloc[0]
                    rep_sign=W1_cy_chartable.iloc[rep_sign_position,0]
                    if rep_sign not in Induc_W2cy_W2.columns:
                        rep_sign=switch_A3_D3(rep_sign,Induc_W2cy_W2.columns)
                    j_induction_sign=(Induc_W2cy_W2[rep_sign][Induc_W2cy_W2[rep_sign]==1].index)[0]
                    j_induction_sign_set.append(j_induction_sign)
        Target.append(max_orbit(list(set([Sat_dict.get(x) for x in set(j_induction_sign_set)])),F4_Data))
    F4_Data["F4_A2+~A2_"+str(cover_fold)]=Target

When $W_c$ is of the type $A_1+A_3$ and $n$ is odd

In [31]:
# Construct W of type F4 and Wc of type A1+A3
W1=coxeter("F",4)
W2=coxeter([[2, 0, 0,0], [0, 2,-1,0], [0, -1, 2,-1],[0,0,-1,2]])
Sat_dict={('[1, 1, 1, 1]', '[1, 1]'):'1',
 ('[1, 1, 1, 1]', '[2]'):'~A1',
 ('[2, 1, 1]', '[1, 1]'):'A1',
 ('[2, 1, 1]', '[2]'):'A1+~A1',
 ('[2, 2]', '[1, 1]'):'~A1',
 ('[2, 2]', '[2]'):'A2',
 ('[3, 1]', '[1, 1]'):'A2',
 ('[3, 1]', '[2]'):'A2+~A1',
 ('[4]', '[1, 1]'):'B2',
 ('[4]', '[2]'):'F4(a3)'}

weight=[2,3,2]
drop_weight=4
node_set_1=[list(p) for p in itertools.product(range(drop_weight+1), repeat=len(weight))]
node_set_2=[]
for node in node_set_1:
    c=(-np.sum(np.array(node)*weight)-1)%drop_weight
    node_set_2.append(np.append(node,c))
    if c==0:
        node_set_2.append(np.append(node,drop_weight))

for cover_fold in range(1,6,2):
    node_set_3=[]
    for node in node_set_2:
        if node[0]<=cover_fold and node[1]+node[2]+node[3]<=cover_fold:
            add_point=[]
            for i in range(4):
                if node[i]==0:
                    add_point.append(i)
            if node[0]==cover_fold:
                add_point.append(0)
            if node[1]+node[2]+node[3]==cover_fold:
                add_point.append(6)
            node_set_3.append(add_point)
    
    node_set_3=[list(tpl) for tpl in {tuple(item) for item in node_set_3}]
    minimal_node_set_W2=minimal_W_cy(node_set_3)
    
    dict={0:3,1:1,2:0,3:47}
    minimal_node_set_W1=[]
    for node in minimal_node_set_W2:
        minimal_node_set_W1.append([convert_fuc(W1.roots,W2.roots,Index,dict) for Index in node])

    Target=[]
    Induc_W1cy_W1_set=[]
    Induc_W2cy_W2_set=[]
    W1_cy_special_rep_set=[]
    for i in range(len(minimal_node_set_W1)):
        Phi1_cy=minimal_node_set_W1[i]
        Phi2_cy=minimal_node_set_W2[i]
        W1_cy=reflectionsubgroup(W1,Phi1_cy)
        W2_cy=reflectionsubgroup(W2,Phi2_cy)
        W1_cy_chartable=pd.DataFrame(chartable(W1_cy))
        W1_cy_special_rep=W1_cy_chartable.loc[W1_cy_chartable['b']==W1_cy_chartable['a']]['charnames']
        W1_cy_special_rep_set.append(W1_cy_special_rep)
        induc_W1cy_W1=inductiontable(W1_cy,W1,invchar=(lambda G:chartable(G, chars=False)['b']))
        Induc_W1cy_W1=pd.DataFrame(induc_W1cy_W1['scalar'],columns=induc_W1cy_W1['charH'],index=induc_W1cy_W1['charW'])
        Induc_W1cy_W1_set.append(Induc_W1cy_W1)
        induc_W2cy_W2=inductiontable(W2_cy,W2,invchar=(lambda G:chartable(G, chars=False)['b']))
        Induc_W2cy_W2=pd.DataFrame(induc_W2cy_W2['scalar'],columns=induc_W2cy_W2['charH'],index=induc_W2cy_W2['charW'])
        Induc_W2cy_W2_set.append(Induc_W2cy_W2)
    for char in F4_Data.loc[:,'charnames']:
        j_induction_sign_set=[]
        for i in range(len(minimal_node_set_W1)):
            Phi1_cy=minimal_node_set_W1[i]
            Phi2_cy=minimal_node_set_W2[i]
            W1_cy=reflectionsubgroup(W1,Phi1_cy)
            W2_cy=reflectionsubgroup(W2,Phi2_cy)
            W1_cy_chartable=pd.DataFrame(chartable(W1_cy))
            W1_cy_special_rep=W1_cy_special_rep_set[i]
            Induc_W1cy_W1=Induc_W1cy_W1_set[i]
            Induc_W2cy_W2=Induc_W2cy_W2_set[i]
            for rep_now in W1_cy_special_rep:
                j_induction_now=dual_rep_F4((Induc_W1cy_W1[rep_now][Induc_W1cy_W1[rep_now]==1].index)[0])
                if list((F4_Data.loc[F4_Data['charnames']==str(j_induction_now)]).loc[:,'orbit'])[0] in list((F4_Data.loc[F4_Data['charnames']==char]).loc[:,'Georbit'])[0]:
                    rep_sign_position=(W1_cy_chartable.loc[W1_cy_chartable['charnames']==rep_now]).loc[:,'permsgn'].iloc[0]
                    rep_sign=W1_cy_chartable.iloc[rep_sign_position,0]
                    if rep_sign not in Induc_W2cy_W2.columns:
                        rep_sign=switch_A3_D3(rep_sign,Induc_W2cy_W2.columns)
                    j_induction_sign=(Induc_W2cy_W2[rep_sign][Induc_W2cy_W2[rep_sign]==1].index)[0]
                    j_induction_sign_set.append(j_induction_sign)
        Target.append(max_orbit(list(set([Sat_dict.get(x) for x in set(j_induction_sign_set)])),F4_Data))
    F4_Data["F4_A1+A3_"+str(cover_fold)]=Target

When $W_c$ is of the type $A_1+A_3$ and $n$ is even

In [32]:
# Construct W of type F4 and Wc of type A1+A3
W1=coxeter("F",4)
W2=coxeter([[2, 0, 0,0], [0, 2,-1,0], [0, -1, 2,-1],[0,0,-1,2]])
Sat_dict={('[1, 1, 1, 1]', '[1, 1]'):'1',
 ('[1, 1, 1, 1]', '[2]'):'~A1',
 ('[2, 1, 1]', '[1, 1]'):'A1',
 ('[2, 1, 1]', '[2]'):'A1+~A1',
 ('[2, 2]', '[1, 1]'):'~A1',
 ('[2, 2]', '[2]'):'A2',
 ('[3, 1]', '[1, 1]'):'A2',
 ('[3, 1]', '[2]'):'A2+~A1',
 ('[4]', '[1, 1]'):'B2',
 ('[4]', '[2]'):'F4(a3)'}

weight=[2,3,2]
drop_weight=4
node_set_1=[list(p) for p in itertools.product(range(drop_weight+1), repeat=len(weight))]
node_set_2=[]
for node in node_set_1:
    c=(-np.sum(np.array(node)*weight)-1)%drop_weight
    node_set_2.append(np.append(node,c))
    if c==0:
        node_set_2.append(np.append(node,drop_weight))

for cover_fold in range(2,7,2):
    node_set_3=[]
    for node in node_set_2:
        if 2*node[0]<=cover_fold and node[1]+node[2]+node[3]<=cover_fold:
            add_point=[]
            for i in range(4):
                if node[i]==0:
                    add_point.append(i)
            if 2*node[0]==cover_fold:
                add_point.append(0)
            if node[1]+node[2]+node[3]==cover_fold:
                add_point.append(6)
            node_set_3.append(add_point)
    
    node_set_3=[list(tpl) for tpl in {tuple(item) for item in node_set_3}]
    minimal_node_set_W2=minimal_W_cy(node_set_3)
    
    dict={0:0,1:2,2:3,3:44}
    minimal_node_set_W1=[]
    for node in minimal_node_set_W2:
        minimal_node_set_W1.append([convert_fuc(W1.roots,W2.roots,Index,dict) for Index in node])

    Target=[]
    Induc_W1cy_W1_set=[]
    Induc_W2cy_W2_set=[]
    W1_cy_special_rep_set=[]
    for i in range(len(minimal_node_set_W1)):
        Phi1_cy=minimal_node_set_W1[i]
        Phi2_cy=minimal_node_set_W2[i]
        W1_cy=reflectionsubgroup(W1,Phi1_cy)
        W2_cy=reflectionsubgroup(W2,Phi2_cy)
        W1_cy_chartable=pd.DataFrame(chartable(W1_cy))
        W1_cy_special_rep=W1_cy_chartable.loc[W1_cy_chartable['b']==W1_cy_chartable['a']]['charnames']
        W1_cy_special_rep_set.append(W1_cy_special_rep)
        induc_W1cy_W1=inductiontable(W1_cy,W1,invchar=(lambda G:chartable(G, chars=False)['b']))
        Induc_W1cy_W1=pd.DataFrame(induc_W1cy_W1['scalar'],columns=induc_W1cy_W1['charH'],index=induc_W1cy_W1['charW'])
        Induc_W1cy_W1_set.append(Induc_W1cy_W1)
        induc_W2cy_W2=inductiontable(W2_cy,W2,invchar=(lambda G:chartable(G, chars=False)['b']))
        Induc_W2cy_W2=pd.DataFrame(induc_W2cy_W2['scalar'],columns=induc_W2cy_W2['charH'],index=induc_W2cy_W2['charW'])
        Induc_W2cy_W2_set.append(Induc_W2cy_W2)
    for char in F4_Data.loc[:,'charnames']:
        j_induction_sign_set=[]
        for i in range(len(minimal_node_set_W1)):
            Phi1_cy=minimal_node_set_W1[i]
            Phi2_cy=minimal_node_set_W2[i]
            W1_cy=reflectionsubgroup(W1,Phi1_cy)
            W2_cy=reflectionsubgroup(W2,Phi2_cy)
            W1_cy_chartable=pd.DataFrame(chartable(W1_cy))
            W1_cy_special_rep=W1_cy_special_rep_set[i]
            Induc_W1cy_W1=Induc_W1cy_W1_set[i]
            Induc_W2cy_W2=Induc_W2cy_W2_set[i]
            for rep_now in W1_cy_special_rep:
                j_induction_now=dual_rep_F4((Induc_W1cy_W1[rep_now][Induc_W1cy_W1[rep_now]==1].index)[0])
                if list((F4_Data.loc[F4_Data['charnames']==str(j_induction_now)]).loc[:,'orbit'])[0] in list((F4_Data.loc[F4_Data['charnames']==char]).loc[:,'Georbit'])[0]:
                    rep_sign_position=(W1_cy_chartable.loc[W1_cy_chartable['charnames']==rep_now]).loc[:,'permsgn'].iloc[0]
                    rep_sign=W1_cy_chartable.iloc[rep_sign_position,0]
                    if rep_sign not in Induc_W2cy_W2.columns:
                        rep_sign=switch_A3_D3(rep_sign,Induc_W2cy_W2.columns)
                    j_induction_sign=(Induc_W2cy_W2[rep_sign][Induc_W2cy_W2[rep_sign]==1].index)[0]
                    j_induction_sign_set.append(j_induction_sign)
        Target.append(max_orbit(list(set([Sat_dict.get(x) for x in set(j_induction_sign_set)])),F4_Data))
    F4_Data["F4_A1+A3_"+str(cover_fold)]=Target

When $W_c$ is of the type $B_4$ and $n$ is odd

In [33]:
# Construct W of type F4 and Wc of type B4
W1=coxeter("F",4)
W2=coxeter("B",4)
Sat_dict={('[[1, 1], [1, 1]]',):'A1+~A1',
 ('[[1], [1, 1, 1]]',):'~A1',
 ('[[], [1, 1, 1, 1]]',):'1',
 ('[[1, 1], [2]]',):'A2+~A1',
 ('[[2, 1], [1]]',):'C3(a1)',
 ('[[1], [2, 1]]',):'A2',
 ('[[2], [1, 1]]',):'B2',
 ('[[], [2, 1, 1]]',):'A1',
 ('[[2], [2]]',):'F4(a3)',
 ('[[], [2, 2]]',):'~A1',
 ('[[1], [3]]',):'B2',
 ('[[3], [1]]',):'B3',
 ('[[4], []]',):'F4(a1)'}

weight=[4,3,2]
drop_weight=2
node_set_1=[list(p) for p in itertools.product(range(drop_weight+1), repeat=len(weight))]
node_set_2=[]
for node in node_set_1:
    c=(-np.sum(np.array(node)*weight)-1)%drop_weight
    node_set_2.append(np.append(node,c))
    if c==0:
        node_set_2.append(np.append(node,drop_weight))

for cover_fold in range(1,10,2):
    node_set_3=[]
    for node in node_set_2:
        if 2*node[0]+2*node[1]+2*node[2]+node[3]<=cover_fold:
            add_point=[]
            for i in range(4):
                if node[i]==0:
                    add_point.append(i)
            if 2*node[0]+2*node[1]+2*node[2]+node[3]==cover_fold:
                add_point.append(15)
            node_set_3.append(add_point)
    
    node_set_3=[list(tpl) for tpl in {tuple(item) for item in node_set_3}]
    minimal_node_set_W2=minimal_W_cy(node_set_3)
    
    dict={0:2,1:1,2:0,3:47}
    minimal_node_set_W1=[]
    for node in minimal_node_set_W2:
        minimal_node_set_W1.append([convert_fuc(W1.roots,W2.roots,Index,dict) for Index in node])

    Target=[]
    Induc_W1cy_W1_set=[]
    Induc_W2cy_W2_set=[]
    W1_cy_special_rep_set=[]
    for i in range(len(minimal_node_set_W1)):
        Phi1_cy=minimal_node_set_W1[i]
        Phi2_cy=minimal_node_set_W2[i]
        W1_cy=reflectionsubgroup(W1,Phi1_cy)
        W2_cy=reflectionsubgroup(W2,Phi2_cy)
        W1_cy_chartable=pd.DataFrame(chartable(W1_cy))
        W1_cy_special_rep=W1_cy_chartable.loc[W1_cy_chartable['b']==W1_cy_chartable['a']]['charnames']
        W1_cy_special_rep_set.append(W1_cy_special_rep)
        induc_W1cy_W1=inductiontable(W1_cy,W1,invchar=(lambda G:chartable(G, chars=False)['b']))
        Induc_W1cy_W1=pd.DataFrame(induc_W1cy_W1['scalar'],columns=induc_W1cy_W1['charH'],index=induc_W1cy_W1['charW'])
        Induc_W1cy_W1_set.append(Induc_W1cy_W1)
        induc_W2cy_W2=inductiontable(W2_cy,W2,invchar=(lambda G:chartable(G, chars=False)['b']))
        Induc_W2cy_W2=pd.DataFrame(induc_W2cy_W2['scalar'],columns=induc_W2cy_W2['charH'],index=induc_W2cy_W2['charW'])
        Induc_W2cy_W2_set.append(Induc_W2cy_W2)
    for char in F4_Data.loc[:,'charnames']:
        j_induction_sign_set=[]
        for i in range(len(minimal_node_set_W1)):
            Phi1_cy=minimal_node_set_W1[i]
            Phi2_cy=minimal_node_set_W2[i]
            W1_cy=reflectionsubgroup(W1,Phi1_cy)
            W2_cy=reflectionsubgroup(W2,Phi2_cy)
            W1_cy_chartable=pd.DataFrame(chartable(W1_cy))
            W1_cy_special_rep=W1_cy_special_rep_set[i]
            Induc_W1cy_W1=Induc_W1cy_W1_set[i]
            Induc_W2cy_W2=Induc_W2cy_W2_set[i]
            for rep_now in W1_cy_special_rep:
                j_induction_now=dual_rep_F4((Induc_W1cy_W1[rep_now][Induc_W1cy_W1[rep_now]==1].index)[0])
                if list((F4_Data.loc[F4_Data['charnames']==str(j_induction_now)]).loc[:,'orbit'])[0] in list((F4_Data.loc[F4_Data['charnames']==char]).loc[:,'Georbit'])[0]:
                    rep_sign_position=(W1_cy_chartable.loc[W1_cy_chartable['charnames']==rep_now]).loc[:,'permsgn'].iloc[0]
                    rep_sign=W1_cy_chartable.iloc[rep_sign_position,0]
                    if rep_sign not in Induc_W2cy_W2.columns:
                        rep_sign=switch_A3_D3(rep_sign,Induc_W2cy_W2.columns)
                    j_induction_sign=(Induc_W2cy_W2[rep_sign][Induc_W2cy_W2[rep_sign]==1].index)[0]
                    j_induction_sign_set.append(j_induction_sign)
        Target.append(max_orbit(list(set([Sat_dict.get(x) for x in set(j_induction_sign_set)])),F4_Data))
    F4_Data["F4_B4_"+str(cover_fold)]=Target

When $W_c$ is of the type $B_4$ and $n$ is even

In [34]:
# Construct W of type F4 and Wc of type B4
W1=coxeter("F",4)
W2=coxeter("C",4)
Sat_dict={('[[1, 1], [1, 1]]',):'A1+~A1',
 ('[[1], [1, 1, 1]]',):'~A1',
 ('[[], [1, 1, 1, 1]]',):'1',
 ('[[1, 1], [2]]',):'A2+~A1',
 ('[[2, 1], [1]]',):'C3(a1)',
 ('[[1], [2, 1]]',):'A2',
 ('[[2], [1, 1]]',):'B2',
 ('[[], [2, 1, 1]]',):'A1',
 ('[[2], [2]]',):'F4(a3)',
 ('[[], [2, 2]]',):'~A1',
 ('[[1], [3]]',):'B2',
 ('[[3], [1]]',):'B3',
 ('[[4], []]',):'F4(a1)'}

weight=[4,3,2]
drop_weight=2
node_set_1=[list(p) for p in itertools.product(range(drop_weight+1), repeat=len(weight))]
node_set_2=[]
for node in node_set_1:
    c=(-np.sum(np.array(node)*weight)-1)%drop_weight
    node_set_2.append(np.append(node,c))
    if c==0:
        node_set_2.append(np.append(node,drop_weight))

for cover_fold in range(2,13,2):
    node_set_3=[]
    for node in node_set_2:
        if 2*node[0]+2*node[1]+2*node[2]+2*node[3]<=cover_fold:
            add_point=[]
            for i in range(4):
                if node[i]==0:
                    add_point.append(i)
            if 2*node[0]+2*node[1]+2*node[2]+2*node[3]==cover_fold:
                add_point.append(15)
            node_set_3.append(add_point)
    
    node_set_3=[list(tpl) for tpl in {tuple(item) for item in node_set_3}]
    minimal_node_set_W2=minimal_W_cy(node_set_3)
    
    dict={0:1,1:2,2:3,3:44}
    minimal_node_set_W1=[]
    for node in minimal_node_set_W2:
        minimal_node_set_W1.append([convert_fuc(W1.roots,W2.roots,Index,dict) for Index in node])

    Target=[]
    Induc_W1cy_W1_set=[]
    Induc_W2cy_W2_set=[]
    W1_cy_special_rep_set=[]
    for i in range(len(minimal_node_set_W1)):
        Phi1_cy=minimal_node_set_W1[i]
        Phi2_cy=minimal_node_set_W2[i]
        W1_cy=reflectionsubgroup(W1,Phi1_cy)
        W2_cy=reflectionsubgroup(W2,Phi2_cy)
        W1_cy_chartable=pd.DataFrame(chartable(W1_cy))
        W1_cy_special_rep=W1_cy_chartable.loc[W1_cy_chartable['b']==W1_cy_chartable['a']]['charnames']
        W1_cy_special_rep_set.append(W1_cy_special_rep)
        induc_W1cy_W1=inductiontable(W1_cy,W1,invchar=(lambda G:chartable(G, chars=False)['b']))
        Induc_W1cy_W1=pd.DataFrame(induc_W1cy_W1['scalar'],columns=induc_W1cy_W1['charH'],index=induc_W1cy_W1['charW'])
        Induc_W1cy_W1_set.append(Induc_W1cy_W1)
        induc_W2cy_W2=inductiontable(W2_cy,W2,invchar=(lambda G:chartable(G, chars=False)['b']))
        Induc_W2cy_W2=pd.DataFrame(induc_W2cy_W2['scalar'],columns=induc_W2cy_W2['charH'],index=induc_W2cy_W2['charW'])
        Induc_W2cy_W2_set.append(Induc_W2cy_W2)
    for char in F4_Data.loc[:,'charnames']:
        j_induction_sign_set=[]
        for i in range(len(minimal_node_set_W1)):
            Phi1_cy=minimal_node_set_W1[i]
            Phi2_cy=minimal_node_set_W2[i]
            W1_cy=reflectionsubgroup(W1,Phi1_cy)
            W2_cy=reflectionsubgroup(W2,Phi2_cy)
            W1_cy_chartable=pd.DataFrame(chartable(W1_cy))
            W1_cy_special_rep=W1_cy_special_rep_set[i]
            Induc_W1cy_W1=Induc_W1cy_W1_set[i]
            Induc_W2cy_W2=Induc_W2cy_W2_set[i]
            for rep_now in W1_cy_special_rep:
                j_induction_now=dual_rep_F4((Induc_W1cy_W1[rep_now][Induc_W1cy_W1[rep_now]==1].index)[0])
                if list((F4_Data.loc[F4_Data['charnames']==str(j_induction_now)]).loc[:,'orbit'])[0] in list((F4_Data.loc[F4_Data['charnames']==char]).loc[:,'Georbit'])[0]:
                    rep_sign_position=(W1_cy_chartable.loc[W1_cy_chartable['charnames']==rep_now]).loc[:,'permsgn'].iloc[0]
                    rep_sign=W1_cy_chartable.iloc[rep_sign_position,0]
                    if rep_sign not in Induc_W2cy_W2.columns:
                        rep_sign=switch_A3_D3(rep_sign,Induc_W2cy_W2.columns)
                    j_induction_sign=(Induc_W2cy_W2[rep_sign][Induc_W2cy_W2[rep_sign]==1].index)[0]
                    j_induction_sign_set.append(j_induction_sign)
        Target.append(max_orbit(list(set([Sat_dict.get(x) for x in set(j_induction_sign_set)])),F4_Data))
    F4_Data["F4_B4_"+str(cover_fold)]=Target

$G_2$

In [35]:
G2_Data=pd.DataFrame({'charnames':["('phi_{1,0}',)", "('phi_{2,1}',)", "('phi_{2,2}',)", '("phi_{1,3}\'\'",)', "('phi_{1,6}',)"],
'orbit':['G2', 'G2(a1)', '~A1', 'A1', '1'],
'Georbit':[['G2'], ['G2(a1)', 'G2'], ['~A1','G2(a1)', 'G2'], ['A1','~A1','G2(a1)', 'G2'], ['1','A1','~A1','G2(a1)', 'G2']]})

When $W_c$ is of type $G_2$ and $n \nmid 3$

In [36]:
# Construct W of type G2
W=coxeter("G",2)

# Give the range of cover fold
for cover_fold in [1,2,4,5,7]:
    
    # Take the minimal Phi_cy sets 
    Phi_cy_Set=[]
    for y1,y2 in itertools.product(range(2),range(2)):
        y=np.array([y1,y2])
        if np.sum(y*np.array(W.roots[5]))<=cover_fold:
            Phi_cy=[]
            if y1==0:
                Phi_cy.append(0)
            if y2==0:
                Phi_cy.append(1)
            if np.sum(y*np.array(W.roots[5]))==cover_fold:
                Phi_cy.append(5)
            Phi_cy_Set.append(Phi_cy)
    min_Phi_cy_Set=minimal_W_cy(Phi_cy_Set)
    
    # In the following, we compute by going through all representations
    Orbit_Set=[]
    Induc_Wcy_W_set=[]
    W_cy_special_rep_set=[]
    for Phi_cy in min_Phi_cy_Set:
        W_cy=reflectionsubgroup(W,Phi_cy)
        W_cy_chartable=pd.DataFrame(chartable(W_cy))
        W_cy_special_rep=W_cy_chartable.loc[W_cy_chartable['b']==W_cy_chartable['a']]['charnames']
        W_cy_special_rep_set.append(W_cy_special_rep)
        induc_Wcy_W=inductiontable(W_cy,W,invchar=(lambda G:chartable(G, chars=False)['b']))
        Induc_Wcy_W=pd.DataFrame(induc_Wcy_W['scalar'],columns=induc_Wcy_W['charH'],index=induc_Wcy_W['charW'])
        Induc_Wcy_W_set.append(Induc_Wcy_W)
    for char in G2_Data.loc[:,'charnames']:
        orbit_set=[]
        for i in range(len(min_Phi_cy_Set)):
            Phi_cy=min_Phi_cy_Set[i]
            W_cy=reflectionsubgroup(W,Phi_cy)
            W_cy_chartable=pd.DataFrame(chartable(W_cy))
            W_cy_special_rep=W_cy_special_rep_set[i]
            Induc_Wcy_W=Induc_Wcy_W_set[i]
            for rep_now in W_cy_special_rep:
                j_induction_now=dual_rep_G2((Induc_Wcy_W[rep_now][Induc_Wcy_W[rep_now]==1].index)[0])
                if list((G2_Data.loc[G2_Data['charnames']==str(j_induction_now)]).loc[:,'orbit'])[0] in list((G2_Data.loc[G2_Data['charnames']==char]).loc[:,'Georbit'])[0]:
                    rep_sign_position=(W_cy_chartable.loc[W_cy_chartable['charnames']==rep_now]).loc[:,'permsgn'].iloc[0]
                    rep_sign=W_cy_chartable.iloc[rep_sign_position,0]
                    j_induction_sign=(Induc_Wcy_W[rep_sign][Induc_Wcy_W[rep_sign]==1].index)[0]
                    orbit_set.append(list((G2_Data.loc[G2_Data['charnames']==str(j_induction_sign)]).loc[:,'orbit'])[0])
        Orbit_Set.append(list(set(orbit_set)))
    Max_Orbit_Set=[]
    for x in Orbit_Set:
            Max_Orbit_Set.append(max_orbit(x,G2_Data))
    
    # Restore the results in G2_Data
    G2_Data[str("G2_G2_")+str(cover_fold)]=Max_Orbit_Set

When $W_c$ is of type $G_2$ and $n \mid 3$

In [37]:
# Construct W of type G2
W=coxeter("G",2)

# Give the range of cover fold
for cover_fold in [3,6,9,12]:
    
    # Take the minimal Phi_cy sets 
    Phi_cy_Set=[]
    for y1,y2 in itertools.product(range(2),range(2)):
        y=np.array([y1,y2])
        if np.sum(y*np.array([6,3]))<=cover_fold:
            Phi_cy=[]
            if y1==0:
                Phi_cy.append(0)
            if y2==0:
                Phi_cy.append(1)
            if np.sum(y*np.array([6,3]))==cover_fold:
                Phi_cy.append(5)
            Phi_cy_Set.append(Phi_cy)
    min_Phi_cy_Set=minimal_W_cy(Phi_cy_Set)
    
    # In the following, we compute by going through all representations
    Orbit_Set=[]
    Induc_Wcy_W_set=[]
    W_cy_special_rep_set=[]
    for Phi_cy in min_Phi_cy_Set:
        W_cy=reflectionsubgroup(W,Phi_cy)
        W_cy_chartable=pd.DataFrame(chartable(W_cy))
        W_cy_special_rep=W_cy_chartable.loc[W_cy_chartable['b']==W_cy_chartable['a']]['charnames']
        W_cy_special_rep_set.append(W_cy_special_rep)
        induc_Wcy_W=inductiontable(W_cy,W,invchar=(lambda G:chartable(G, chars=False)['b']))
        Induc_Wcy_W=pd.DataFrame(induc_Wcy_W['scalar'],columns=induc_Wcy_W['charH'],index=induc_Wcy_W['charW'])
        Induc_Wcy_W_set.append(Induc_Wcy_W)
    for char in G2_Data.loc[:,'charnames']:
        orbit_set=[]
        for i in range(len(min_Phi_cy_Set)):
            Phi_cy=min_Phi_cy_Set[i]
            W_cy=reflectionsubgroup(W,Phi_cy)
            W_cy_chartable=pd.DataFrame(chartable(W_cy))
            W_cy_special_rep=W_cy_special_rep_set[i]
            Induc_Wcy_W=Induc_Wcy_W_set[i]
            for rep_now in W_cy_special_rep:
                j_induction_now=dual_rep_G2((Induc_Wcy_W[rep_now][Induc_Wcy_W[rep_now]==1].index)[0])
                if list((G2_Data.loc[G2_Data['charnames']==str(j_induction_now)]).loc[:,'orbit'])[0] in list((G2_Data.loc[G2_Data['charnames']==char]).loc[:,'Georbit'])[0]:
                    rep_sign_position=(W_cy_chartable.loc[W_cy_chartable['charnames']==rep_now]).loc[:,'permsgn'].iloc[0]
                    rep_sign=W_cy_chartable.iloc[rep_sign_position,0]
                    j_induction_sign=dual_rep_G2((Induc_Wcy_W[rep_sign][Induc_Wcy_W[rep_sign]==1].index)[0])
                    orbit_set.append(list((G2_Data.loc[G2_Data['charnames']==str(j_induction_sign)]).loc[:,'orbit'])[0])
        Orbit_Set.append(list(set(orbit_set)))
    Max_Orbit_Set=[]
    for x in Orbit_Set:
            Max_Orbit_Set.append(max_orbit(x,G2_Data))
    
    # Restore the results in G2_Data
    G2_Data[str("G2_G2_")+str(cover_fold)]=Max_Orbit_Set

When $W_c$ is of type $A_1+\tilde{A_1}$ and $n \nmid 3$

In [38]:
# Construct W of type G2 and Wc of type A1+~A1
W1=coxeter("G",2)
W2=coxeter([[2,0],[0,2]])
Sat_dict={('[1, 1]', '[1, 1]'):'1', ('[1, 1]', '[2]'):'A1', ('[2]', '[1, 1]'):'~A1', ('[2]', '[2]'):'G2(a1)'}

weight=[3]
drop_weight=2
node_set_1=[list(p) for p in itertools.product(range(drop_weight+1), repeat=len(weight))]
node_set_2=[]
for node in node_set_1:
    c=(-np.sum(np.array(node)*weight)-1)%drop_weight
    node_set_2.append(np.append(node,c))
    if c==0:
        node_set_2.append(np.append(node,drop_weight))

for cover_fold in [1,2,4]:
    node_set_3=[]
    for node in node_set_2:
        if node[0]<=cover_fold and node[1]<=cover_fold:
            add_point=[]
            for i in range(2):
                if node[i]==0:
                    add_point.append(i)
            if node[0]==cover_fold:
                add_point.append(0)
            if node[1]==cover_fold:
                add_point.append(1)
            node_set_3.append(add_point)
    
    node_set_3=[list(tpl) for tpl in {tuple(item) for item in node_set_3}]
    minimal_node_set_W2=minimal_W_cy(node_set_3)
    
    dict={0:1,1:11}
    minimal_node_set_W1=[]
    for node in minimal_node_set_W2:
        minimal_node_set_W1.append([convert_fuc(W1.roots,W2.roots,Index,dict) for Index in node])

    Target=[]
    Induc_W1cy_W1_set=[]
    Induc_W2cy_W2_set=[]
    W1_cy_special_rep_set=[]
    for i in range(len(minimal_node_set_W1)):
        Phi1_cy=minimal_node_set_W1[i]
        Phi2_cy=minimal_node_set_W2[i]
        W1_cy=reflectionsubgroup(W1,Phi1_cy)
        W2_cy=reflectionsubgroup(W2,Phi2_cy)
        W1_cy_chartable=pd.DataFrame(chartable(W1_cy))
        W1_cy_special_rep=W1_cy_chartable.loc[W1_cy_chartable['b']==W1_cy_chartable['a']]['charnames']
        W1_cy_special_rep_set.append(W1_cy_special_rep)
        induc_W1cy_W1=inductiontable(W1_cy,W1,invchar=(lambda G:chartable(G, chars=False)['b']))
        Induc_W1cy_W1=pd.DataFrame(induc_W1cy_W1['scalar'],columns=induc_W1cy_W1['charH'],index=induc_W1cy_W1['charW'])
        Induc_W1cy_W1_set.append(Induc_W1cy_W1)
        induc_W2cy_W2=inductiontable(W2_cy,W2,invchar=(lambda G:chartable(G, chars=False)['b']))
        Induc_W2cy_W2=pd.DataFrame(induc_W2cy_W2['scalar'],columns=induc_W2cy_W2['charH'],index=induc_W2cy_W2['charW'])
        Induc_W2cy_W2_set.append(Induc_W2cy_W2)
    for char in G2_Data.loc[:,'charnames']:
        j_induction_sign_set=[]
        for i in range(len(minimal_node_set_W1)):
            Phi1_cy=minimal_node_set_W1[i]
            Phi2_cy=minimal_node_set_W2[i]
            W1_cy=reflectionsubgroup(W1,Phi1_cy)
            W2_cy=reflectionsubgroup(W2,Phi2_cy)
            W1_cy_chartable=pd.DataFrame(chartable(W1_cy))
            W1_cy_special_rep=W1_cy_special_rep_set[i]
            Induc_W1cy_W1=Induc_W1cy_W1_set[i]
            Induc_W2cy_W2=Induc_W2cy_W2_set[i]
            for rep_now in W1_cy_special_rep:
                j_induction_now=dual_rep_G2((Induc_W1cy_W1[rep_now][Induc_W1cy_W1[rep_now]==1].index)[0])
                if list((G2_Data.loc[G2_Data['charnames']==str(j_induction_now)]).loc[:,'orbit'])[0] in list((G2_Data.loc[G2_Data['charnames']==char]).loc[:,'Georbit'])[0]:
                    rep_sign_position=(W1_cy_chartable.loc[W1_cy_chartable['charnames']==rep_now]).loc[:,'permsgn'].iloc[0]
                    rep_sign=W1_cy_chartable.iloc[rep_sign_position,0]
                    j_induction_sign=(Induc_W2cy_W2[rep_sign][Induc_W2cy_W2[rep_sign]==1].index)[0]
                    j_induction_sign_set.append(j_induction_sign)
        Target.append(max_orbit(list(set([Sat_dict.get(x) for x in set(j_induction_sign_set)])),G2_Data))
    G2_Data["G2_A1+~A1_"+str(cover_fold)]=Target

When $W_c$ is of type $A_1+\tilde{A_1}$ and $n \mid 3$

In [39]:
# Construct W of type G2 and Wc of type A1+~A1
W1=coxeter("G",2)
W2=coxeter([[2,0],[0,2]])
Sat_dict={('[1, 1]', '[1, 1]'):'1', ('[1, 1]', '[2]'):'A1', ('[2]', '[1, 1]'):'~A1', ('[2]', '[2]'):'G2(a1)'}

weight=[3]
drop_weight=2
node_set_1=[list(p) for p in itertools.product(range(drop_weight+1), repeat=len(weight))]
node_set_2=[]
for node in node_set_1:
    c=(-np.sum(np.array(node)*weight)-1)%drop_weight
    node_set_2.append(np.append(node,c))
    if c==0:
        node_set_2.append(np.append(node,drop_weight))

for cover_fold in [3,6]:
    node_set_3=[]
    for node in node_set_2:
        if 3*node[0]<=cover_fold and node[1]<=cover_fold:
            add_point=[]
            for i in range(2):
                if node[i]==0:
                    add_point.append(i)
            if 3*node[0]==cover_fold:
                add_point.append(0)
            if node[1]==cover_fold:
                add_point.append(1)
            node_set_3.append(add_point)
    
    node_set_3=[list(tpl) for tpl in {tuple(item) for item in node_set_3}]
    minimal_node_set_W2=minimal_W_cy(node_set_3)
    
    dict={0:0,1:9}
    minimal_node_set_W1=[]
    for node in minimal_node_set_W2:
        minimal_node_set_W1.append([convert_fuc(W1.roots,W2.roots,Index,dict) for Index in node])

    Target=[]
    Induc_W1cy_W1_set=[]
    Induc_W2cy_W2_set=[]
    W1_cy_special_rep_set=[]
    for i in range(len(minimal_node_set_W1)):
        Phi1_cy=minimal_node_set_W1[i]
        Phi2_cy=minimal_node_set_W2[i]
        W1_cy=reflectionsubgroup(W1,Phi1_cy)
        W2_cy=reflectionsubgroup(W2,Phi2_cy)
        W1_cy_chartable=pd.DataFrame(chartable(W1_cy))
        W1_cy_special_rep=W1_cy_chartable.loc[W1_cy_chartable['b']==W1_cy_chartable['a']]['charnames']
        W1_cy_special_rep_set.append(W1_cy_special_rep)
        induc_W1cy_W1=inductiontable(W1_cy,W1,invchar=(lambda G:chartable(G, chars=False)['b']))
        Induc_W1cy_W1=pd.DataFrame(induc_W1cy_W1['scalar'],columns=induc_W1cy_W1['charH'],index=induc_W1cy_W1['charW'])
        Induc_W1cy_W1_set.append(Induc_W1cy_W1)
        induc_W2cy_W2=inductiontable(W2_cy,W2,invchar=(lambda G:chartable(G, chars=False)['b']))
        Induc_W2cy_W2=pd.DataFrame(induc_W2cy_W2['scalar'],columns=induc_W2cy_W2['charH'],index=induc_W2cy_W2['charW'])
        Induc_W2cy_W2_set.append(Induc_W2cy_W2)
    for char in G2_Data.loc[:,'charnames']:
        j_induction_sign_set=[]
        for i in range(len(minimal_node_set_W1)):
            Phi1_cy=minimal_node_set_W1[i]
            Phi2_cy=minimal_node_set_W2[i]
            W1_cy=reflectionsubgroup(W1,Phi1_cy)
            W2_cy=reflectionsubgroup(W2,Phi2_cy)
            W1_cy_chartable=pd.DataFrame(chartable(W1_cy))
            W1_cy_special_rep=W1_cy_special_rep_set[i]
            Induc_W1cy_W1=Induc_W1cy_W1_set[i]
            Induc_W2cy_W2=Induc_W2cy_W2_set[i]
            for rep_now in W1_cy_special_rep:
                j_induction_now=dual_rep_G2((Induc_W1cy_W1[rep_now][Induc_W1cy_W1[rep_now]==1].index)[0])
                if list((G2_Data.loc[G2_Data['charnames']==str(j_induction_now)]).loc[:,'orbit'])[0] in list((G2_Data.loc[G2_Data['charnames']==char]).loc[:,'Georbit'])[0]:
                    rep_sign_position=(W1_cy_chartable.loc[W1_cy_chartable['charnames']==rep_now]).loc[:,'permsgn'].iloc[0]
                    rep_sign=W1_cy_chartable.iloc[rep_sign_position,0]
                    j_induction_sign=(Induc_W2cy_W2[rep_sign][Induc_W2cy_W2[rep_sign]==1].index)[0]
                    j_induction_sign_set.append(j_induction_sign)
        Target.append(max_orbit(list(set([Sat_dict.get(x) for x in set(j_induction_sign_set)])),G2_Data))
    G2_Data["G2_A1+~A1_"+str(cover_fold)]=Target

When $W_c$ is of type $A_2$ and $n \nmid 3$

In [40]:
# Construct W of type G2 and Wc of type A2
W1=coxeter("G",2)
W2=coxeter("A",2)
Sat_dict={('[1, 1, 1]',):'1', ('[2, 1]',):'A1', ('[3]',):'G2(a1)'}

weight=[2]
drop_weight=3
node_set_1=[list(p) for p in itertools.product(range(drop_weight+1), repeat=len(weight))]
node_set_2=[]
for node in node_set_1:
    c=(-np.sum(np.array(node)*weight)-1)%drop_weight
    node_set_2.append(np.append(node,c))
    if c==0:
        node_set_2.append(np.append(node,drop_weight))

for cover_fold in [1,2,4]:
    node_set_3=[]
    for node in node_set_2:
        if node[0]+node[1]<=cover_fold:
            add_point=[]
            for i in range(2):
                if node[i]==0:
                    add_point.append(i)
            if node[0]+node[1]==cover_fold:
                add_point.append(2)
            node_set_3.append(add_point)
    
    node_set_3=[list(tpl) for tpl in {tuple(item) for item in node_set_3}]
    minimal_node_set_W2=minimal_W_cy(node_set_3)
    
    dict={0:0,1:11}
    minimal_node_set_W1=[]
    for node in minimal_node_set_W2:
        minimal_node_set_W1.append([convert_fuc(W1.roots,W2.roots,Index,dict) for Index in node])

    Target=[]
    Induc_W1cy_W1_set=[]
    Induc_W2cy_W2_set=[]
    W1_cy_special_rep_set=[]
    for i in range(len(minimal_node_set_W1)):
        Phi1_cy=minimal_node_set_W1[i]
        Phi2_cy=minimal_node_set_W2[i]
        W1_cy=reflectionsubgroup(W1,Phi1_cy)
        W2_cy=reflectionsubgroup(W2,Phi2_cy)
        W1_cy_chartable=pd.DataFrame(chartable(W1_cy))
        W1_cy_special_rep=W1_cy_chartable.loc[W1_cy_chartable['b']==W1_cy_chartable['a']]['charnames']
        W1_cy_special_rep_set.append(W1_cy_special_rep)
        induc_W1cy_W1=inductiontable(W1_cy,W1,invchar=(lambda G:chartable(G, chars=False)['b']))
        Induc_W1cy_W1=pd.DataFrame(induc_W1cy_W1['scalar'],columns=induc_W1cy_W1['charH'],index=induc_W1cy_W1['charW'])
        Induc_W1cy_W1_set.append(Induc_W1cy_W1)
        induc_W2cy_W2=inductiontable(W2_cy,W2,invchar=(lambda G:chartable(G, chars=False)['b']))
        Induc_W2cy_W2=pd.DataFrame(induc_W2cy_W2['scalar'],columns=induc_W2cy_W2['charH'],index=induc_W2cy_W2['charW'])
        Induc_W2cy_W2_set.append(Induc_W2cy_W2)
    for char in G2_Data.loc[:,'charnames']:
        j_induction_sign_set=[]
        for i in range(len(minimal_node_set_W1)):
            Phi1_cy=minimal_node_set_W1[i]
            Phi2_cy=minimal_node_set_W2[i]
            W1_cy=reflectionsubgroup(W1,Phi1_cy)
            W2_cy=reflectionsubgroup(W2,Phi2_cy)
            W1_cy_chartable=pd.DataFrame(chartable(W1_cy))
            W1_cy_special_rep=W1_cy_special_rep_set[i]
            Induc_W1cy_W1=Induc_W1cy_W1_set[i]
            Induc_W2cy_W2=Induc_W2cy_W2_set[i]
            for rep_now in W1_cy_special_rep:
                j_induction_now=dual_rep_G2((Induc_W1cy_W1[rep_now][Induc_W1cy_W1[rep_now]==1].index)[0])
                if list((G2_Data.loc[G2_Data['charnames']==str(j_induction_now)]).loc[:,'orbit'])[0] in list((G2_Data.loc[G2_Data['charnames']==char]).loc[:,'Georbit'])[0]:
                    rep_sign_position=(W1_cy_chartable.loc[W1_cy_chartable['charnames']==rep_now]).loc[:,'permsgn'].iloc[0]
                    rep_sign=W1_cy_chartable.iloc[rep_sign_position,0]
                    j_induction_sign=(Induc_W2cy_W2[rep_sign][Induc_W2cy_W2[rep_sign]==1].index)[0]
                    j_induction_sign_set.append(j_induction_sign)
        Target.append(max_orbit(list(set([Sat_dict.get(x) for x in set(j_induction_sign_set)])),G2_Data))
    G2_Data["G2_A2_"+str(cover_fold)]=Target

When $W_c$ is of type $A_2$ and $n \mid 3$

In [41]:
# Construct W of type G2 and Wc of type A2
W1=coxeter("G",2)
W2=coxeter("A",2)
Sat_dict={('[1, 1, 1]',):'1', ('[2, 1]',):'A1', ('[3]',):'G2(a1)'}

weight=[2]
drop_weight=3
node_set_1=[list(p) for p in itertools.product(range(drop_weight+1), repeat=len(weight))]
node_set_2=[]
for node in node_set_1:
    c=(-np.sum(np.array(node)*weight)-1)%drop_weight
    node_set_2.append(np.append(node,c))
    if c==0:
        node_set_2.append(np.append(node,drop_weight))

for cover_fold in [3,6]:
    node_set_3=[]
    for node in node_set_2:
        if node[0]+node[1]<=cover_fold:
            add_point=[]
            for i in range(2):
                if node[i]==0:
                    add_point.append(i)
            if node[0]+node[1]==cover_fold:
                add_point.append(2)
            node_set_3.append(add_point)
    
    node_set_3=[list(tpl) for tpl in {tuple(item) for item in node_set_3}]
    minimal_node_set_W2=minimal_W_cy(node_set_3)
    
    dict={0:1,1:9}
    minimal_node_set_W1=[]
    for node in minimal_node_set_W2:
        minimal_node_set_W1.append([convert_fuc(W1.roots,W2.roots,Index,dict) for Index in node])

    Target=[]
    Induc_W1cy_W1_set=[]
    Induc_W2cy_W2_set=[]
    W1_cy_special_rep_set=[]
    for i in range(len(minimal_node_set_W1)):
        Phi1_cy=minimal_node_set_W1[i]
        Phi2_cy=minimal_node_set_W2[i]
        W1_cy=reflectionsubgroup(W1,Phi1_cy)
        W2_cy=reflectionsubgroup(W2,Phi2_cy)
        W1_cy_chartable=pd.DataFrame(chartable(W1_cy))
        W1_cy_special_rep=W1_cy_chartable.loc[W1_cy_chartable['b']==W1_cy_chartable['a']]['charnames']
        W1_cy_special_rep_set.append(W1_cy_special_rep)
        induc_W1cy_W1=inductiontable(W1_cy,W1,invchar=(lambda G:chartable(G, chars=False)['b']))
        Induc_W1cy_W1=pd.DataFrame(induc_W1cy_W1['scalar'],columns=induc_W1cy_W1['charH'],index=induc_W1cy_W1['charW'])
        Induc_W1cy_W1_set.append(Induc_W1cy_W1)
        induc_W2cy_W2=inductiontable(W2_cy,W2,invchar=(lambda G:chartable(G, chars=False)['b']))
        Induc_W2cy_W2=pd.DataFrame(induc_W2cy_W2['scalar'],columns=induc_W2cy_W2['charH'],index=induc_W2cy_W2['charW'])
        Induc_W2cy_W2_set.append(Induc_W2cy_W2)
    for char in G2_Data.loc[:,'charnames']:
        j_induction_sign_set=[]
        for i in range(len(minimal_node_set_W1)):
            Phi1_cy=minimal_node_set_W1[i]
            Phi2_cy=minimal_node_set_W2[i]
            W1_cy=reflectionsubgroup(W1,Phi1_cy)
            W2_cy=reflectionsubgroup(W2,Phi2_cy)
            W1_cy_chartable=pd.DataFrame(chartable(W1_cy))
            W1_cy_special_rep=W1_cy_special_rep_set[i]
            Induc_W1cy_W1=Induc_W1cy_W1_set[i]
            Induc_W2cy_W2=Induc_W2cy_W2_set[i]
            for rep_now in W1_cy_special_rep:
                j_induction_now=dual_rep_G2((Induc_W1cy_W1[rep_now][Induc_W1cy_W1[rep_now]==1].index)[0])
                if list((G2_Data.loc[G2_Data['charnames']==str(j_induction_now)]).loc[:,'orbit'])[0] in list((G2_Data.loc[G2_Data['charnames']==char]).loc[:,'Georbit'])[0]:
                    rep_sign_position=(W1_cy_chartable.loc[W1_cy_chartable['charnames']==rep_now]).loc[:,'permsgn'].iloc[0]
                    rep_sign=W1_cy_chartable.iloc[rep_sign_position,0]
                    j_induction_sign=(Induc_W2cy_W2[rep_sign][Induc_W2cy_W2[rep_sign]==1].index)[0]
                    j_induction_sign_set.append(j_induction_sign)
        Target.append(max_orbit(list(set([Sat_dict.get(x) for x in set(j_induction_sign_set)])),G2_Data))
    G2_Data["G2_A2_"+str(cover_fold)]=Target

In [53]:
E6_Data.columns[4][-2]+E6_Data.columns[4][-1]

'_2'

In [56]:
E6_Data['E6_E6_1'][0]+E6_Data['E6_E6_1'][0]

['0', '0']

In [61]:
for i in range(3,len(E6_Data.columns)):
    columnname=E6_Data.columns[i]
    if columnname[-2]=='_':
        E6_column="E6_E6_"+columnname[-1]
    else:
        E6_column="E6_E6_"+columnname[-2]+columnname[-1]
    print(columnname)
    for j in range(len(E6_Data[columnname])):
        AIM=E6_Data[E6_column][j]
        X=max_orbit(list(set(E6_Data[E6_column][j]+E6_Data[columnname][j])),E6_Data)
        if X!=AIM:
            print("wow")

E6_E6_1
E6_E6_2
E6_E6_3
E6_E6_4
E6_E6_5
E6_E6_6
E6_E6_7
E6_E6_8
E6_E6_9
E6_E6_10
E6_E6_11
E6_E6_12
E6_A2*A2*A2_1
E6_A2*A2*A2_2
E6_A2*A2*A2_3
E6_A2*A2*A2_4
E6_A1*A5_1
E6_A1*A5_2
E6_A1*A5_3
E6_A1*A5_4
E6_A1*A5_5
E6_A1*A5_6


In [150]:
for i in range(3,len(E7_Data.columns)):
    columnname=E7_Data.columns[i]
    if columnname[-2]=='_':
        E7_column="E7_E7_"+columnname[-1]
    else:
        E7_column="E7_E7_"+columnname[-2]+columnname[-1]
    for j in range(len(E7_Data[columnname])):
        AIM=E7_Data[E7_column][j]
        X=max_orbit(list(set(E7_Data[E7_column][j]+E7_Data[columnname][j])),E7_Data)
        if X!=AIM:
            print(j)
            print(columnname)
            print(X)

In [147]:
E7_Data[['charnames','orbit','E7_E7_2','E7_A2A5_2']]

,charnames,orbit,E7_E7_2,E7_A2A5_2
0,"('1_a',)",E7,[4A1],[4A1]
1,"(""7_a'"",)",E7(a1),[A2+3A1],[A2+3A1]
2,"('27_a',)",E7(a2),[A3+2A1],[A3+2A1]
3,"(""56_a'"",)",E7(a3),[A3+A2+A1],[A3+A2+A1]
4,"(""21_b'"",)",E6,[A3+2A1],[A3+2A1]
5,"('35_b',)",D6,[D4+A1],[A3+A2+A1]
6,"('120_a',)",E6(a1),[A4+A1],[A4+A1]
7,"(""189_b'"",)",E7(a4),[D5(a1)+A1],[A4+A2]
8,"('210_a',)",D6(a1),[D5(a1)+A1],[A4+A2]
9,"('105_b',)",A6,[D5(a1)+A1],[A4+A2]


In [144]:
Induc_W2cy_W2_set[2]

,"([1, 1], [1, 1], [1, 1], [1, 1])","([1, 1], [1, 1], [1, 1], [2])","([1, 1], [1, 1], [2], [1, 1])","([1, 1], [1, 1], [2], [2])","([1, 1], [2], [1, 1], [1, 1])","([1, 1], [2], [1, 1], [2])","([1, 1], [2], [2], [1, 1])","([1, 1], [2], [2], [2])","([2], [1, 1], [1, 1], [1, 1])","([2], [1, 1], [1, 1], [2])","([2], [1, 1], [2], [1, 1])","([2], [1, 1], [2], [2])","([2], [2], [1, 1], [1, 1])","([2], [2], [1, 1], [2])","([2], [2], [2], [1, 1])","([2], [2], [2], [2])"
"([1, 1, 1, 1, 1, 1], [1, 1, 1])",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"([1, 1, 1, 1, 1, 1], [2, 1])",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"([1, 1, 1, 1, 1, 1], [3])",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"([2, 1, 1, 1, 1], [1, 1, 1])",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"([2, 1, 1, 1, 1], [2, 1])",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"([2, 1, 1, 1, 1], [3])",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"([2, 2, 1, 1], [1, 1, 1])",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"([2, 2, 1, 1], [2, 1])",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"([2, 2, 1, 1], [3])",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"([2, 2, 2], [1, 1, 1])",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [145]:
minimal_node_set_W2[2]

[2, 4, 6, 17]

In [149]:
# Construct W of type E7 and Wc of type A2A5 with a cartan matrix
W1=coxeter("E",7)
W2=reflectionsubgroup(W1,[0,1,3,4,5,6,125])
induc_W2_W1=inductiontable(W2,W1,invchar=(lambda G:chartable(G, chars=False)['b']))
Induc_W2_W1=pd.DataFrame(induc_W2_W1['scalar'],columns=induc_W2_W1['charH'],index=induc_W2_W1['charW'])

# Record the saturation dictionary
Sat_dict={('[1, 1, 1, 1, 1, 1]', '[1, 1, 1]'):'1',('[1, 1, 1, 1, 1, 1]', '[2, 1]'):'A1',('[1, 1, 1, 1, 1, 1]', '[3]'):'A2',('[2, 1, 1, 1, 1]', '[1, 1, 1]'):'A1', ('[2, 1, 1, 1, 1]', '[2, 1]'):'2A1',('[2, 1, 1, 1, 1]', '[3]'):'A2+A1',
         ('[2, 2, 1, 1]', '[1, 1, 1]'):'2A1',('[2, 2, 1, 1]', '[2, 1]'):"3A1'",('[2, 2, 1, 1]', '[3]'):'A2+2A1',('[2, 2, 2]', '[1, 1, 1]'):"3A1''",('[2, 2, 2]', '[2, 1]'):'4A1', ('[2, 2, 2]', '[3]'):'A2+3A1',
         ('[3, 1, 1, 1]', '[1, 1, 1]'):'A2',('[3, 1, 1, 1]', '[2, 1]'):'A2+A1',('[3, 1, 1, 1]', '[3]'):'2A2',('[3, 2, 1]', '[1, 1, 1]'):'A2+A1',('[3, 2, 1]', '[2, 1]'):'A2+2A1',('[3, 2, 1]', '[3]'):'2A2+A1',
         ('[3, 3]', '[1, 1, 1]'):'2A2',('[3, 3]', '[2, 1]'):'2A2+A1',('[3, 3]', '[3]'):'D4(a1)',('[4, 1, 1]', '[1, 1, 1]'):'A3',('[4, 1, 1]', '[2, 1]'):"(A3+A1)'",('[4, 1, 1]', '[3]'):'A3+A2',
         ('[4, 2]', '[1, 1, 1]'):"(A3+A1)''",('[4, 2]', '[2, 1]'):'A3+2A1',('[4, 2]', '[3]'):'A3+A2+A1',('[5, 1]', '[1, 1, 1]'):'A4',('[5, 1]', '[2, 1]'):'A4+A1',('[5, 1]', '[3]'):'A4+A2',
         ('[6]', '[1, 1, 1]'):"A5''",('[6]', '[2, 1]'):'A5+A1',('[6]', '[3]'):'E7(a5)'}


weight=[2,2,4,3,2,1]
drop_weight=3
node_set_1=[list(p) for p in itertools.product(range(drop_weight+1), repeat=len(weight))]
node_set_2=[]
for node in node_set_1:
    c=(-np.sum(np.array(node)*weight)-1)%drop_weight
    node_set_2.append(np.append(node,c))
    if c==0:
        node_set_2.append(np.append(node,drop_weight))

for cover_fold in range(1,7):
    node_set_3=[]
    for node in node_set_2:
        if node[0]+node[6]<=cover_fold and node[1]+node[2]+node[3]+node[4]+node[5]<=cover_fold:
            add_point=[]
            for i in range(7):
                if node[i]==0:
                    add_point.append(i)
            if node[0]+node[6]==cover_fold:
                add_point.append(7)
            if node[1]+node[2]+node[3]+node[4]+node[5]==cover_fold:
                add_point.append(17)
            node_set_3.append(add_point)
    
    node_set_3=[list(tpl) for tpl in {tuple(item) for item in node_set_3}]
    minimal_node_set_W2=minimal_W_cy(node_set_3)
    
    Target=[]
    Induc_W2cy_W2_set=[]
    W2_cy_special_rep_set=[]
    for i in range(len(minimal_node_set_W2)):
        Phi2_cy=minimal_node_set_W2[i]
        W2_cy=reflectionsubgroup(W2,Phi2_cy)
        W2_cy_chartable=pd.DataFrame(chartable(W2_cy))
        W2_cy_special_rep=W2_cy_chartable.loc[W2_cy_chartable['b']==W2_cy_chartable['a']]['charnames']
        W2_cy_special_rep_set.append(W2_cy_special_rep)
        induc_W2cy_W2=inductiontable(W2_cy,W2,invchar=(lambda G:chartable(G, chars=False)['b']))
        Induc_W2cy_W2=pd.DataFrame(induc_W2cy_W2['scalar'],columns=induc_W2cy_W2['charH'],index=induc_W2cy_W2['charW'])
        Induc_W2cy_W2_set.append(Induc_W2cy_W2)
    for char in E7_Data.loc[:,'charnames']:
        j_induction_sign_set=[]
        for i in range(len(minimal_node_set_W2)):
            Phi2_cy=minimal_node_set_W2[i]
            W2_cy=reflectionsubgroup(W2,Phi2_cy)
            W2_cy_chartable=pd.DataFrame(chartable(W2_cy))
            W2_cy_special_rep=W2_cy_special_rep_set[i]
            Induc_W2cy_W2=Induc_W2cy_W2_set[i]
            for rep_now in W2_cy_special_rep:
                j_induction_now=(Induc_W2_W1[(Induc_W2cy_W2[rep_now][Induc_W2cy_W2[rep_now]==1].index)[0]][Induc_W2_W1[(Induc_W2cy_W2[rep_now][Induc_W2cy_W2[rep_now]==1].index)[0]]==1].index)[0]
                if list((E7_Data.loc[E7_Data['charnames']==str(j_induction_now)]).loc[:,'orbit'])[0] in list((E7_Data.loc[E7_Data['charnames']==char]).loc[:,'Georbit'])[0]:
                    rep_sign_position=(W2_cy_chartable.loc[W2_cy_chartable['charnames']==rep_now]).loc[:,'permsgn'].iloc[0]
                    rep_sign=W2_cy_chartable.iloc[rep_sign_position,0]
                    j_induction_sign=(Induc_W2cy_W2[rep_sign][Induc_W2cy_W2[rep_sign]==1].index)[0]
                    j_induction_sign_set.append(j_induction_sign)
        Target.append(max_orbit(list(set([Sat_dict.get(x) for x in set(j_induction_sign_set)])),E7_Data))
    E7_Data["E7_A2A5_"+str(cover_fold)]=Target

1
2
3
4
5
6


In [66]:
for i in range(3,len(E8_Data.columns)):
    columnname=E8_Data.columns[i]
    if columnname[-2]=='_':
        E8_column="E8_E8_"+columnname[-1]
    else:
        E8_column="E8_E8_"+columnname[-2]+columnname[-1]
    print(columnname)
    for j in range(len(E8_Data[columnname])):
        AIM=E8_Data[E8_column][j]
        X=max_orbit(list(set(E8_Data[E8_column][j]+E8_Data[columnname][j])),E8_Data)
        if X!=AIM:
            print(columnname)
            print(X)

E8_E8_1
E8_E8_2
E8_E8_3
E8_E8_4
E8_E8_5
E8_E8_6
E8_E8_7
E8_E8_8
E8_E8_9
E8_E8_10
E8_E8_11
E8_E8_12
E8_E8_13
E8_E8_14
E8_E8_15
E8_E8_16
E8_E8_17
E8_E8_18
E8_E8_19
E8_E8_20
E8_E8_21
E8_E8_22
E8_E8_23
E8_E8_24
E8_E8_25
E8_E8_26
E8_E8_27
E8_E8_28
E8_E8_29
E8_D8_1
E8_D8_2
E8_D8_2
['4A1', 'A2']
E8_D8_2
['A4+A1']
E8_D8_2
['A4+A1']
E8_D8_2
['A4+A1', 'D4+A1']
E8_D8_2
['D6(a2)', 'E6(a3)']
E8_D8_2
['D6(a2)', 'E6(a3)']
E8_D8_3
E8_D8_3
['D6(a2)', 'E6(a3)+A1']
E8_D8_3
['E8(b6)', 'D6']
E8_D8_3
['D6', 'E6+A1']
E8_D8_3
['D7']
E8_D8_3
['D6', 'E6+A1']
E8_D8_4
E8_D8_4
['A4+A1', '2A3']
E8_D8_5
E8_D8_5
['E8(b6)', 'D6']
E8_D8_6
E8_D8_7
E8_D8_8
E8_D8_8
['E6(a1)', 'A7']
E8_D8_9
E8_D8_10
E8_D8_11
E8_D8_12
E8_D8_13
E8_D8_14
E8_D8_15
E8_A1+A7_1
E8_A1+A7_2
E8_A1+A7_3
E8_A1+A7_4
E8_A1+A7_5
E8_A1+A7_6
E8_A1+A7_7
E8_A1+A7_8
E8_A1+A7_9
E8_A2+A1+A5_1
E8_A2+A1+A5_2
E8_A2+A1+A5_3
E8_A2+A1+A5_4
E8_A2+A1+A5_5
E8_A2+A1+A5_6
E8_A8_1
E8_A8_2
E8_A8_3
E8_A8_4
E8_A8_5
E8_A8_6
E8_A8_7
E8_A8_8
E8_A8_9
E8_A8_10
E8_2A4_1
E8_2A4_2
E8

In [97]:
W2=coxeter([[2, 0, -1, 0, 0, 0,0], [0, 2, -1, 0, 0, 0,0],[-1, -1, 2, -1, 0, 0,0],[0, 0, -1, 2, -1, 0,0],[0, 0, 0, -1, 2, -1,0], [0, 0, 0, 0, -1, 2,0],[0,0,0,0,0,0,2]])
chartable(W2)['charnames']

[('[[1, 1, 1], +]', '[1, 1]'),
 ('[[1, 1, 1], +]', '[2]'),
 ('[[1, 1, 1], -]', '[1, 1]'),
 ('[[1, 1, 1], -]', '[2]'),
 ('[[1, 1], [1, 1, 1, 1]]', '[1, 1]'),
 ('[[1, 1], [1, 1, 1, 1]]', '[2]'),
 ('[[1], [1, 1, 1, 1, 1]]', '[1, 1]'),
 ('[[1], [1, 1, 1, 1, 1]]', '[2]'),
 ('[[], [1, 1, 1, 1, 1, 1]]', '[1, 1]'),
 ('[[], [1, 1, 1, 1, 1, 1]]', '[2]'),
 ('[[1, 1, 1, 1], [2]]', '[1, 1]'),
 ('[[1, 1, 1, 1], [2]]', '[2]'),
 ('[[1, 1, 1], [2, 1]]', '[1, 1]'),
 ('[[1, 1, 1], [2, 1]]', '[2]'),
 ('[[1, 1], [2, 1, 1]]', '[1, 1]'),
 ('[[1, 1], [2, 1, 1]]', '[2]'),
 ('[[1], [2, 1, 1, 1]]', '[1, 1]'),
 ('[[1], [2, 1, 1, 1]]', '[2]'),
 ('[[], [2, 1, 1, 1, 1]]', '[1, 1]'),
 ('[[], [2, 1, 1, 1, 1]]', '[2]'),
 ('[[1, 1], [2, 2]]', '[1, 1]'),
 ('[[1, 1], [2, 2]]', '[2]'),
 ('[[2, 1], +]', '[1, 1]'),
 ('[[2, 1], +]', '[2]'),
 ('[[2, 1], -]', '[1, 1]'),
 ('[[2, 1], -]', '[2]'),
 ('[[1], [2, 2, 1]]', '[1, 1]'),
 ('[[1], [2, 2, 1]]', '[2]'),
 ('[[2], [2, 1, 1]]', '[1, 1]'),
 ('[[2], [2, 1, 1]]', '[2]'),
 ('[[], [

In [118]:
W1=coxeter("E",7)
W2=reflectionsubgroup(W1,[0,1])
H=reflectionsubgroup(W2,[0])

In [122]:
inductiontable(H,W2,invchar=(lambda G:chartable(G, chars=False)['b']))

{'charH': [('[1, 1]',), ('[2]',)],
 'charW': [('[1, 1]', '[1, 1]'),
  ('[1, 1]', '[2]'),
  ('[2]', '[1, 1]'),
  ('[2]', '[2]')],
 'scalar': [[0, 0], [1, 0], [0, 0], [0, 1]]}

In [117]:
W2.roots

[(1, 0), (0, 1), (-1, 0), (0, -1)]

In [104]:
# Construct W of type E7 and Wc of type A1D6 with a cartan matrix
W1=coxeter("E",7)
W2=coxeter([[2, 0, -1, 0, 0, 0,0], [0, 2, -1, 0, 0, 0,0],[-1, -1, 2, -1, 0, 0,0],[0, 0, -1, 2, -1, 0,0],[0, 0, 0, -1, 2, -1,0], [0, 0, 0, 0, -1, 2,0],[0,0,0,0,0,0,2]])

# Record the saturation dictionary
Sat_dict={ ('[[], [1, 1, 1, 1, 1, 1]]', '[1, 1]'):'1',('[[], [1, 1, 1, 1, 1, 1]]', '[2]'):'A1',('[[1], [1, 1, 1, 1, 1]]', '[1, 1]'):'A1',('[[1], [1, 1, 1, 1, 1]]', '[2]'):'2A1',
        ('[[], [2, 1, 1, 1, 1]]', '[1, 1]'):'2A1',('[[], [2, 1, 1, 1, 1]]', '[2]'):"3A1''",('[[1, 1], [1, 1, 1, 1]]', '[1, 1]'):'2A1', ('[[1, 1, 1], -]', '[1, 1]'):"3A1''",
         ('[[1, 1, 1], +]', '[1, 1]'):"3A1'",('[[1, 1], [1, 1, 1, 1]]', '[2]'):"3A1'",('[[1, 1, 1], -]', '[2]'):'4A1',('[[1, 1, 1], +]', '[2]'):'A2',
        ('[[], [2, 2, 1, 1]]', '[1, 1]'):"3A1'",('[[], [2, 2, 1, 1]]', '[2]'):'4A1', ('[[1], [2, 1, 1, 1]]', '[1, 1]'):'A2',('[[], [2, 2, 2]]', '[1, 1]'):'4A1',
         ('[[1], [2, 1, 1, 1]]', '[2]'):'A2+A1',('[[], [3, 1, 1, 1]]', '[1, 1]'):'A3',('[[], [2, 2, 2]]', '[2]'):'A2+A1',('[[], [3, 1, 1, 1]]', '[2]'):"(A3+A1)''",
        ('[[1, 1], [2, 1, 1]]', '[1, 1]'):'A2+A1',('[[1, 1], [2, 1, 1]]', '[2]'):'A2+2A1', ('[[1], [2, 2, 1]]', '[1, 1]'):'A2+2A1', ('[[1, 1], [2, 2]]', '[1, 1]'):'2A2',
         ('[[], [3, 2, 1]]', '[1, 1]'):"(A3+A1)'",('[[1], [2, 2, 1]]', '[2]'):'A2+3A1',('[[2], [2, 1, 1]]', '[1, 1]'):'A3',('[[2, 1], -]', '[1, 1]'):"(A3+A1)''",
         ('[[2, 1], +]', '[1, 1]'):"(A3+A1)'",('[[1, 1], [2, 2]]', '[2]'):"2A2+A1",('[[], [3, 2, 1]]', '[2]'):"A3+2A1",('[[2], [2, 1, 1]]', '[2]'):"(A3+A1)'",
         ('[[1], [3, 1, 1]]', '[1, 1]'):'D4(a1)',('[[2, 1], -]', '[2]'):'A3+2A1',('[[2, 1], +]', '[2]'):'D4(a1)',('[[1, 1], [3, 1]]', '[1, 1]'):'D4(a1)+A1',
         ('[[2], [2, 2]]', '[1, 1]'):'A3+2A1',('[[1], [3, 1, 1]]', '[2]'):'D4(a1)+A1',('[[], [4, 1, 1]]', '[1, 1]'):'D4',('[[1, 1], [3, 1]]', '[2]'):'A3+A2',
         ('[[2], [2, 2]]', '[2]'):'D4(a1)+A1',('[[1], [3, 2]]', '[1, 1]'):'A3+A2',('[[], [4, 1, 1]]', '[2]'):'D4+A1',('[[], [4, 2]]', '[1, 1]'):'D4+A1',
         ('[[1], [3, 2]]', '[2]'):'A3+A2+A1',('[[2], [3, 1]]', '[1, 1]'):'A4',('[[3], -]', '[1, 1]'):"A5''",('[[3], +]', '[1, 1]'):"A5'",
          ('[[2], [4]]', '[1, 1]'):'D6(a2)',('[[1], [4, 1]]', '[2]'):'D5(a1)+A1', ('[[], [5, 1]]', '[1, 1]'):'D5',('[[2], [4]]', '[2]'):'E7(a5)',
         ('[[], [4, 2]]', '[2]'):'D4+A1',('[[2], [3, 1]]', '[2]'):'A3+A2+A1',('[[1], [4, 1]]', '[1, 1]'):'A4',('[[3], -]', '[2]'):"A5+A1",
         ('[[3], +]', '[2]'):"E6(a3)",('[[1], [5]]', '[1, 1]'):'D6(a1)',('[[], [5, 1]]', '[2]'):'D5+A1',('[[1], [5]]', '[2]'):'E7(a4)',
         ('[[], [6]]', '[1, 1]'):'D6',('[[], [6]]', '[2]'):'E7(a3)'}

weight=[2,3,4,3,2,1]
drop_weight=2
node_set_1=[list(p) for p in itertools.product(range(drop_weight+1), repeat=len(weight))]
node_set_2=[]
for node in node_set_1:
    c=(-np.sum(np.array(node)*weight)-1)%drop_weight
    node_set_2.append(np.append(node,c))
    if c==0:
        node_set_2.append(np.append(node,drop_weight))

for cover_fold in range(1,11):
    node_set_3=[]
    for node in node_set_2:
        if node[0]+node[1]+2*node[2]+2*node[3]+2*node[4]+node[5]<=cover_fold and node[6]<=cover_fold:
            add_point=[]
            for i in range(7):
                if node[i]==0:
                    add_point.append(i)
            if node[0]+node[1]+2*node[2]+2*node[3]+2*node[4]+node[5]==cover_fold:
                add_point.append(30)
            if node[6]==cover_fold:
                add_point.append(6)
            node_set_3.append(add_point)
    
    node_set_3=[list(tpl) for tpl in {tuple(item) for item in node_set_3}]
    minimal_node_set_W2=minimal_W_cy(node_set_3)
    
    dict={0:1,1:2,2:3,3:4,4:5,5:6,6:125}
    minimal_node_set_W1=[]
    for node in minimal_node_set_W2:
        minimal_node_set_W1.append([convert_fuc(W1.roots,W2.roots,Index,dict) for Index in node])

    Target=[]
    Induc_W1cy_W1_set=[]
    Induc_W2cy_W2_set=[]
    W1_cy_special_rep_set=[]
    for i in range(len(minimal_node_set_W1)):
        Phi1_cy=minimal_node_set_W1[i]
        Phi2_cy=minimal_node_set_W2[i]
        W1_cy=reflectionsubgroup(W1,Phi1_cy)
        W2_cy=reflectionsubgroup(W2,Phi2_cy)
        W1_cy_chartable=pd.DataFrame(chartable(W1_cy))
        W1_cy_special_rep=W1_cy_chartable.loc[W1_cy_chartable['b']==W1_cy_chartable['a']]['charnames']
        W1_cy_special_rep_set.append(W1_cy_special_rep)
        induc_W1cy_W1=inductiontable(W1_cy,W1,invchar=(lambda G:chartable(G, chars=False)['b']))
        Induc_W1cy_W1=pd.DataFrame(induc_W1cy_W1['scalar'],columns=induc_W1cy_W1['charH'],index=induc_W1cy_W1['charW'])
        Induc_W1cy_W1_set.append(Induc_W1cy_W1)
        induc_W2cy_W2=inductiontable(W2_cy,W2,invchar=(lambda G:chartable(G, chars=False)['b']))
        Induc_W2cy_W2=pd.DataFrame(induc_W2cy_W2['scalar'],columns=induc_W2cy_W2['charH'],index=induc_W2cy_W2['charW'])
        Induc_W2cy_W2_set.append(Induc_W2cy_W2)
    for char in E7_Data.loc[:,'charnames']:
        print(char)
        j_induction_sign_set=[]
        for i in range(len(minimal_node_set_W1)):
            Phi1_cy=minimal_node_set_W1[i]
            Phi2_cy=minimal_node_set_W2[i]
            W1_cy=reflectionsubgroup(W1,Phi1_cy)
            W2_cy=reflectionsubgroup(W2,Phi2_cy)
            W1_cy_chartable=pd.DataFrame(chartable(W1_cy))
            W1_cy_special_rep=W1_cy_special_rep_set[i]
            Induc_W1cy_W1=Induc_W1cy_W1_set[i]
            Induc_W2cy_W2=Induc_W2cy_W2_set[i]
            for rep_now in W1_cy_special_rep:
                j_induction_now=(Induc_W1cy_W1[rep_now][Induc_W1cy_W1[rep_now]==1].index)[0]
                if list((E7_Data.loc[E7_Data['charnames']==str(j_induction_now)]).loc[:,'orbit'])[0] in list((E7_Data.loc[E7_Data['charnames']==char]).loc[:,'Georbit'])[0]:
                    rep_sign_position=(W1_cy_chartable.loc[W1_cy_chartable['charnames']==rep_now]).loc[:,'permsgn'].iloc[0]
                    rep_sign=W1_cy_chartable.iloc[rep_sign_position,0]
                    j_induction_sign=(Induc_W2cy_W2[rep_sign][Induc_W2cy_W2[rep_sign]==1].index)[0]
                    j_induction_sign_set.append(j_induction_sign)
        Target.append(max_orbit(list(set([Sat_dict.get(x) for x in set(j_induction_sign_set)])),E7_Data))
    E7_Data["E7_A1D6_"+str(cover_fold)]=Target

('1_a',)
("7_a'",)
('27_a',)
("56_a'",)
("21_b'",)
('35_b',)
('120_a',)
("189_b'",)
('210_a',)
('105_b',)
('168_a',)
("315_a'",)
("189_c'",)
('280_b',)
('405_a',)
("70_a'",)
("216_a'",)
("378_a'",)
('210_b',)
('420_a',)
("512_a'",)
('105_c',)
('84_a',)
("420_a'",)
("210_b'",)
('378_a',)
("405_a'",)
("105_c'",)
('315_a',)
('216_a',)
("280_b'",)
('70_a',)
('189_c',)
("210_a'",)
("168_a'",)
("105_b'",)
('189_b',)
("120_a'",)
('15_a',)
('56_a',)
("35_b'",)
('21_b',)
("27_a'",)
('7_a',)
("1_a'",)
('1_a',)
("7_a'",)
('27_a',)
("56_a'",)
("21_b'",)
('35_b',)
('120_a',)
("189_b'",)
('210_a',)
('105_b',)
('168_a',)
("315_a'",)
("189_c'",)
('280_b',)
('405_a',)
("70_a'",)
("216_a'",)
("378_a'",)
('210_b',)
('420_a',)
("512_a'",)
('105_c',)
('84_a',)
("420_a'",)
("210_b'",)
('378_a',)
("405_a'",)
("105_c'",)
('315_a',)
('216_a',)
("280_b'",)
('70_a',)
('189_c',)
("210_a'",)
("168_a'",)
("105_b'",)
('189_b',)
("120_a'",)
('15_a',)
('56_a',)
("35_b'",)
('21_b',)
("27_a'",)
('7_a',)
("1_a'",)
('1_a'

In [90]:
Phi1_cy

[1, 2, 3, 4, 5, 6, 125]

In [70]:
inductiontable(W1_cy,W1,invchar=(lambda G:chartable(G, chars=False)['b']))

{'charH': [('[[1, 1, 1], +]', '[1, 1]'),
  ('[[1, 1, 1], +]', '[2]'),
  ('[[1, 1, 1], -]', '[1, 1]'),
  ('[[1, 1, 1], -]', '[2]'),
  ('[[1, 1], [1, 1, 1, 1]]', '[1, 1]'),
  ('[[1, 1], [1, 1, 1, 1]]', '[2]'),
  ('[[1], [1, 1, 1, 1, 1]]', '[1, 1]'),
  ('[[1], [1, 1, 1, 1, 1]]', '[2]'),
  ('[[], [1, 1, 1, 1, 1, 1]]', '[1, 1]'),
  ('[[], [1, 1, 1, 1, 1, 1]]', '[2]'),
  ('[[1, 1, 1, 1], [2]]', '[1, 1]'),
  ('[[1, 1, 1, 1], [2]]', '[2]'),
  ('[[1, 1, 1], [2, 1]]', '[1, 1]'),
  ('[[1, 1, 1], [2, 1]]', '[2]'),
  ('[[1, 1], [2, 1, 1]]', '[1, 1]'),
  ('[[1, 1], [2, 1, 1]]', '[2]'),
  ('[[1], [2, 1, 1, 1]]', '[1, 1]'),
  ('[[1], [2, 1, 1, 1]]', '[2]'),
  ('[[], [2, 1, 1, 1, 1]]', '[1, 1]'),
  ('[[], [2, 1, 1, 1, 1]]', '[2]'),
  ('[[1, 1], [2, 2]]', '[1, 1]'),
  ('[[1, 1], [2, 2]]', '[2]'),
  ('[[2, 1], +]', '[1, 1]'),
  ('[[2, 1], +]', '[2]'),
  ('[[2, 1], -]', '[1, 1]'),
  ('[[2, 1], -]', '[2]'),
  ('[[1], [2, 2, 1]]', '[1, 1]'),
  ('[[1], [2, 2, 1]]', '[2]'),
  ('[[2], [2, 1, 1]]', '[1, 1]'),
 

In [71]:
pd.DataFrame(induc_W1cy_W1['scalar'],columns=induc_W1cy_W1['charH'],index=induc_W1cy_W1['charW'])

,"([[1, 1, 1], +], [1, 1])","([[1, 1, 1], +], [2])","([[1, 1, 1], -], [1, 1])","([[1, 1, 1], -], [2])","([[1, 1], [1, 1, 1, 1]], [1, 1])","([[1, 1], [1, 1, 1, 1]], [2])","([[1], [1, 1, 1, 1, 1]], [1, 1])","([[1], [1, 1, 1, 1, 1]], [2])","([[], [1, 1, 1, 1, 1, 1]], [1, 1])","([[], [1, 1, 1, 1, 1, 1]], [2])",...,"([[2], [4]], [1, 1])","([[2], [4]], [2])","([[], [4, 2]], [1, 1])","([[], [4, 2]], [2])","([[1], [5]], [1, 1])","([[1], [5]], [2])","([[], [5, 1]], [1, 1])","([[], [5, 1]], [2])","([[], [6]], [1, 1])","([[], [6]], [2])"
"(1_a,)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
"(1_a',)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"(7_a,)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"(7_a',)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
"(15_a,)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"(15_a',)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"(21_a,)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"(21_a',)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"(21_b,)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"(21_b',)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [114]:
W=coxeter("E",7)
H=reflectionsubgroup(W,[3, 4, 5, 125, 29])
induc_W1cy_W1=inductiontable(H,W,invchar=(lambda G:chartable(G, chars=False)['b']))
pd.DataFrame(induc_W1cy_W1['scalar'],columns=induc_W1cy_W1['charH'],index=induc_W1cy_W1['charW'])[('[2, 2]', '[2]', '[1, 1]')]

(1_a,)       0
(1_a',)      0
(7_a,)       0
(7_a',)      0
(15_a,)      0
(15_a',)     0
(21_a,)      0
(21_a',)     0
(21_b,)      0
(21_b',)     0
(27_a,)      0
(27_a',)     0
(35_a,)      0
(35_a',)     0
(35_b,)      0
(35_b',)     0
(56_a,)      0
(56_a',)     1
(70_a,)      0
(70_a',)     0
(84_a,)      0
(84_a',)     0
(105_a,)     0
(105_a',)    0
(105_b,)     0
(105_b',)    0
(105_c,)     0
(105_c',)    0
(120_a,)     0
(120_a',)    0
(168_a,)     0
(168_a',)    0
(189_a,)     0
(189_a',)    0
(189_b,)     0
(189_b',)    0
(189_c,)     0
(189_c',)    0
(210_a,)     0
(210_a',)    0
(210_b,)     0
(210_b',)    0
(216_a,)     0
(216_a',)    0
(280_a,)     0
(280_a',)    0
(280_b,)     0
(280_b',)    0
(315_a,)     0
(315_a',)    0
(336_a,)     0
(336_a',)    0
(378_a,)     0
(378_a',)    0
(405_a,)     0
(405_a',)    0
(420_a,)     0
(420_a',)    0
(512_a,)     0
(512_a',)    0
Name: ([2, 2], [2], [1, 1]), dtype: int64

In [77]:
pd.DataFrame(chartable(W1_cy)).iloc[55,:]

charnames                                          ([[3], -], [2])
classnames                                  ([[4, 1, 1], []], [2])
classlengths                                                   720
classreps                                             [0, 2, 3, 6]
b                                                                3
a                                                                3
irreducibles     [10, 10, -2, -2, 2, 2, -10, -10, 4, 4, -2, -2,...
position_id                                                     73
position_sgn                                                     8
position_refl                                                False
permsgn                                                          0
Name: 55, dtype: object

In [78]:
W1_cy=coxeter("D",6)

In [79]:
pd.DataFrame(chartable(W1_cy))

,charnames,classnames,classlengths,classreps,b,a,irreducibles,position_id,position_sgn,position_refl,permsgn
0,"([[1, 1, 1], +],)","([[1, 1, 1, 1, 1, 1], []],)",1,[],15,15,"[10, -2, 2, -10, -4, 2, 0, -2, 4, 2, -2, 0, -2...",36,4,34,27
1,"([[1, 1, 1], -],)","([[1, 1, 1, 1], [1, 1]],)",15,"[0, 1]",15,15,"[10, -2, 2, -10, -4, 2, 0, -2, 4, 2, -2, 0, -2...",36,4,34,26
2,"([[1, 1], [1, 1, 1, 1]],)","([[1, 1], [1, 1, 1, 1]],)",15,"[0, 1, 2, 0, 1, 2, 3, 2, 0, 1, 2, 3]",16,16,"[15, -1, -1, 15, -7, 1, 1, 1, -7, 3, -1, -1, 3...",36,4,34,32
3,"([[1], [1, 1, 1, 1, 1]],)","([[], [1, 1, 1, 1, 1, 1]],)",1,"[0, 1, 2, 0, 1, 2, 3, 2, 0, 1, 2, 3, 4, 3, 2, ...",21,21,"[6, 2, -2, -6, -4, -2, 0, 2, 4, 2, 2, 0, -2, -...",36,4,34,34
4,"([[], [1, 1, 1, 1, 1, 1]],)","([[2, 1, 1, 1, 1], []],)",30,[0],30,30,"[1, 1, 1, 1, -1, -1, -1, -1, -1, 1, 1, 1, 1, 1...",36,4,34,36
5,"([[1, 1, 1, 1], [2]],)","([[1, 1, 1], [2, 1]],)",120,"[0, 1, 2]",14,13,"[15, -1, -1, 15, -5, -1, 3, -1, -5, -1, 3, -1,...",36,4,34,29
6,"([[1, 1, 1], [2, 1]],)","([[2, 1, 1], [1, 1]],)",180,"[0, 1, 3]",11,10,"[40, -8, 8, -40, -8, 4, 0, -4, 8, 0, 0, 0, 0, ...",36,4,34,22
7,"([[1, 1], [2, 1, 1]],)","([[1], [2, 1, 1, 1]],)",120,"[0, 1, 2, 0, 1, 2, 3, 2, 0, 1, 2, 3, 4]",10,10,"[45, -3, -3, 45, -9, 3, -1, 3, -9, 1, -3, 1, 1...",36,4,34,24
8,"([[1], [2, 1, 1, 1]],)","([[2], [1, 1, 1, 1]],)",30,"[0, 1, 2, 0, 1, 2, 3, 2, 0, 1, 2, 3, 5]",13,13,"[24, 8, -8, -24, -8, -4, 0, 4, 8, 0, 0, 0, 0, ...",36,4,34,30
9,"([[], [2, 1, 1, 1, 1]],)","([[2, 2, 1, 1], []],)",180,"[0, 3]",20,20,"[5, 5, 5, 5, -3, -3, -3, -3, -3, 1, 1, 1, 1, 1...",36,4,34,35


In [80]:
for i in range(3,len(F4_Data.columns)):
    columnname=F4_Data.columns[i]
    if columnname[-2]=='_':
        F4_column="F4_F4_"+columnname[-1]
    else:
        F4_column="F4_F4_"+columnname[-2]+columnname[-1]
    print(columnname)
    for j in range(len(F4_Data[columnname])):
        AIM=F4_Data[F4_column][j]
        X=max_orbit(list(set(F4_Data[F4_column][j]+F4_Data[columnname][j])),F4_Data)
        if X!=AIM:
            print(columnname)
            print(X)

F4_F4_1
F4_F4_3
F4_F4_5
F4_F4_7
F4_F4_9
F4_F4_11
F4_F4_13
F4_F4_2
F4_F4_4
F4_F4_6
F4_F4_8
F4_F4_10
F4_F4_12
F4_F4_14
F4_F4_16
F4_F4_18
F4_A1+C3_1
F4_A1+C3_3
F4_A1+C3_5
F4_A1+C3_7
F4_A1+C3_2
F4_A1+C3_4
F4_A1+C3_6
F4_A1+C3_8
F4_A1+C3_10
F4_A2+~A2_1
F4_A2+~A2_1
['A2', '~A2']
F4_A2+~A2_1
['~A2', 'A2']
F4_A2+~A2_3
F4_A2+~A2_5
F4_A2+~A2_2
F4_A2+~A2_4
F4_A2+~A2_6
F4_A1+A3_1
F4_A1+A3_3
F4_A1+A3_5
F4_A1+A3_2
F4_A1+A3_4
F4_A1+A3_6
F4_B4_1
F4_B4_3
F4_B4_5
F4_B4_7
F4_B4_9
F4_B4_2
F4_B4_4
F4_B4_6
F4_B4_8
F4_B4_10
F4_B4_12


In [89]:
F4_Data[['F4_F4_1','F4_A2+~A2_1']]

,F4_F4_1,F4_A2+~A2_1
0,[1],[1]
1,[~A1],[~A1]
2,[A1+~A1],[A1+~A1]
3,[~A2],[A2]
4,[A2],[~A2]
5,[F4(a3)],[~A2+A1]
6,[F4(a3)],[~A2+A1]
7,[F4(a3)],[~A2+A1]
8,[F4(a3)],[F4(a3)]
9,[F4(a3)],[F4(a3)]


In [ ]:
# Construct W of type F4 and Wc of type A2+~A2
W1=coxeter("F",4)
W2=coxeter([[2, -1, 0,0], [-1, 2, 0,0], [0, 0, 2,-1],[0,0,-1,2]])
Sat_dict={('[1, 1, 1]', '[1, 1, 1]'):'1',
 ('[1, 1, 1]', '[2, 1]'):'A1',
 ('[1, 1, 1]', '[3]'):'A2',
 ('[2, 1]', '[1, 1, 1]'):'~A1',
 ('[2, 1]', '[2, 1]'):'A1+~A1',
 ('[2, 1]', '[3]'):'A2+~A1',
 ('[3]', '[1, 1, 1]'):'~A2',
 ('[3]', '[2, 1]'):'~A2+A1',
 ('[3]', '[3]'):'F4(a3)'}

weight=[4,2,2]
drop_weight=3
node_set_1=[list(p) for p in itertools.product(range(drop_weight+1), repeat=len(weight))]
node_set_2=[]
for node in node_set_1:
    c=(-np.sum(np.array(node)*weight)-1)%drop_weight
    node_set_2.append(np.append(node,c))
    if c==0:
        node_set_2.append(np.append(node,drop_weight))

for cover_fold in range(1,2,2):
    node_set_3=[]
    for node in node_set_2:
        if node[0]+node[1]<=cover_fold and node[2]+node[3]<=cover_fold:
            add_point=[]
            for i in range(4):
                if node[i]==0:
                    add_point.append(i)
            if node[0]+node[1]==cover_fold:
                add_point.append(4)
            if node[2]+node[3]==cover_fold:
                add_point.append(5)
            node_set_3.append(add_point)
    
    node_set_3=[list(tpl) for tpl in {tuple(item) for item in node_set_3}]
    minimal_node_set_W2=minimal_W_cy(node_set_3)
    
    dict={0:2,1:3,2:0,3:47}
    minimal_node_set_W1=[]
    for node in minimal_node_set_W2:
        minimal_node_set_W1.append([convert_fuc(W1.roots,W2.roots,Index,dict) for Index in node])

    Target=[]
    Induc_W1cy_W1_set=[]
    Induc_W2cy_W2_set=[]
    W1_cy_special_rep_set=[]
    for i in range(len(minimal_node_set_W1)):
        Phi1_cy=minimal_node_set_W1[i]
        Phi2_cy=minimal_node_set_W2[i]
        W1_cy=reflectionsubgroup(W1,Phi1_cy)
        W2_cy=reflectionsubgroup(W2,Phi2_cy)
        W1_cy_chartable=pd.DataFrame(chartable(W1_cy))
        W1_cy_special_rep=W1_cy_chartable.loc[W1_cy_chartable['b']==W1_cy_chartable['a']]['charnames']
        W1_cy_special_rep_set.append(W1_cy_special_rep)
        induc_W1cy_W1=inductiontable(W1_cy,W1,invchar=(lambda G:chartable(G, chars=False)['b']))
        Induc_W1cy_W1=pd.DataFrame(induc_W1cy_W1['scalar'],columns=induc_W1cy_W1['charH'],index=induc_W1cy_W1['charW'])
        Induc_W1cy_W1_set.append(Induc_W1cy_W1)
        induc_W2cy_W2=inductiontable(W2_cy,W2,invchar=(lambda G:chartable(G, chars=False)['b']))
        Induc_W2cy_W2=pd.DataFrame(induc_W2cy_W2['scalar'],columns=induc_W2cy_W2['charH'],index=induc_W2cy_W2['charW'])
        Induc_W2cy_W2_set.append(Induc_W2cy_W2)
    for char in F4_Data.loc[:,'charnames']:
        print(char)
        j_induction_sign_set=[]
        for i in range(len(minimal_node_set_W1)):
            Phi1_cy=minimal_node_set_W1[i]
            Phi2_cy=minimal_node_set_W2[i]
            W1_cy=reflectionsubgroup(W1,Phi1_cy)
            W2_cy=reflectionsubgroup(W2,Phi2_cy)
            W1_cy_chartable=pd.DataFrame(chartable(W1_cy))
            W1_cy_special_rep=W1_cy_special_rep_set[i]
            Induc_W1cy_W1=Induc_W1cy_W1_set[i]
            Induc_W2cy_W2=Induc_W2cy_W2_set[i]
            for rep_now in W1_cy_special_rep:
                j_induction_now=dual_rep_F4((Induc_W1cy_W1[rep_now][Induc_W1cy_W1[rep_now]==1].index)[0])
                if list((F4_Data.loc[F4_Data['charnames']==str(j_induction_now)]).loc[:,'orbit'])[0] in list((F4_Data.loc[F4_Data['charnames']==char]).loc[:,'Georbit'])[0]:
                    rep_sign_position=(W1_cy_chartable.loc[W1_cy_chartable['charnames']==rep_now]).loc[:,'permsgn'].iloc[0]
                    rep_sign=W1_cy_chartable.iloc[rep_sign_position,0]
                    if rep_sign not in Induc_W2cy_W2.columns:
                        rep_sign=switch_A3_D3(rep_sign,Induc_W2cy_W2.columns)
                    j_induction_sign=(Induc_W2cy_W2[rep_sign][Induc_W2cy_W2[rep_sign]==1].index)[0]
                    j_induction_sign_set.append(j_induction_sign)
        Target.append(max_orbit(list(set([Sat_dict.get(x) for x in set(j_induction_sign_set)])),F4_Data))
    F4_Data["F4_A2+~A2_"+str(cover_fold)]=Target

In [95]:
E8_Data.to_csv("E8_all.csv")

In [12]:
len(W.roots)

112

In [10]:
W=coxeter("D",8)
H=reflectionsubgroup(W,[0,1,2,3,7,6,5,55])
induc_H_W=inductiontable(H,W,invchar=(lambda G:chartable(G, chars=False)['b']))
A=pd.DataFrame(induc_H_W['scalar'],columns=induc_H_W['charH'],index=induc_H_W['charW'])

In [21]:
print(A)

                                 ([[1, 1], +], [[1, 1], +])  \
([[1, 1, 1, 1], +],)                                      0   
([[1, 1, 1, 1], -],)                                      0   
([[1, 1, 1], [1, 1, 1, 1, 1]],)                           0   
([[1, 1], [1, 1, 1, 1, 1, 1]],)                           0   
([[1], [1, 1, 1, 1, 1, 1, 1]],)                           0   
...                                                     ...   
([[2], [6]],)                                             0   
([[], [6, 2]],)                                           0   
([[1], [7]],)                                             0   
([[], [7, 1]],)                                           0   
([[], [8]],)                                              0   

                                 ([[1, 1], +], [[1, 1], -])  \
([[1, 1, 1, 1], +],)                                      0   
([[1, 1, 1, 1], -],)                                      0   
([[1, 1, 1], [1, 1, 1, 1, 1]],)                       

In [31]:
for i in range(len(A[('[[1, 1], +]', '[[2], +]')].index)):
    print(A[('[[2], +]', '[[1, 1], +]')].index[i])
    print(A[('[[2], +]', '[[1, 1], +]')].iloc[i])

('[[1, 1, 1, 1], +]',)
0
('[[1, 1, 1, 1], -]',)
0
('[[1, 1, 1], [1, 1, 1, 1, 1]]',)
0
('[[1, 1], [1, 1, 1, 1, 1, 1]]',)
0
('[[1], [1, 1, 1, 1, 1, 1, 1]]',)
0
('[[], [1, 1, 1, 1, 1, 1, 1, 1]]',)
0
('[[1, 1, 1, 1, 1, 1], [2]]',)
0
('[[1, 1, 1, 1, 1], [2, 1]]',)
0
('[[1, 1, 1, 1], [2, 1, 1]]',)
0
('[[1, 1, 1], [2, 1, 1, 1]]',)
0
('[[1, 1], [2, 1, 1, 1, 1]]',)
0
('[[1], [2, 1, 1, 1, 1, 1]]',)
0
('[[], [2, 1, 1, 1, 1, 1, 1]]',)
0
('[[1, 1, 1, 1], [2, 2]]',)
0
('[[1, 1, 1], [2, 2, 1]]',)
0
('[[2, 1, 1], +]',)
0
('[[2, 1, 1], -]',)
0
('[[1, 1], [2, 2, 1, 1]]',)
0
('[[2, 1], [2, 1, 1, 1]]',)
0
('[[1], [2, 2, 1, 1, 1]]',)
0
('[[2], [2, 1, 1, 1, 1]]',)
0
('[[], [2, 2, 1, 1, 1, 1]]',)
0
('[[2, 1, 1], [2, 2]]',)
0
('[[1, 1], [2, 2, 2]]',)
0
('[[2, 1], [2, 2, 1]]',)
0
('[[1], [2, 2, 2, 1]]',)
0
('[[2], [2, 2, 1, 1]]',)
0
('[[], [2, 2, 2, 1, 1]]',)
0
('[[2, 2], +]',)
0
('[[2, 2], -]',)
0
('[[2], [2, 2, 2]]',)
0
('[[], [2, 2, 2, 2]]',)
0
('[[1, 1, 1, 1, 1], [3]]',)
0
('[[1, 1, 1, 1], [3, 1]]',)
0
('[

In [32]:
for i in range(len(A[('[[1, 1], +]', '[[2], +]')].index)):
    print(A[('[[1, 1], -]', '[[2], -]')].index[i])
    print(A[('[[1, 1], -]', '[[2], -]')].iloc[i])

('[[1, 1, 1, 1], +]',)
0
('[[1, 1, 1, 1], -]',)
0
('[[1, 1, 1], [1, 1, 1, 1, 1]]',)
0
('[[1, 1], [1, 1, 1, 1, 1, 1]]',)
0
('[[1], [1, 1, 1, 1, 1, 1, 1]]',)
0
('[[], [1, 1, 1, 1, 1, 1, 1, 1]]',)
0
('[[1, 1, 1, 1, 1, 1], [2]]',)
0
('[[1, 1, 1, 1, 1], [2, 1]]',)
0
('[[1, 1, 1, 1], [2, 1, 1]]',)
0
('[[1, 1, 1], [2, 1, 1, 1]]',)
0
('[[1, 1], [2, 1, 1, 1, 1]]',)
0
('[[1], [2, 1, 1, 1, 1, 1]]',)
0
('[[], [2, 1, 1, 1, 1, 1, 1]]',)
0
('[[1, 1, 1, 1], [2, 2]]',)
0
('[[1, 1, 1], [2, 2, 1]]',)
0
('[[2, 1, 1], +]',)
0
('[[2, 1, 1], -]',)
0
('[[1, 1], [2, 2, 1, 1]]',)
0
('[[2, 1], [2, 1, 1, 1]]',)
0
('[[1], [2, 2, 1, 1, 1]]',)
0
('[[2], [2, 1, 1, 1, 1]]',)
0
('[[], [2, 2, 1, 1, 1, 1]]',)
0
('[[2, 1, 1], [2, 2]]',)
0
('[[1, 1], [2, 2, 2]]',)
0
('[[2, 1], [2, 2, 1]]',)
0
('[[1], [2, 2, 2, 1]]',)
0
('[[2], [2, 2, 1, 1]]',)
0
('[[], [2, 2, 2, 1, 1]]',)
0
('[[2, 2], +]',)
0
('[[2, 2], -]',)
0
('[[2], [2, 2, 2]]',)
0
('[[], [2, 2, 2, 2]]',)
0
('[[1, 1, 1, 1, 1], [3]]',)
0
('[[1, 1, 1, 1], [3, 1]]',)
0
('[

In [36]:
pd.DataFrame(chartable(H))

,charnames,classnames,classlengths,classreps,b,a,irreducibles,position_id,position_sgn,position_refl,permsgn
0,"([[1, 1], +], [[1, 1], +])","([[1, 1, 1, 1], []], [[1, 1, 1, 1], []])",1,[],12,12,"[9, -3, 9, -3, 3, -3, 9, -3, -3, 0, 0, -3, 3, ...",168,42,False,98
1,"([[1, 1], +], [[1, 1], -])","([[1, 1, 1, 1], []], [[1, 1], [1, 1]])",6,"[4, 7]",12,12,"[9, -3, 9, -3, 3, -3, -3, 9, -3, 0, 0, 3, -3, ...",168,42,False,99
2,"([[1, 1], +], [[1], [1, 1, 1]])","([[1, 1, 1, 1], []], [[], [1, 1, 1, 1]])",1,"[4, 7, 5, 4, 7, 5, 6, 5, 4, 7, 5, 6]",13,13,"[12, 0, -12, -6, 0, 6, 0, 0, 0, 3, -3, 0, 0, -...",168,42,False,101
3,"([[1, 1], +], [[], [1, 1, 1, 1]])","([[1, 1, 1, 1], []], [[2, 1, 1], []])",12,[4],18,18,"[3, 3, 3, -3, -3, -3, 3, 3, 3, 3, 3, -3, -3, -...",168,42,False,103
4,"([[1, 1], +], [[1, 1], [2]])","([[1, 1, 1, 1], []], [[1], [2, 1]])",24,"[4, 7, 5]",10,9,"[18, -6, 18, 0, 0, 0, -6, -6, 6, 0, 0, 0, 0, -...",168,42,False,95
...,...,...,...,...,...,...,...,...,...,...,...
164,"([[], [4]], [[2], -])","([[4], -], [[], [2, 2]])",288,"[1, 2, 3, 4, 5, 4, 7, 5, 6]",2,2,"[3, -1, 3, 1, -1, 1, -1, 3, -1, 0, 0, -1, 1, 3...",168,42,False,40
165,"([[], [4]], [[], [2, 2]])","([[4], -], [[3, 1], []])",768,"[1, 2, 3, 4, 5]",4,3,"[2, 2, 2, 0, 0, 0, 2, 2, 2, -1, -1, 0, 0, 2, 2...",168,42,False,48
166,"([[], [4]], [[1], [3]])","([[4], -], [[], [3, 1]])",768,"[1, 2, 3, 4, 7, 5, 6]",1,1,"[4, 0, -4, 2, 0, -2, 0, 0, 0, 1, -1, 0, 0, 4, ...",168,42,False,41
167,"([[], [4]], [[], [3, 1]])","([[4], -], [[4], +])",576,"[1, 2, 3, 4, 5, 6]",2,2,"[3, 3, 3, 1, 1, 1, -1, -1, -1, 0, 0, -1, -1, 3...",168,42,False,45


In [38]:
pd.DataFrame(chartable(coxeter("D",6)))

,charnames,classnames,classlengths,classreps,b,a,irreducibles,position_id,position_sgn,position_refl,permsgn
0,"([[1, 1, 1], +],)","([[1, 1, 1, 1, 1, 1], []],)",1,[],15,15,"[10, -2, 2, -10, -4, 2, 0, -2, 4, 2, -2, 0, -2...",36,4,34,27
1,"([[1, 1, 1], -],)","([[1, 1, 1, 1], [1, 1]],)",15,"[0, 1]",15,15,"[10, -2, 2, -10, -4, 2, 0, -2, 4, 2, -2, 0, -2...",36,4,34,26
2,"([[1, 1], [1, 1, 1, 1]],)","([[1, 1], [1, 1, 1, 1]],)",15,"[0, 1, 2, 0, 1, 2, 3, 2, 0, 1, 2, 3]",16,16,"[15, -1, -1, 15, -7, 1, 1, 1, -7, 3, -1, -1, 3...",36,4,34,32
3,"([[1], [1, 1, 1, 1, 1]],)","([[], [1, 1, 1, 1, 1, 1]],)",1,"[0, 1, 2, 0, 1, 2, 3, 2, 0, 1, 2, 3, 4, 3, 2, ...",21,21,"[6, 2, -2, -6, -4, -2, 0, 2, 4, 2, 2, 0, -2, -...",36,4,34,34
4,"([[], [1, 1, 1, 1, 1, 1]],)","([[2, 1, 1, 1, 1], []],)",30,[0],30,30,"[1, 1, 1, 1, -1, -1, -1, -1, -1, 1, 1, 1, 1, 1...",36,4,34,36
5,"([[1, 1, 1, 1], [2]],)","([[1, 1, 1], [2, 1]],)",120,"[0, 1, 2]",14,13,"[15, -1, -1, 15, -5, -1, 3, -1, -5, -1, 3, -1,...",36,4,34,29
6,"([[1, 1, 1], [2, 1]],)","([[2, 1, 1], [1, 1]],)",180,"[0, 1, 3]",11,10,"[40, -8, 8, -40, -8, 4, 0, -4, 8, 0, 0, 0, 0, ...",36,4,34,22
7,"([[1, 1], [2, 1, 1]],)","([[1], [2, 1, 1, 1]],)",120,"[0, 1, 2, 0, 1, 2, 3, 2, 0, 1, 2, 3, 4]",10,10,"[45, -3, -3, 45, -9, 3, -1, 3, -9, 1, -3, 1, 1...",36,4,34,24
8,"([[1], [2, 1, 1, 1]],)","([[2], [1, 1, 1, 1]],)",30,"[0, 1, 2, 0, 1, 2, 3, 2, 0, 1, 2, 3, 5]",13,13,"[24, 8, -8, -24, -8, -4, 0, 4, 8, 0, 0, 0, 0, ...",36,4,34,30
9,"([[], [2, 1, 1, 1, 1]],)","([[2, 2, 1, 1], []],)",180,"[0, 3]",20,20,"[5, 5, 5, 5, -3, -3, -3, -3, -3, 1, 1, 1, 1, 1...",36,4,34,35


In [39]:
W=coxeter("D",6)
H=reflectionsubgroup(W,[0,2,3,4,5])
induc_H_W=inductiontable(H,W,invchar=(lambda G:chartable(G, chars=False)['b']))
B=pd.DataFrame(induc_H_W['scalar'],columns=induc_H_W['charH'],index=induc_H_W['charW'])

In [40]:
B

,"([1, 1, 1, 1, 1, 1],)","([2, 1, 1, 1, 1],)","([2, 2, 1, 1],)","([2, 2, 2],)","([3, 1, 1, 1],)","([3, 2, 1],)","([3, 3],)","([4, 1, 1],)","([4, 2],)","([5, 1],)","([6],)"
"([[1, 1, 1], +],)",1,0,0,0,0,0,0,0,0,0,0
"([[1, 1, 1], -],)",0,0,0,0,0,0,0,0,0,0,0
"([[1, 1], [1, 1, 1, 1]],)",0,0,0,0,0,0,0,0,0,0,0
"([[1], [1, 1, 1, 1, 1]],)",0,0,0,0,0,0,0,0,0,0,0
"([[], [1, 1, 1, 1, 1, 1]],)",0,0,0,0,0,0,0,0,0,0,0
"([[1, 1, 1, 1], [2]],)",0,0,0,0,0,0,0,0,0,0,0
"([[1, 1, 1], [2, 1]],)",0,0,0,0,0,0,0,0,0,0,0
"([[1, 1], [2, 1, 1]],)",0,1,0,0,0,0,0,0,0,0,0
"([[1], [2, 1, 1, 1]],)",0,0,0,0,0,0,0,0,0,0,0
"([[], [2, 1, 1, 1, 1]],)",0,0,0,0,0,0,0,0,0,0,0


In [43]:
W=coxeter("D",6)
H=reflectionsubgroup(W,[1,3,5])
induc_H_W=inductiontable(H,W,invchar=(lambda G:chartable(G, chars=False)['b']))
C=pd.DataFrame(induc_H_W['scalar'],columns=induc_H_W['charH'],index=induc_H_W['charW'])

In [44]:
C

,"([1, 1], [1, 1], [1, 1])","([1, 1], [1, 1], [2])","([1, 1], [2], [1, 1])","([1, 1], [2], [2])","([2], [1, 1], [1, 1])","([2], [1, 1], [2])","([2], [2], [1, 1])","([2], [2], [2])"
"([[1, 1, 1], +],)",0,0,0,0,0,0,0,0
"([[1, 1, 1], -],)",0,0,0,0,0,0,0,0
"([[1, 1], [1, 1, 1, 1]],)",0,0,0,0,0,0,0,0
"([[1], [1, 1, 1, 1, 1]],)",0,0,0,0,0,0,0,0
"([[], [1, 1, 1, 1, 1, 1]],)",0,0,0,0,0,0,0,0
"([[1, 1, 1, 1], [2]],)",0,0,0,0,0,0,0,0
"([[1, 1, 1], [2, 1]],)",0,0,0,0,0,0,0,0
"([[1, 1], [2, 1, 1]],)",0,0,0,0,0,0,0,0
"([[1], [2, 1, 1, 1]],)",0,0,0,0,0,0,0,0
"([[], [2, 1, 1, 1, 1]],)",0,0,0,0,0,0,0,0
